In [1]:
from oauth2client.service_account import ServiceAccountCredentials
import gspread
import numpy as np

scope = ["https://www.googleapis.com/auth/spreadsheets"] 

credentials = ServiceAccountCredentials.from_json_keyfile_name('dev-poet-352108-e21771e52eaa.json', scope)
gc = gspread.authorize(credentials)

sheet = gc.open_by_url("https://docs.google.com/spreadsheets/d/16bKAdRGWE-JFjMGxyB2W5i1fffJDFfKEy3GKaTpSzGI/edit?resourcekey#gid=402671462")
worksheet = sheet.get_worksheet(0) # sheet index in spreadsheets

In [2]:
import pandas as pd

Data = pd.DataFrame(worksheet.get_all_records())
Data
Data.columns

Index(['ประทับเวลา',
       'หลังจากท่านได้ทราบถึงภาพรวมและวัตถุประสงค์ของแบบสอบถามฉบับนี้แล้ว ท่านยินดีตอบแบบสอบถามฉบับนี้หรือไม่',
       'ประเภทอุตสาหกรรมหลักของสถานประกอบการ ',
       'กิจการของท่านอยู่ภายใต้การกำกับของหน่วยงานใดบ้าง (สามารถตอบได้มากกว่าหนึ่งข้อ)',
       'รายได้ต่อปีของสถานประกอบการ ในปี พ.ศ. 2562 (ก่อนวิกฤตโควิด)',
       'จำนวนลูกจ้างในสถานประกอบการ',
       'ในกรณีที่จำเป็น ท่านยินดีให้ผู้วิจัย (สถาบันวิจัยเพื่อการพัฒนาประเทศไทย) ติดต่อท่านอีกครั้งเพื่อรวบรวมข้อมูลเพิ่มเติมจากข้อมูลที่ได้ท่านให้ไว้หรือไม่ ',
       'เบอร์โทรศัพท์',
       'สิ่งใดเป็นความผิดตามพระราชบัญญัติการแข่งขันทางการค้า พ.ศ. 2560',
       'ลักษณะใดเป็นเกณฑ์ของการเป็นผู้มีอำนาจเหนือตลาดตามพระราชบัญญัติการแข่งขันทางการค้า พ.ศ. 2560',
       'ค่าปรับสำหรับการกระทำผิดตามพระราชบัญญัติการแข่งขันทางการค้า พ.ศ. 2560 ขึ้นกับปัจจัยอะไรบ้าง',
       'การกระทำผิดในข้อใดหากฝ่าฝืนแล้วมีโทษจำคุก ',
       'การรวมธุรกิจลักษณะใดต้องได้รับอนุญาตจากคณะกรรมการการแข่งขันทางการค้า',
       'ข้อใดคือที่อยู่ของสำ

In [3]:
col_income_2562 = 'รายได้ต่อปีของสถานประกอบการ ในปี พ.ศ. 2562 (ก่อนวิกฤตโควิด)' # ชื่อคอลัมน์
col_unique = list(Data[col_income_2562].unique()) # ['11 – 100 ล้านบาท', 'ตั้งแต่ 100 ล้านบาทขึ้นไป', nan, '1 – 10 ล้านบาท']
col_unique

['11 – 100 ล้านบาท', 'ตั้งแต่ 100 ล้านบาทขึ้นไป', '', '1 – 10 ล้านบาท']

In [4]:
# Alternative 1
mapper = {'ตั้งแต่ 100 ล้านบาทขึ้นไป': "Big Corp",
          '1 – 10 ล้านบาท': "SME",
          '11 – 100 ล้านบาท': "SME"} # สร้าง mapper สำหรับการแปลงข้อมูลราคาเป็นหมวดหมู่
col_corp_category = 'ประเภทสถานประกอบการ ในปี พ.ศ. 2562 (ก่อนวิกฤตโควิด)' # ชื่อคอลัมน์ใหม่
Data[col_corp_category] = Data[col_income_2562].map(mapper) # สร้างคอลัมน์ใหม่ใน dataframe ด้วยค่าที่กำหนดไว้ใน mapper จากค่าที่กำหนดไว้ใน col_income_2562

In [7]:
# Alternative 2
conditions = [
(Data[col_income_2562] == '1 – 10 ล้านบาท'),
(Data[col_income_2562] == '11 – 100 ล้านบาท'),
(Data[col_income_2562] == 'ตั้งแต่ 100 ล้านบาทขึ้นไป'),]
choices = ["SME","SME","Big Corp"]
Data[col_corp_category] = np.select(conditions, choices, default="Big Corp")

In [8]:
income = pd.DataFrame(Data[col_corp_category].value_counts()) # สร้าง dataframe จาก ข้อมูลประเภทสถานประกอบการ ในปี พ.ศ. 2562 (ก่อนวิกฤตโควิด)
income

,ประเภทสถานประกอบการ ในปี พ.ศ. 2562 (ก่อนวิกฤตโควิด)
Big Corp,22
SME,13


## Knowledge

In [9]:
# create variable for score of responsdent (Q_sum) 
import numpy as np

Q1 = 'สิ่งใดเป็นความผิดตามพระราชบัญญัติการแข่งขันทางการค้า พ.ศ. 2560'
Data['Q1_score'] = 0
Correct_choice_num = 3
Correct_pt = 1/Correct_choice_num
Mistake_pt = (-1/2)*Correct_pt
correct_response = ['การใช้อำนาจเหนือตลาดอย่างไม่เป็นธรรม','การตกลงร่วมกัน ผูกขาด ลด หรือจำกัดการแข่งขัน','กำหนดเงื่อนไขทางการค้าอันเป็นการจำกัดหรือขัดขวางการประกอบธุรกิจของผู้อื่นอย่างไม่เป็นธรรม']
for ans in correct_response:
    Data.loc[Data[Q1].str.contains(ans), 'Q1_score'] += Correct_pt
wrong_response = ['กระทำการรวมธุรกิจอันอาจก่อให้เกิดการลดการแข่งขันอย่างมีนัยสำคัญในตลาด']
for ans in wrong_response:
    Data.loc[Data[Q1].str.contains(ans), 'Q1_score'] += Mistake_pt
Data['Q1_score']

#เนื่องจากให้ตอบได้ข้อเดียวจึงให้คะแนนเป็น 0 1 
Q2 = 'ลักษณะใดเป็นเกณฑ์ของการเป็นผู้มีอำนาจเหนือตลาดตามพระราชบัญญัติการแข่งขันทางการค้า พ.ศ. 2560'
Data['Q2_score'] = 0
Correct_choice_num = 1
Correct_pt = 1/Correct_choice_num
Mistake_pt = (-1/2)*Correct_pt
correct_response = ['ผู้ประกอบธุรกิจรายใดรายหนึ่งที่มีส่วนแบ่งตลาดในปีที่ผ่านมาตั้งแต่ 50 % ขึ้นไป และ มียอดเงินขายในปีที่ผ่านมาตั้งแต่ 1,000 ล้านบาทขึ้นไป หรือ ผู้ประกอบธุรกิจ 3 รายแรกที่มีส่วนแบ่งตลาดในปีที่ผ่านมาตั้งแต่ 75% ขึ้นไป']
for ans in correct_response:
    Data.loc[Data[Q2]== ans, 'Q2_score'] = 1
Data['Q2_score']

Q3 = 'ค่าปรับสำหรับการกระทำผิดตามพระราชบัญญัติการแข่งขันทางการค้า พ.ศ. 2560 ขึ้นกับปัจจัยอะไรบ้าง'
Data['Q3_score'] = 0
Correct_choice_num = 2
Correct_pt = 1/Correct_choice_num
Mistake_pt = (-1/2)*Correct_pt
correct_response = ['รายได้ในปีที่กระทำความผิด','มูลค่าธุรกรรมในการรวมธุรกิจ']
for ans in correct_response:
    Data.loc[Data[Q3].str.contains(ans), 'Q3_score'] += Correct_pt
wrong_response = ['กำไรในปีที่กระทำความผิด']
for ans in wrong_response:
    Data.loc[Data[Q3].str.contains(ans), 'Q3_score'] += Mistake_pt
Data['Q3_score']

Q4 = 'การกระทำผิดในข้อใดหากฝ่าฝืนแล้วมีโทษจำคุก '
Data['Q4_score'] = 0
Correct_choice_num = 2
Correct_pt = 1/Correct_choice_num
Mistake_pt = (-1/2)*Correct_pt
correct_response = ['การใช้อำนาจเหนือตลาดอย่างไม่เป็นธรรม','การตกลงร่วมกัน ผูกขาด ลด หรือจำกัดการแข่งขัน']
for ans in correct_response:
    Data.loc[Data[Q4].str.contains(ans), 'Q4_score'] += Correct_pt
wrong_response = ['กระทำการรวมธุรกิจอันอาจก่อให้เกิดการลดการแข่งขันอย่างมีนัยสำคัญในตลาด','กำหนดเงื่อนไขทางการค้าอันเป็นการจำกัดหรือขัดขวางการประกอบธุรกิจของผู้อื่นอย่างไม่เป็นธรรม']
for ans in wrong_response:
    Data.loc[(Data[Q4].str.contains(ans)), 'Q4_score'] += Mistake_pt
Data.loc[(Data['Q4_score']<=0),'Q4_score'] = 0
Data['Q4_score']

Q5 = 'การรวมธุรกิจลักษณะใดต้องได้รับอนุญาตจากคณะกรรมการการแข่งขันทางการค้า'
Data['Q5_score'] = 0
Correct_choice_num = 1
Correct_pt = 1/Correct_choice_num
Mistake_pt = (-1/2)*Correct_pt
correct_response = ['การรวมธุรกิจอันอาจก่อให้เกิดการผูกขาดหรือการเป็นผู้ประกอบธุรกิจซึ่งมีอำนาจเหนือตลาด']
for ans in correct_response:
    Data.loc[Data[Q5].str.contains(ans), 'Q5_score'] += Correct_pt
wrong_response = ['การรวมธุรกิจอันอาจก่อให้เกิดอำนาจสั่งการ','การรวมธุรกิจอันอาจก่อให้เกิดการลดการแข่งขันอย่างมีนัยสำคัญในตลาดใดตลาดหนึ่ง','การรวมธุรกิจเพื่อปรับโครงสร้างภายในของผู้ประกอบธุรกิจที่มีความสัมพันธ์กันทางนโยบายหรืออำนาจสั่งการ']
for ans in wrong_response:
    Data.loc[(Data[Q5].str.contains(ans)), 'Q5_score'] += Mistake_pt
Data.loc[(Data['Q5_score']<=0),'Q5_score'] = 0
Data['Q5_score']

#เนื่องจากให้ตอบได้ข้อเดียวจึงให้คะแนนเป็น 0 1 
Q6 = 'ข้อใดคือที่อยู่ของสำนักงานคณะกรรมการการแข่งขันทางการค้า'
Data['Q6_score'] = 0
Correct_choice_num = 1
Correct_pt = 1/Correct_choice_num
Mistake_pt = (-1/2)*Correct_pt
correct_response = ['ศูนย์ราชการเฉลิมพระเกียรติ 80 พรรษา เลขที่ 120 ถนนแจ้งวัฒนะ แขวงทุ่งสองห้อง เขตหลักสี่ กรุงเทพมหานคร 10210']
for ans in correct_response:
    Data.loc[Data[Q6]== ans, 'Q6_score'] = 1
Data['Q6_score']

Data['Q_sum'] = Data[['Q1_score','Q2_score','Q3_score','Q4_score','Q5_score','Q6_score']].sum(axis=1)

print("Percentage of correct answer:", sum(Data['Q_sum'])/(6*len(Data.index))*100,"%" )
#Data.groupby(['Q_sum'])['ประทับเวลา'].count()/sum(Data.groupby(['Q_sum'])['ประทับเวลา'].count())

Percentage of correct answer: 59.1269841269841 %


In [10]:
# Show the distribution of correct respondents seperated by question answered 
n=sum(Data[['Q2_score']].count())
print("Respondents_num:",n)
Data[['Q1_score','Q2_score','Q3_score','Q4_score','Q5_score','Q6_score']].sum()/n

Respondents_num: 35


Q1_score    0.747619
Q2_score    0.542857
Q3_score    0.585714
Q4_score    0.471429
Q5_score    0.485714
Q6_score    0.714286
dtype: float64

In [11]:
# Show the distribution of correct respondents seperated by question answered 
Knowledge_q = Data.groupby([col_corp_category]).agg(Q1_score=('Q1_score', np.sum),Q2_score=('Q2_score', np.sum)
                                                    ,Q3_score=('Q3_score', np.sum),Q4_score=('Q4_score', np.sum)
                                                    ,Q5_score=('Q5_score', np.sum),Q6_score=('Q6_score', np.sum)
                                                    ,Total_response=('ประทับเวลา','count'))

Knowledge_q_proportion = Knowledge_q.iloc[:,:-1].div(Knowledge_q.Total_response, axis=0)
Knowledge_q_proportion = pd.merge(Knowledge_q_proportion,Knowledge_q['Total_response'],left_index=True, right_index=True)
Knowledge_q_proportion

,Q1_score,Q2_score,Q3_score,Q4_score,Q5_score,Q6_score,Total_response
ประเภทสถานประกอบการ ในปี พ.ศ. 2562 (ก่อนวิกฤตโควิด),,,,,,,
Big Corp,0.727273,0.681818,0.636364,0.511364,0.590909,0.818182,22
SME,0.782051,0.307692,0.500000,0.403846,0.307692,0.538462,13


In [10]:
# low score and high score variable
conditions = [
(Data['Q_sum'] < 3),
(Data['Q_sum'] >= 3)]
choices = ["low","high"]
Data['Q_category'] = np.select(conditions, choices, default=np.nan)

# Show the distribution of correct respondents seperated by question answered 
Data.groupby('Q_category')['ประทับเวลา'].count()

Q_category
high    21
low     14
Name: ประทับเวลา, dtype: int64

In [95]:
Data['Q1_score'] = np.where(Con_Q1, 1, 0)
Data['Q2_score'] = np.where(Con_Q2, 1, 0)
Data['Q3_score'] = np.where(Con_Q3, 1, 0)
Data['Q4_score'] = np.where(Con_Q4, 1, 0)
Data['Q5_score'] = np.where(Con_Q5, 1, 0)
Data['Q6_score'] = np.where(Con_Q6, 1, 0)
Data['Q_sum'] = Data[['Q2_score','Q3_score','Q4_score','Q5_score','Q6_score']].sum(axis=1)

print("Percentage of correct:", sum(Data['Q_sum'])/(5*len(Data.index))*100,"%" )
pd.DataFrame(Data.groupby(['Q_sum'])['ประทับเวลา'].count()/sum(Data.groupby(['Q_sum'])['ประทับเวลา'].count()))


Percentage of correct: 42.285714285714285 %


,ประทับเวลา
Q_sum,
0,0.228571
1,0.142857
2,0.257143
3,0.171429
4,0.057143
5,0.142857


In [12]:
Data['Q_proportion'] = Data['Q_sum']/6*100 # คำนวณอัตราตอบถูกของแต่ละผู้ประกอบการ
categories = ['ร้อยละ 0-20', 'ร้อยละ 21-40', 'ร้อยละ 41-60', 'ร้อยละ 61-80', 'ร้อยละ 81-100'] # กำหนดหมวดหมู่คะแนน
def categorize_score(x): # สร้างฟังก์ชันจำแนกหนวดหมู่คะแนน
    if x >= 81: 
        return categories[-1]
    elif x >= 61:
        return categories[-2]
    elif x >= 41:
        return categories[-3]
    elif x >= 21:
        return categories[-4]
    else:
        return categories[0]
Data['Q_proportion_category'] = Data['Q_proportion'].apply(categorize_score) # คำนวณอัตราตอบถูกของแต่ละผู้ประกอบการแบบแยกหมวดหมู่
Data['Q_proportion_category'].head()

0     ร้อยละ 61-80
1     ร้อยละ 41-60
2    ร้อยละ 81-100
3     ร้อยละ 61-80
4     ร้อยละ 61-80
Name: Q_proportion_category, dtype: object

In [13]:
cat_row = Data[col_corp_category]
cat_col = Data['Q_proportion_category']
tabel = pd.crosstab(cat_row, cat_col, normalize='index') # สร้าง crosstab ของหน่วยงานที่ประกอบการเทียบกับสัดส่วนคะแนน
tabel
# %%

Q_proportion_category,ร้อยละ 21-40,ร้อยละ 41-60,ร้อยละ 61-80,ร้อยละ 81-100
ประเภทสถานประกอบการ ในปี พ.ศ. 2562 (ก่อนวิกฤตโควิด),,,,
Big Corp,0.227273,0.136364,0.409091,0.227273
SME,0.538462,0.153846,0.230769,0.076923


In [14]:
Knowledge_q_proportion

,Q1_score,Q2_score,Q3_score,Q4_score,Q5_score,Q6_score,Total_response
ประเภทสถานประกอบการ ในปี พ.ศ. 2562 (ก่อนวิกฤตโควิด),,,,,,,
Big Corp,0.727273,0.681818,0.636364,0.511364,0.590909,0.818182,22
SME,0.782051,0.307692,0.500000,0.403846,0.307692,0.538462,13


## Practice

### การคำนวณ Distribution

In [13]:
### Complete Codes for Computing Distribution _ Merger 

TempValueCount = pd.DataFrame(Data['1.3 ในจำนวนแผนการรวมธุรกิจและการรวมธุรกิจทั้งหมดที่ท่านได้ระบุในข้อ 1.1. มีแผนการรวมธุรกิจและการรวมธุรกิจที่ถูกยกเลิกหรือปรับเปลี่ยนไปแล้ว เนื่องจากเหตุผลในตารางต่อไปนี้ เป็นสัดส่วนมากน้อยเพียงใด (ประมาณการ)  [ผู้ประกอบการเกรงว่าจะไม่ได้รับอนุญาตหรือจะได้รับการตั้งเงื่อนไขการรวมธุรกิจที่เข้มงวดจากคณะกรรมการการแข่งขันทางการค้า]'].value_counts()).transpose()

## โค้ดสำหรับตัดคนที่ไม่ทราบออก
# โดยหลังการ Value Count ก็ตัด column สุดท้ายออก
TempValueCount_Excluded = TempValueCount.loc[:, TempValueCount.columns!='ไม่ทราบ/ไม่แน่ใจ']

# คำนวณเป็น Proportion
Merger_Deter_Propotion = (TempValueCount_Excluded/TempValueCount_Excluded.sum(1)[0])*100

Merger_Deter_Propotion

,ร้อยละ 0-20,ร้อยละ 21-40,ร้อยละ 61-80,ร้อยละ 41-60
1.3 ในจำนวนแผนการรวมธุรกิจและการรวมธุรกิจทั้งหมดที่ท่านได้ระบุในข้อ 1.1. มีแผนการรวมธุรกิจและการรวมธุรกิจที่ถูกยกเลิกหรือปรับเปลี่ยนไปแล้ว เนื่องจากเหตุผลในตารางต่อไปนี้ เป็นสัดส่วนมากน้อยเพียงใด (ประมาณการ) [ผู้ประกอบการเกรงว่าจะไม่ได้รับอนุญาตหรือจะได้รับการตั้งเงื่อนไขการรวมธุรกิจที่เข้มงวดจากคณะกรรมการการแข่งขันทางการค้า],46.666667,26.666667,13.333333,13.333333


In [35]:
cross_tab_Merger_Deter = pd.crosstab(index=Data['รายได้ต่อปีของสถานประกอบการ ในปี พ.ศ. 2562 (ก่อนวิกฤตโควิด)'],
                             columns=Data['1.3 ในจำนวนแผนการรวมธุรกิจและการรวมธุรกิจทั้งหมดที่ท่านได้ระบุในข้อ 1.1. มีแผนการรวมธุรกิจและการรวมธุรกิจที่ถูกยกเลิกหรือปรับเปลี่ยนไปแล้ว เนื่องจากเหตุผลในตารางต่อไปนี้ เป็นสัดส่วนมากน้อยเพียงใด (ประมาณการ)  [ผู้ประกอบการเกรงว่าจะไม่ได้รับอนุญาตหรือจะได้รับการตั้งเงื่อนไขการรวมธุรกิจที่เข้มงวดจากคณะกรรมการการแข่งขันทางการค้า]'],
                             normalize="index")

cross_tab_Merger_Deter 

1.3 ในจำนวนแผนการรวมธุรกิจและการรวมธุรกิจทั้งหมดที่ท่านได้ระบุในข้อ 1.1. มีแผนการรวมธุรกิจและการรวมธุรกิจที่ถูกยกเลิกหรือปรับเปลี่ยนไปแล้ว เนื่องจากเหตุผลในตารางต่อไปนี้ เป็นสัดส่วนมากน้อยเพียงใด (ประมาณการ) [ผู้ประกอบการเกรงว่าจะไม่ได้รับอนุญาตหรือจะได้รับการตั้งเงื่อนไขการรวมธุรกิจที่เข้มงวดจากคณะกรรมการการแข่งขันทางการค้า],ร้อยละ 0-20,ร้อยละ 21-40,ร้อยละ 41-60,ร้อยละ 61-80,ไม่ทราบ/ไม่แน่ใจ
รายได้ต่อปีของสถานประกอบการ ในปี พ.ศ. 2562 (ก่อนวิกฤตโควิด),,,,,
,1.000,0.000,0.0,0.000,0.000
1 – 10 ล้านบาท,0.125,0.125,0.0,0.125,0.625
11 – 100 ล้านบาท,0.200,0.200,0.0,0.000,0.600
ตั้งแต่ 100 ล้านบาทขึ้นไป,0.150,0.100,0.1,0.050,0.600


In [37]:
### Complete Codes for Computing Distribution _ Abuse of Dominance 

TempValueCount = pd.DataFrame(Data['2.3 จากข้อมูลที่ท่านทราบ ในจำนวนกิจกรรมหรือแผนกิจกรรมที่อาจเข้าข่ายเป็นการกระทำผิดฐานใช้อำนาจเหนือตลาดอย่างไม่เป็นธรรม ทั้งหมดที่ท่านระบุในข้อ 2.1 มีกิจกรรมหรือแผนกิจกรรมที่ถูกยกเลิกหรือปรับเปลี่ยนไปแล้ว เนื่องจากเหตุผลในตารางต่อไปนี้ เป็นสัดส่วนมากน้อยเพียงใด (ประมาณการ) [ผู้ประกอบการต้องการป้องกันความเสี่ยงที่อาจจะถูกสอบสวนสืบสวนโดย สำนักงาน กขค. และ/หรือถูกลงโทษตาม พ.ร.บ การแข่งขันทางการค้า]'].value_counts()).transpose()

## โค้ดสำหรับตัดคนที่ไม่ทราบออก
# โดยหลังการ Value Count ก็ตัด column สุดท้ายออก
TempValueCount_Excluded = TempValueCount.loc[:, TempValueCount.columns!='ไม่ทราบ/ไม่แน่ใจ']

# คำนวณเป็น Proportion
Abuse_Deter_Propotion = (TempValueCount_Excluded/TempValueCount_Excluded.sum(1)[0])*100

Abuse_Deter_Propotion

,ร้อยละ 0-20,ร้อยละ 61-80,ร้อยละ 21-40,ร้อยละ 41-60
2.3 จากข้อมูลที่ท่านทราบ ในจำนวนกิจกรรมหรือแผนกิจกรรมที่อาจเข้าข่ายเป็นการกระทำผิดฐานใช้อำนาจเหนือตลาดอย่างไม่เป็นธรรม ทั้งหมดที่ท่านระบุในข้อ 2.1 มีกิจกรรมหรือแผนกิจกรรมที่ถูกยกเลิกหรือปรับเปลี่ยนไปแล้ว เนื่องจากเหตุผลในตารางต่อไปนี้ เป็นสัดส่วนมากน้อยเพียงใด (ประมาณการ) [ผู้ประกอบการต้องการป้องกันความเสี่ยงที่อาจจะถูกสอบสวนสืบสวนโดย สำนักงาน กขค. และ/หรือถูกลงโทษตาม พ.ร.บ การแข่งขันทางการค้า],53.846154,15.384615,15.384615,15.384615


In [38]:
cross_tab_Abuse_Deter = pd.crosstab(index=Data['รายได้ต่อปีของสถานประกอบการ ในปี พ.ศ. 2562 (ก่อนวิกฤตโควิด)'],
                             columns=Data['2.3 จากข้อมูลที่ท่านทราบ ในจำนวนกิจกรรมหรือแผนกิจกรรมที่อาจเข้าข่ายเป็นการกระทำผิดฐานใช้อำนาจเหนือตลาดอย่างไม่เป็นธรรม ทั้งหมดที่ท่านระบุในข้อ 2.1 มีกิจกรรมหรือแผนกิจกรรมที่ถูกยกเลิกหรือปรับเปลี่ยนไปแล้ว เนื่องจากเหตุผลในตารางต่อไปนี้ เป็นสัดส่วนมากน้อยเพียงใด (ประมาณการ) [ผู้ประกอบการต้องการป้องกันความเสี่ยงที่อาจจะถูกสอบสวนสืบสวนโดย สำนักงาน กขค. และ/หรือถูกลงโทษตาม พ.ร.บ การแข่งขันทางการค้า]'],
                             normalize="index")

cross_tab_Abuse_Deter 

2.3 จากข้อมูลที่ท่านทราบ ในจำนวนกิจกรรมหรือแผนกิจกรรมที่อาจเข้าข่ายเป็นการกระทำผิดฐานใช้อำนาจเหนือตลาดอย่างไม่เป็นธรรม ทั้งหมดที่ท่านระบุในข้อ 2.1 มีกิจกรรมหรือแผนกิจกรรมที่ถูกยกเลิกหรือปรับเปลี่ยนไปแล้ว เนื่องจากเหตุผลในตารางต่อไปนี้ เป็นสัดส่วนมากน้อยเพียงใด (ประมาณการ) [ผู้ประกอบการต้องการป้องกันความเสี่ยงที่อาจจะถูกสอบสวนสืบสวนโดย สำนักงาน กขค. และ/หรือถูกลงโทษตาม พ.ร.บ การแข่งขันทางการค้า],ร้อยละ 0-20,ร้อยละ 21-40,ร้อยละ 41-60,ร้อยละ 61-80,ไม่ทราบ/ไม่แน่ใจ
รายได้ต่อปีของสถานประกอบการ ในปี พ.ศ. 2562 (ก่อนวิกฤตโควิด),,,,,
,1.000,0.000,0.0,0.0,0.00
1 – 10 ล้านบาท,0.125,0.125,0.0,0.0,0.75
11 – 100 ล้านบาท,0.200,0.200,0.0,0.0,0.60
ตั้งแต่ 100 ล้านบาทขึ้นไป,0.150,0.000,0.1,0.1,0.65


In [ ]:
income = pd.DataFrame(Data['รายได้ต่อปีของสถานประกอบการ ในปี พ.ศ. 2562 (ก่อนวิกฤตโควิด)'].value_counts())
income

In [39]:
### Complete Codes for Computing Distribution _ Cartels

TempValueCount = pd.DataFrame(Data['3.3 จากข้อมูลที่ท่านทราบ ในจำนวนความร่วมมือหรือแผนความร่วมมือกับผู้ประกอบการรายอื่นที่อาจเข้าข่ายเป็นการกระทำผิดฐานตกลงร่วมกัน ผูกขาด ลด และจำกัดการแข่งขัน ทั้งหมดที่ท่านระบุในข้อ 3.1 มีความร่วมมือหรือแผนความร่วมมือที่ถูกยกเลิกหรือปรับเปลี่ยนไปแล้ว เนื่องจากเหตุผลในตารางต่อไปนี้ เป็นสัดส่วนมากน้อยเพียงใด (ประมาณการ) [ผู้ประกอบการต้องการป้องกันความเสี่ยงที่อาจจะถูกสอบสวนสืบสวนโดย สำนักงาน กขค. และ/หรือถูกลงโทษตาม พ.ร.บ การแข่งขันทางการค้า]'].value_counts()).transpose()

TempValueCount_Excluded = TempValueCount.loc[:, TempValueCount.columns!='ไม่ทราบ/ไม่แน่ใจ']

Cartels_Deter_Propotion = (TempValueCount_Excluded/TempValueCount_Excluded.sum(1)[0])*100

Cartels_Deter_Propotion

,ร้อยละ 0-20,ร้อยละ 21-40,ร้อยละ 61-80,ร้อยละ 81-100,ร้อยละ 41-60
3.3 จากข้อมูลที่ท่านทราบ ในจำนวนความร่วมมือหรือแผนความร่วมมือกับผู้ประกอบการรายอื่นที่อาจเข้าข่ายเป็นการกระทำผิดฐานตกลงร่วมกัน ผูกขาด ลด และจำกัดการแข่งขัน ทั้งหมดที่ท่านระบุในข้อ 3.1 มีความร่วมมือหรือแผนความร่วมมือที่ถูกยกเลิกหรือปรับเปลี่ยนไปแล้ว เนื่องจากเหตุผลในตารางต่อไปนี้ เป็นสัดส่วนมากน้อยเพียงใด (ประมาณการ) [ผู้ประกอบการต้องการป้องกันความเสี่ยงที่อาจจะถูกสอบสวนสืบสวนโดย สำนักงาน กขค. และ/หรือถูกลงโทษตาม พ.ร.บ การแข่งขันทางการค้า],53.846154,23.076923,7.692308,7.692308,7.692308


In [42]:
cross_tab_Cartels_Deter = pd.crosstab(index=Data['รายได้ต่อปีของสถานประกอบการ ในปี พ.ศ. 2562 (ก่อนวิกฤตโควิด)'],
                             columns=Data['3.3 จากข้อมูลที่ท่านทราบ ในจำนวนความร่วมมือหรือแผนความร่วมมือกับผู้ประกอบการรายอื่นที่อาจเข้าข่ายเป็นการกระทำผิดฐานตกลงร่วมกัน ผูกขาด ลด และจำกัดการแข่งขัน ทั้งหมดที่ท่านระบุในข้อ 3.1 มีความร่วมมือหรือแผนความร่วมมือที่ถูกยกเลิกหรือปรับเปลี่ยนไปแล้ว เนื่องจากเหตุผลในตารางต่อไปนี้ เป็นสัดส่วนมากน้อยเพียงใด (ประมาณการ) [ผู้ประกอบการต้องการป้องกันความเสี่ยงที่อาจจะถูกสอบสวนสืบสวนโดย สำนักงาน กขค. และ/หรือถูกลงโทษตาม พ.ร.บ การแข่งขันทางการค้า]'],
                             normalize="index")

cross_tab_Cartels_Deter 

3.3 จากข้อมูลที่ท่านทราบ ในจำนวนความร่วมมือหรือแผนความร่วมมือกับผู้ประกอบการรายอื่นที่อาจเข้าข่ายเป็นการกระทำผิดฐานตกลงร่วมกัน ผูกขาด ลด และจำกัดการแข่งขัน ทั้งหมดที่ท่านระบุในข้อ 3.1 มีความร่วมมือหรือแผนความร่วมมือที่ถูกยกเลิกหรือปรับเปลี่ยนไปแล้ว เนื่องจากเหตุผลในตารางต่อไปนี้ เป็นสัดส่วนมากน้อยเพียงใด (ประมาณการ) [ผู้ประกอบการต้องการป้องกันความเสี่ยงที่อาจจะถูกสอบสวนสืบสวนโดย สำนักงาน กขค. และ/หรือถูกลงโทษตาม พ.ร.บ การแข่งขันทางการค้า],ร้อยละ 0-20,ร้อยละ 21-40,ร้อยละ 41-60,ร้อยละ 61-80,ร้อยละ 81-100,ไม่ทราบ/ไม่แน่ใจ
รายได้ต่อปีของสถานประกอบการ ในปี พ.ศ. 2562 (ก่อนวิกฤตโควิด),,,,,,
,1.000,0.000,0.00,0.00,0.000,0.000
1 – 10 ล้านบาท,0.125,0.125,0.00,0.00,0.125,0.625
11 – 100 ล้านบาท,0.200,0.200,0.00,0.00,0.000,0.600
ตั้งแต่ 100 ล้านบาทขึ้นไป,0.150,0.050,0.05,0.05,0.000,0.700


In [40]:
### Complete Codes for Computing Distribution _ Unfair

TempValueCount = pd.DataFrame(Data['4.3 จากข้อมูลที่ท่านทราบ ในจำนวนกิจกรรมหรือแผนกิจกรรมที่อาจเข้าข่ายเป็นการกระทำผิดฐานปฏิบัติทางการค้าที่ไม่เป็นธรรม ทั้งหมดที่ท่านระบุในข้อ 4.1 มีกิจกรรมหรือแผนกิจกรรมที่ถูกล้มเลิกหรือปรับเปลี่ยนไปแล้ว เนื่องจากเหตุผลในตารางต่อไปนี้ เป็นสัดส่วนมากน้อยเพียงใด (ประมาณการ) [ผู้ประกอบการต้องการป้องกันความเสี่ยงที่อาจจะถูกสอบสวนสืบสวนโดย สำนักงาน กขค. และ/หรือถูกลงโทษตาม พ.ร.บ การแข่งขันทางการค้า]'].value_counts()).transpose()

TempValueCount_Excluded = TempValueCount.loc[:, TempValueCount.columns!='ไม่ทราบ/ไม่แน่ใจ']

Unfair_Deter_Propotion = (TempValueCount_Excluded/TempValueCount_Excluded.sum(1)[0])*100

Unfair_Deter_Propotion

,ร้อยละ 0-20,ร้อยละ 21-40,ร้อยละ 61-80,ร้อยละ 81-100,ร้อยละ 41-60
4.3 จากข้อมูลที่ท่านทราบ ในจำนวนกิจกรรมหรือแผนกิจกรรมที่อาจเข้าข่ายเป็นการกระทำผิดฐานปฏิบัติทางการค้าที่ไม่เป็นธรรม ทั้งหมดที่ท่านระบุในข้อ 4.1 มีกิจกรรมหรือแผนกิจกรรมที่ถูกล้มเลิกหรือปรับเปลี่ยนไปแล้ว เนื่องจากเหตุผลในตารางต่อไปนี้ เป็นสัดส่วนมากน้อยเพียงใด (ประมาณการ) [ผู้ประกอบการต้องการป้องกันความเสี่ยงที่อาจจะถูกสอบสวนสืบสวนโดย สำนักงาน กขค. และ/หรือถูกลงโทษตาม พ.ร.บ การแข่งขันทางการค้า],46.153846,23.076923,15.384615,7.692308,7.692308


In [48]:
cross_tab_Unfair_Deter = pd.crosstab(index=Data['รายได้ต่อปีของสถานประกอบการ ในปี พ.ศ. 2562 (ก่อนวิกฤตโควิด)'],
                             columns=Data['4.3 จากข้อมูลที่ท่านทราบ ในจำนวนกิจกรรมหรือแผนกิจกรรมที่อาจเข้าข่ายเป็นการกระทำผิดฐานปฏิบัติทางการค้าที่ไม่เป็นธรรม ทั้งหมดที่ท่านระบุในข้อ 4.1 มีกิจกรรมหรือแผนกิจกรรมที่ถูกล้มเลิกหรือปรับเปลี่ยนไปแล้ว เนื่องจากเหตุผลในตารางต่อไปนี้ เป็นสัดส่วนมากน้อยเพียงใด (ประมาณการ) [ผู้ประกอบการต้องการป้องกันความเสี่ยงที่อาจจะถูกสอบสวนสืบสวนโดย สำนักงาน กขค. และ/หรือถูกลงโทษตาม พ.ร.บ การแข่งขันทางการค้า]'],
                             normalize="index")

cross_tab_Unfair_Deter 

4.3 จากข้อมูลที่ท่านทราบ ในจำนวนกิจกรรมหรือแผนกิจกรรมที่อาจเข้าข่ายเป็นการกระทำผิดฐานปฏิบัติทางการค้าที่ไม่เป็นธรรม ทั้งหมดที่ท่านระบุในข้อ 4.1 มีกิจกรรมหรือแผนกิจกรรมที่ถูกล้มเลิกหรือปรับเปลี่ยนไปแล้ว เนื่องจากเหตุผลในตารางต่อไปนี้ เป็นสัดส่วนมากน้อยเพียงใด (ประมาณการ) [ผู้ประกอบการต้องการป้องกันความเสี่ยงที่อาจจะถูกสอบสวนสืบสวนโดย สำนักงาน กขค. และ/หรือถูกลงโทษตาม พ.ร.บ การแข่งขันทางการค้า],ร้อยละ 0-20,ร้อยละ 21-40,ร้อยละ 41-60,ร้อยละ 61-80,ร้อยละ 81-100,ไม่ทราบ/ไม่แน่ใจ
รายได้ต่อปีของสถานประกอบการ ในปี พ.ศ. 2562 (ก่อนวิกฤตโควิด),,,,,,
,1.00,0.000,0.00,0.0,0.000,0.00
1 – 10 ล้านบาท,0.25,0.125,0.00,0.0,0.125,0.50
11 – 100 ล้านบาท,0.20,0.200,0.00,0.0,0.000,0.60
ตั้งแต่ 100 ล้านบาทขึ้นไป,0.05,0.050,0.05,0.1,0.000,0.75


In [44]:
## รวมเป็น Dataframe เดียว

Summary_Proportion_deter = pd.concat([Merger_Deter_Propotion,Abuse_Deter_Propotion,Cartels_Deter_Propotion,cross_tab_Unfair_Deter])


Summary_Proportion_deter

,ร้อยละ 0-20,ร้อยละ 21-40,ร้อยละ 61-80,ร้อยละ 41-60,ร้อยละ 81-100
1.3 ในจำนวนแผนการรวมธุรกิจและการรวมธุรกิจทั้งหมดที่ท่านได้ระบุในข้อ 1.1. มีแผนการรวมธุรกิจและการรวมธุรกิจที่ถูกยกเลิกหรือปรับเปลี่ยนไปแล้ว เนื่องจากเหตุผลในตารางต่อไปนี้ เป็นสัดส่วนมากน้อยเพียงใด (ประมาณการ) [ผู้ประกอบการเกรงว่าจะไม่ได้รับอนุญาตหรือจะได้รับการตั้งเงื่อนไขการรวมธุรกิจที่เข้มงวดจากคณะกรรมการการแข่งขันทางการค้า],46.666667,26.666667,13.333333,13.333333,NaN
2.3 จากข้อมูลที่ท่านทราบ ในจำนวนกิจกรรมหรือแผนกิจกรรมที่อาจเข้าข่ายเป็นการกระทำผิดฐานใช้อำนาจเหนือตลาดอย่างไม่เป็นธรรม ทั้งหมดที่ท่านระบุในข้อ 2.1 มีกิจกรรมหรือแผนกิจกรรมที่ถูกยกเลิกหรือปรับเปลี่ยนไปแล้ว เนื่องจากเหตุผลในตารางต่อไปนี้ เป็นสัดส่วนมากน้อยเพียงใด (ประมาณการ) [ผู้ประกอบการต้องการป้องกันความเสี่ยงที่อาจจะถูกสอบสวนสืบสวนโดย สำนักงาน กขค. และ/หรือถูกลงโทษตาม พ.ร.บ การแข่งขันทางการค้า],53.846154,15.384615,15.384615,15.384615,NaN
3.3 จากข้อมูลที่ท่านทราบ ในจำนวนความร่วมมือหรือแผนความร่วมมือกับผู้ประกอบการรายอื่นที่อาจเข้าข่ายเป็นการกระทำผิดฐานตกลงร่วมกัน ผูกขาด ลด และจำกัดการแข่งขัน ทั้งหมดที่ท่านระบุในข้อ 3.1 มีความร่วมมือหรือแผนความร่วมมือที่ถูกยกเลิกหรือปรับเปลี่ยนไปแล้ว เนื่องจากเหตุผลในตารางต่อไปนี้ เป็นสัดส่วนมากน้อยเพียงใด (ประมาณการ) [ผู้ประกอบการต้องการป้องกันความเสี่ยงที่อาจจะถูกสอบสวนสืบสวนโดย สำนักงาน กขค. และ/หรือถูกลงโทษตาม พ.ร.บ การแข่งขันทางการค้า],53.846154,23.076923,7.692308,7.692308,7.692308
4.3 จากข้อมูลที่ท่านทราบ ในจำนวนกิจกรรมหรือแผนกิจกรรมที่อาจเข้าข่ายเป็นการกระทำผิดฐานปฏิบัติทางการค้าที่ไม่เป็นธรรม ทั้งหมดที่ท่านระบุในข้อ 4.1 มีกิจกรรมหรือแผนกิจกรรมที่ถูกล้มเลิกหรือปรับเปลี่ยนไปแล้ว เนื่องจากเหตุผลในตารางต่อไปนี้ เป็นสัดส่วนมากน้อยเพียงใด (ประมาณการ) [ผู้ประกอบการต้องการป้องกันความเสี่ยงที่อาจจะถูกสอบสวนสืบสวนโดย สำนักงาน กขค. และ/หรือถูกลงโทษตาม พ.ร.บ การแข่งขันทางการค้า],46.153846,23.076923,15.384615,7.692308,7.692308


In [47]:
Summary_Proportion_deter_each_firms = pd.concat([cross_tab_Merger_Deter,cross_tab_Abuse_Deter,cross_tab_Cartels_Deter,Unfair_Deter_Propotion])


Summary_Proportion_deter_each_firms
#ขาด 4.3 

,ร้อยละ 0-20,ร้อยละ 21-40,ร้อยละ 41-60,ร้อยละ 61-80,ไม่ทราบ/ไม่แน่ใจ,ร้อยละ 81-100
,1.000000,0.000000,0.000000,0.000000,0.000,NaN
1 – 10 ล้านบาท,0.125000,0.125000,0.000000,0.000000,0.750,NaN
11 – 100 ล้านบาท,0.200000,0.200000,0.000000,0.000000,0.600,NaN
ตั้งแต่ 100 ล้านบาทขึ้นไป,0.150000,0.000000,0.100000,0.100000,0.650,NaN
,1.000000,0.000000,0.000000,0.000000,0.000,0.000000
1 – 10 ล้านบาท,0.125000,0.125000,0.000000,0.000000,0.625,0.125000
11 – 100 ล้านบาท,0.200000,0.200000,0.000000,0.000000,0.600,0.000000
ตั้งแต่ 100 ล้านบาทขึ้นไป,0.150000,0.050000,0.050000,0.050000,0.700,0.000000
,1.000000,0.000000,0.000000,0.000000,0.000,0.000000
1 – 10 ล้านบาท,0.125000,0.125000,0.000000,0.000000,0.625,0.125000


ValueError: could not broadcast input array from shape (5,) into shape (0,)

In [49]:
TempValueCount = pd.DataFrame(Data['1.2 ในจำนวนแผนการรวมธุรกิจและการรวมธุรกิจทั้งหมดที่ท่านได้ระบุในข้อ 1.1 มีจำนวนแผนการรวมธุรกิจและการรวมธุรกิจที่ถูกยกเลิกหรือปรับเปลี่ยนไปแล้ว เป็นสัดส่วนมากน้อยเพียงใด (ประมาณการ)'].value_counts()).transpose()

TempValueCount_Excluded = TempValueCount.loc[:, TempValueCount.columns!='ไม่ทราบ/ไม่แน่ใจ']

Fail_Merger_Propotion = (TempValueCount_Excluded/TempValueCount_Excluded.sum(1)[0])*100

Fail_Merger_Propotion

,ร้อยละ 0-20,ร้อยละ 21-40,ร้อยละ 61-80,ร้อยละ 41-60
1.2 ในจำนวนแผนการรวมธุรกิจและการรวมธุรกิจทั้งหมดที่ท่านได้ระบุในข้อ 1.1 มีจำนวนแผนการรวมธุรกิจและการรวมธุรกิจที่ถูกยกเลิกหรือปรับเปลี่ยนไปแล้ว เป็นสัดส่วนมากน้อยเพียงใด (ประมาณการ),53.846154,23.076923,15.384615,7.692308


In [29]:
Summary_Proportion.rename(index = {'1.3 ในจำนวนแผนการรวมธุรกิจและการรวมธุรกิจทั้งหมดที่ท่านได้ระบุในข้อ 1.1. มีแผนการรวมธุรกิจและการรวมธุรกิจที่ถูกยกเลิกหรือปรับเปลี่ยนไปแล้ว เนื่องจากเหตุผลในตารางต่อไปนี้ เป็นสัดส่วนมากน้อยเพียงใด (ประมาณการ)  [ผู้ประกอบการเกรงว่าจะไม่ได้รับอนุญาตหรือจะได้รับการตั้งเงื่อนไขการรวมธุรกิจที่เข้มงวดจากคณะกรรมการการแข่งขันทางการค้า]':'Merger'
                                   , '2.3 จากข้อมูลที่ท่านทราบ ในจำนวนกิจกรรมหรือแผนกิจกรรมที่อาจเข้าข่ายเป็นการกระทำผิดฐานใช้อำนาจเหนือตลาดอย่างไม่เป็นธรรม ทั้งหมดที่ท่านระบุในข้อ 2.1 มีกิจกรรมหรือแผนกิจกรรมที่ถูกยกเลิกหรือปรับเปลี่ยนไปแล้ว เนื่องจากเหตุผลในตารางต่อไปนี้ เป็นสัดส่วนมากน้อยเพียงใด (ประมาณการ) [ผู้ประกอบการต้องการป้องกันความเสี่ยงที่อาจจะถูกสอบสวนสืบสวนโดย สำนักงาน กขค. และ/หรือถูกลงโทษตาม พ.ร.บ การแข่งขันทางการค้า]':'Abuse'
                                   , '3.3 จากข้อมูลที่ท่านทราบ ในจำนวนความร่วมมือหรือแผนความร่วมมือกับผู้ประกอบการรายอื่นที่อาจเข้าข่ายเป็นการกระทำผิดฐานตกลงร่วมกัน ผูกขาด ลด และจำกัดการแข่งขัน ทั้งหมดที่ท่านระบุในข้อ 3.1 มีความร่วมมือหรือแผนความร่วมมือที่ถูกยกเลิกหรือปรับเปลี่ยนไปแล้ว เนื่องจากเหตุผลในตารางต่อไปนี้ เป็นสัดส่วนมากน้อยเพียงใด (ประมาณการ) [ผู้ประกอบการต้องการป้องกันความเสี่ยงที่อาจจะถูกสอบสวนสืบสวนโดย สำนักงาน กขค. และ/หรือถูกลงโทษตาม พ.ร.บ การแข่งขันทางการค้า]':'Cartels'
                                   , '4.3 จากข้อมูลที่ท่านทราบ ในจำนวนกิจกรรมหรือแผนกิจกรรมที่อาจเข้าข่ายเป็นการกระทำผิดฐานปฏิบัติทางการค้าที่ไม่เป็นธรรม ทั้งหมดที่ท่านระบุในข้อ 4.1 มีกิจกรรมหรือแผนกิจกรรมที่ถูกล้มเลิกหรือปรับเปลี่ยนไปแล้ว เนื่องจากเหตุผลในตารางต่อไปนี้ เป็นสัดส่วนมากน้อยเพียงใด (ประมาณการ) [ผู้ประกอบการต้องการป้องกันความเสี่ยงที่อาจจะถูกสอบสวนสืบสวนโดย สำนักงาน กขค. และ/หรือถูกลงโทษตาม พ.ร.บ การแข่งขันทางการค้า]':'Unfair'}, inplace = True)


NameError: name 'Summary_Proportion' is not defined

In [50]:
TempValueCount = pd.DataFrame(Data['2.2 จากข้อมูลที่ท่านทราบ ในจำนวนกิจกรรมหรือแผนกิจกรรมที่อาจเข้าข่ายเป็นการกระทำผิดฐานใช้อำนาจเหนือตลาดอย่างไม่เป็นธรรม ทั้งหมดที่ท่านระบุในข้อ 2.1 มีกิจกรรมหรือแผนกิจกรรมที่ถูกยกเลิกหรือปรับเปลี่ยนไปแล้ว เป็นสัดส่วนมากน้อยเพียงใด (ประมาณการ)'].value_counts()).transpose()

TempValueCount_Excluded = TempValueCount.loc[:, TempValueCount.columns!='ไม่ทราบ/ไม่แน่ใจ']

Fail_Abuse_Propotion = (TempValueCount_Excluded/TempValueCount_Excluded.sum(1)[0])*100

Fail_Abuse_Propotion

,ร้อยละ 0-20,ร้อยละ 61-80,ร้อยละ 21-40,ร้อยละ 41-60
2.2 จากข้อมูลที่ท่านทราบ ในจำนวนกิจกรรมหรือแผนกิจกรรมที่อาจเข้าข่ายเป็นการกระทำผิดฐานใช้อำนาจเหนือตลาดอย่างไม่เป็นธรรม ทั้งหมดที่ท่านระบุในข้อ 2.1 มีกิจกรรมหรือแผนกิจกรรมที่ถูกยกเลิกหรือปรับเปลี่ยนไปแล้ว เป็นสัดส่วนมากน้อยเพียงใด (ประมาณการ),75.0,8.333333,8.333333,8.333333


In [51]:
TempValueCount = pd.DataFrame(Data['3.2 จากข้อมูลที่ท่านทราบ ในจำนวนความร่วมมือหรือแผนความร่วมมือกับผู้ประกอบการรายอื่นที่อาจเข้าข่ายเป็นการกระทำผิดฐานตกลงร่วมกัน ผูกขาด ลด และจำกัดการแข่งขัน ทั้งหมดที่ท่านระบุในข้อ 3.1 มีความร่วมมือหรือแผนความร่วมมือที่ถูกยกเลิกหรือปรับเปลี่ยนไปแล้ว เป็นสัดส่วนมากน้อยเพียงใด  (ประมาณการ)'].value_counts()).transpose()

TempValueCount_Excluded = TempValueCount.loc[:, TempValueCount.columns!='ไม่ทราบ/ไม่แน่ใจ']

Fail_Cartels_Propotion = (TempValueCount_Excluded/TempValueCount_Excluded.sum(1)[0])*100

Fail_Cartels_Propotion

,ร้อยละ 0-20,ร้อยละ 61-80,ร้อยละ 21-40,ร้อยละ 81-100
3.2 จากข้อมูลที่ท่านทราบ ในจำนวนความร่วมมือหรือแผนความร่วมมือกับผู้ประกอบการรายอื่นที่อาจเข้าข่ายเป็นการกระทำผิดฐานตกลงร่วมกัน ผูกขาด ลด และจำกัดการแข่งขัน ทั้งหมดที่ท่านระบุในข้อ 3.1 มีความร่วมมือหรือแผนความร่วมมือที่ถูกยกเลิกหรือปรับเปลี่ยนไปแล้ว เป็นสัดส่วนมากน้อยเพียงใด (ประมาณการ),72.727273,9.090909,9.090909,9.090909


In [52]:
TempValueCount = pd.DataFrame(Data['4.2 จากข้อมูลที่ท่านทราบ ในจำนวนกิจกรรมหรือแผนกิจกรรมที่อาจเข้าข่ายเป็นการกระทำผิดฐานปฏิบัติทางการค้าที่ไม่เป็นธรรม ทั้งหมดที่ท่านระบุในข้อ 4.4.1 มีกิจกรรมหรือแผนกิจกรรมที่ถูกยกเลิกหรือปรับเปลี่ยนไปแล้ว เป็นสัดส่วนมากน้อยเพียงใด (ประมาณการ)'].value_counts()).transpose()

TempValueCount_Excluded = TempValueCount.loc[:, TempValueCount.columns!='ไม่ทราบ/ไม่แน่ใจ']

Fail_Unfair_Propotion = (TempValueCount_Excluded/TempValueCount_Excluded.sum(1)[0])*100

Fail_Unfair_Propotion

,ร้อยละ 0-20,ร้อยละ 21-40,ร้อยละ 81-100
4.2 จากข้อมูลที่ท่านทราบ ในจำนวนกิจกรรมหรือแผนกิจกรรมที่อาจเข้าข่ายเป็นการกระทำผิดฐานปฏิบัติทางการค้าที่ไม่เป็นธรรม ทั้งหมดที่ท่านระบุในข้อ 4.4.1 มีกิจกรรมหรือแผนกิจกรรมที่ถูกยกเลิกหรือปรับเปลี่ยนไปแล้ว เป็นสัดส่วนมากน้อยเพียงใด (ประมาณการ),69.230769,23.076923,7.692308


In [27]:
Summary_Proportion_Fail_Violation = pd.concat([Fail_Merger_Propotion,Fail_Abuse_Propotion,Fail_Cartels_Propotion,Fail_Unfair_Propotion])

Summary_Proportion_Fail_Violation


,ร้อยละ 0-20,ร้อยละ 21-40,ร้อยละ 61-80,ร้อยละ 41-60,ร้อยละ 81-100
1.2 ในจำนวนแผนการรวมธุรกิจและการรวมธุรกิจทั้งหมดที่ท่านได้ระบุในข้อ 1.1 มีจำนวนแผนการรวมธุรกิจและการรวมธุรกิจที่ถูกยกเลิกหรือปรับเปลี่ยนไปแล้ว เป็นสัดส่วนมากน้อยเพียงใด (ประมาณการ),53.846154,23.076923,15.384615,7.692308,NaN
2.2 จากข้อมูลที่ท่านทราบ ในจำนวนกิจกรรมหรือแผนกิจกรรมที่อาจเข้าข่ายเป็นการกระทำผิดฐานใช้อำนาจเหนือตลาดอย่างไม่เป็นธรรม ทั้งหมดที่ท่านระบุในข้อ 2.1 มีกิจกรรมหรือแผนกิจกรรมที่ถูกยกเลิกหรือปรับเปลี่ยนไปแล้ว เป็นสัดส่วนมากน้อยเพียงใด (ประมาณการ),75.000000,8.333333,8.333333,8.333333,NaN
3.2 จากข้อมูลที่ท่านทราบ ในจำนวนความร่วมมือหรือแผนความร่วมมือกับผู้ประกอบการรายอื่นที่อาจเข้าข่ายเป็นการกระทำผิดฐานตกลงร่วมกัน ผูกขาด ลด และจำกัดการแข่งขัน ทั้งหมดที่ท่านระบุในข้อ 3.1 มีความร่วมมือหรือแผนความร่วมมือที่ถูกยกเลิกหรือปรับเปลี่ยนไปแล้ว เป็นสัดส่วนมากน้อยเพียงใด (ประมาณการ),72.727273,9.090909,9.090909,NaN,9.090909
4.2 จากข้อมูลที่ท่านทราบ ในจำนวนกิจกรรมหรือแผนกิจกรรมที่อาจเข้าข่ายเป็นการกระทำผิดฐานปฏิบัติทางการค้าที่ไม่เป็นธรรม ทั้งหมดที่ท่านระบุในข้อ 4.4.1 มีกิจกรรมหรือแผนกิจกรรมที่ถูกยกเลิกหรือปรับเปลี่ยนไปแล้ว เป็นสัดส่วนมากน้อยเพียงใด (ประมาณการ),69.230769,23.076923,NaN,NaN,7.692308


In [14]:
Data.columns

Index(['ประทับเวลา',
       'หลังจากท่านได้ทราบถึงภาพรวมและวัตถุประสงค์ของแบบสอบถามฉบับนี้แล้ว ท่านยินดีตอบแบบสอบถามฉบับนี้หรือไม่',
       'ประเภทอุตสาหกรรมหลักของสถานประกอบการ ',
       'กิจการของท่านอยู่ภายใต้การกำกับของหน่วยงานใดบ้าง (สามารถตอบได้มากกว่าหนึ่งข้อ)',
       'รายได้ต่อปีของสถานประกอบการ ในปี พ.ศ. 2562 (ก่อนวิกฤตโควิด)',
       'จำนวนลูกจ้างในสถานประกอบการ',
       'ในกรณีที่จำเป็น ท่านยินดีให้ผู้วิจัย (สถาบันวิจัยเพื่อการพัฒนาประเทศไทย) ติดต่อท่านอีกครั้งเพื่อรวบรวมข้อมูลเพิ่มเติมจากข้อมูลที่ได้ท่านให้ไว้หรือไม่ ',
       'เบอร์โทรศัพท์',
       'สิ่งใดเป็นความผิดตามพระราชบัญญัติการแข่งขันทางการค้า พ.ศ. 2560',
       'ลักษณะใดเป็นเกณฑ์ของการเป็นผู้มีอำนาจเหนือตลาดตามพระราชบัญญัติการแข่งขันทางการค้า พ.ศ. 2560',
       'ค่าปรับสำหรับการกระทำผิดตามพระราชบัญญัติการแข่งขันทางการค้า พ.ศ. 2560 ขึ้นกับปัจจัยอะไรบ้าง',
       'การกระทำผิดในข้อใดหากฝ่าฝืนแล้วมีโทษจำคุก ',
       'การรวมธุรกิจลักษณะใดต้องได้รับอนุญาตจากคณะกรรมการการแข่งขันทางการค้า',
       'ข้อใดคือที่อยู่ของสำ

In [15]:
TempValueCount = pd.DataFrame(Data['1.3 ในจำนวนแผนการรวมธุรกิจและการรวมธุรกิจทั้งหมดที่ท่านได้ระบุในข้อ 1.1. มีแผนการรวมธุรกิจและการรวมธุรกิจที่ถูกยกเลิกหรือปรับเปลี่ยนไปแล้ว เนื่องจากเหตุผลในตารางต่อไปนี้ เป็นสัดส่วนมากน้อยเพียงใด (ประมาณการ)  [ผู้ประกอบการเกรงว่าจะไม่ได้รับอนุญาตหรือจะได้รับการตั้งเงื่อนไขการรวมธุรกิจที่เข้มงวดจากคณะกรรมการการแข่งขันทางการค้า]'].value_counts()).transpose()

TempValueCount_Excluded = TempValueCount.loc[:, TempValueCount.columns!='ไม่ทราบ/ไม่แน่ใจ']

Merger_311_Propotion = (TempValueCount_Excluded/TempValueCount_Excluded.sum(1)[0])*100

Merger_311_Propotion

,ร้อยละ 0-20,ร้อยละ 21-40,ร้อยละ 41-60,ร้อยละ 61-80
1.3 ในจำนวนแผนการรวมธุรกิจและการรวมธุรกิจทั้งหมดที่ท่านได้ระบุในข้อ 1.1. มีแผนการรวมธุรกิจและการรวมธุรกิจที่ถูกยกเลิกหรือปรับเปลี่ยนไปแล้ว เนื่องจากเหตุผลในตารางต่อไปนี้ เป็นสัดส่วนมากน้อยเพียงใด (ประมาณการ) [ผู้ประกอบการเกรงว่าจะไม่ได้รับอนุญาตหรือจะได้รับการตั้งเงื่อนไขการรวมธุรกิจที่เข้มงวดจากคณะกรรมการการแข่งขันทางการค้า],46.666667,26.666667,13.333333,13.333333


In [16]:
TempValueCount = pd.DataFrame(Data['1.3 ในจำนวนแผนการรวมธุรกิจและการรวมธุรกิจทั้งหมดที่ท่านได้ระบุในข้อ 1.1. มีแผนการรวมธุรกิจและการรวมธุรกิจที่ถูกยกเลิกหรือปรับเปลี่ยนไปแล้ว เนื่องจากเหตุผลในตารางต่อไปนี้ เป็นสัดส่วนมากน้อยเพียงใด (ประมาณการ)  [ผู้ประกอบการเกรงว่าจะไม่ได้รับอนุญาตหรือจะได้รับการตั้งเงื่อนไขการรวมธุรกิจที่เข้มงวด จากกฎหมายเฉพาะฉบับอื่น ๆ]'].value_counts()).transpose()

TempValueCount_Excluded = TempValueCount.loc[:, TempValueCount.columns!='ไม่ทราบ/ไม่แน่ใจ']

Merger_312_Propotion = (TempValueCount_Excluded/TempValueCount_Excluded.sum(1)[0])*100

Merger_312_Propotion

,ร้อยละ 0-20,ร้อยละ 21-40,ร้อยละ 41-60,ร้อยละ 61-80
1.3 ในจำนวนแผนการรวมธุรกิจและการรวมธุรกิจทั้งหมดที่ท่านได้ระบุในข้อ 1.1. มีแผนการรวมธุรกิจและการรวมธุรกิจที่ถูกยกเลิกหรือปรับเปลี่ยนไปแล้ว เนื่องจากเหตุผลในตารางต่อไปนี้ เป็นสัดส่วนมากน้อยเพียงใด (ประมาณการ) [ผู้ประกอบการเกรงว่าจะไม่ได้รับอนุญาตหรือจะได้รับการตั้งเงื่อนไขการรวมธุรกิจที่เข้มงวด จากกฎหมายเฉพาะฉบับอื่น ๆ],53.333333,20.0,13.333333,13.333333


In [17]:
Summary_Proportion_31 = pd.concat([Merger_311_Propotion,Merger_312_Propotion])

Summary_Proportion_31

,ร้อยละ 0-20,ร้อยละ 21-40,ร้อยละ 41-60,ร้อยละ 61-80
1.3 ในจำนวนแผนการรวมธุรกิจและการรวมธุรกิจทั้งหมดที่ท่านได้ระบุในข้อ 1.1. มีแผนการรวมธุรกิจและการรวมธุรกิจที่ถูกยกเลิกหรือปรับเปลี่ยนไปแล้ว เนื่องจากเหตุผลในตารางต่อไปนี้ เป็นสัดส่วนมากน้อยเพียงใด (ประมาณการ) [ผู้ประกอบการเกรงว่าจะไม่ได้รับอนุญาตหรือจะได้รับการตั้งเงื่อนไขการรวมธุรกิจที่เข้มงวดจากคณะกรรมการการแข่งขันทางการค้า],46.666667,26.666667,13.333333,13.333333
1.3 ในจำนวนแผนการรวมธุรกิจและการรวมธุรกิจทั้งหมดที่ท่านได้ระบุในข้อ 1.1. มีแผนการรวมธุรกิจและการรวมธุรกิจที่ถูกยกเลิกหรือปรับเปลี่ยนไปแล้ว เนื่องจากเหตุผลในตารางต่อไปนี้ เป็นสัดส่วนมากน้อยเพียงใด (ประมาณการ) [ผู้ประกอบการเกรงว่าจะไม่ได้รับอนุญาตหรือจะได้รับการตั้งเงื่อนไขการรวมธุรกิจที่เข้มงวด จากกฎหมายเฉพาะฉบับอื่น ๆ],53.333333,20.000000,13.333333,13.333333


In [18]:
TempValueCount = pd.DataFrame(Data['2.3 จากข้อมูลที่ท่านทราบ ในจำนวนกิจกรรมหรือแผนกิจกรรมที่อาจเข้าข่ายเป็นการกระทำผิดฐานใช้อำนาจเหนือตลาดอย่างไม่เป็นธรรม ทั้งหมดที่ท่านระบุในข้อ 2.1 มีกิจกรรมหรือแผนกิจกรรมที่ถูกยกเลิกหรือปรับเปลี่ยนไปแล้ว เนื่องจากเหตุผลในตารางต่อไปนี้ เป็นสัดส่วนมากน้อยเพียงใด (ประมาณการ) [ผู้ประกอบการต้องการป้องกันความเสี่ยงที่อาจจะถูกสอบสวนสืบสวนโดย สำนักงาน กขค. และ/หรือถูกลงโทษตาม พ.ร.บ การแข่งขันทางการค้า]'].value_counts()).transpose()

TempValueCount_Excluded = TempValueCount.loc[:, TempValueCount.columns!='ไม่ทราบ/ไม่แน่ใจ']

Abuse_321_Propotion = (TempValueCount_Excluded/TempValueCount_Excluded.sum(1)[0])*100

Abuse_321_Propotion

,ร้อยละ 0-20,ร้อยละ 41-60,ร้อยละ 61-80,ร้อยละ 21-40
2.3 จากข้อมูลที่ท่านทราบ ในจำนวนกิจกรรมหรือแผนกิจกรรมที่อาจเข้าข่ายเป็นการกระทำผิดฐานใช้อำนาจเหนือตลาดอย่างไม่เป็นธรรม ทั้งหมดที่ท่านระบุในข้อ 2.1 มีกิจกรรมหรือแผนกิจกรรมที่ถูกยกเลิกหรือปรับเปลี่ยนไปแล้ว เนื่องจากเหตุผลในตารางต่อไปนี้ เป็นสัดส่วนมากน้อยเพียงใด (ประมาณการ) [ผู้ประกอบการต้องการป้องกันความเสี่ยงที่อาจจะถูกสอบสวนสืบสวนโดย สำนักงาน กขค. และ/หรือถูกลงโทษตาม พ.ร.บ การแข่งขันทางการค้า],53.846154,15.384615,15.384615,15.384615


In [19]:
TempValueCount = pd.DataFrame(Data['2.3 จากข้อมูลที่ท่านทราบ ในจำนวนกิจกรรมหรือแผนกิจกรรมที่อาจเข้าข่ายเป็นการกระทำผิดฐานใช้อำนาจเหนือตลาดอย่างไม่เป็นธรรม ทั้งหมดที่ท่านระบุในข้อ 2.1 มีกิจกรรมหรือแผนกิจกรรมที่ถูกยกเลิกหรือปรับเปลี่ยนไปแล้ว เนื่องจากเหตุผลในตารางต่อไปนี้ เป็นสัดส่วนมากน้อยเพียงใด (ประมาณการ) [ผู้ประกอบการต้องการป้องกันความเสี่ยงที่อาจจะถูกลงโทษตามกฎหมายเฉพาะอื่น ๆ]'].value_counts()).transpose()

TempValueCount_Excluded = TempValueCount.loc[:, TempValueCount.columns!='ไม่ทราบ/ไม่แน่ใจ']

Abuse_322_Propotion = (TempValueCount_Excluded/TempValueCount_Excluded.sum(1)[0])*100

Abuse_322_Propotion

,ร้อยละ 0-20,ร้อยละ 41-60,ร้อยละ 61-80,ร้อยละ 21-40
2.3 จากข้อมูลที่ท่านทราบ ในจำนวนกิจกรรมหรือแผนกิจกรรมที่อาจเข้าข่ายเป็นการกระทำผิดฐานใช้อำนาจเหนือตลาดอย่างไม่เป็นธรรม ทั้งหมดที่ท่านระบุในข้อ 2.1 มีกิจกรรมหรือแผนกิจกรรมที่ถูกยกเลิกหรือปรับเปลี่ยนไปแล้ว เนื่องจากเหตุผลในตารางต่อไปนี้ เป็นสัดส่วนมากน้อยเพียงใด (ประมาณการ) [ผู้ประกอบการต้องการป้องกันความเสี่ยงที่อาจจะถูกลงโทษตามกฎหมายเฉพาะอื่น ๆ],57.142857,14.285714,14.285714,14.285714


In [20]:
TempValueCount = pd.DataFrame(Data['2.3 จากข้อมูลที่ท่านทราบ ในจำนวนกิจกรรมหรือแผนกิจกรรมที่อาจเข้าข่ายเป็นการกระทำผิดฐานใช้อำนาจเหนือตลาดอย่างไม่เป็นธรรม ทั้งหมดที่ท่านระบุในข้อ 2.1 มีกิจกรรมหรือแผนกิจกรรมที่ถูกยกเลิกหรือปรับเปลี่ยนไปแล้ว เนื่องจากเหตุผลในตารางต่อไปนี้ เป็นสัดส่วนมากน้อยเพียงใด (ประมาณการ) [ผู้ประกอบการต้องการป้องกันความเสี่ยงที่อาจจะถูกฟ้องโดยผู้ประกอบการรายอื่น]'].value_counts()).transpose()

TempValueCount_Excluded = TempValueCount.loc[:, TempValueCount.columns!='ไม่ทราบ/ไม่แน่ใจ']

Abuse_323_Propotion = (TempValueCount_Excluded/TempValueCount_Excluded.sum(1)[0])*100

Abuse_323_Propotion

,ร้อยละ 0-20,ร้อยละ 41-60,ร้อยละ 61-80,ร้อยละ 21-40
2.3 จากข้อมูลที่ท่านทราบ ในจำนวนกิจกรรมหรือแผนกิจกรรมที่อาจเข้าข่ายเป็นการกระทำผิดฐานใช้อำนาจเหนือตลาดอย่างไม่เป็นธรรม ทั้งหมดที่ท่านระบุในข้อ 2.1 มีกิจกรรมหรือแผนกิจกรรมที่ถูกยกเลิกหรือปรับเปลี่ยนไปแล้ว เนื่องจากเหตุผลในตารางต่อไปนี้ เป็นสัดส่วนมากน้อยเพียงใด (ประมาณการ) [ผู้ประกอบการต้องการป้องกันความเสี่ยงที่อาจจะถูกฟ้องโดยผู้ประกอบการรายอื่น],57.142857,14.285714,14.285714,14.285714


In [21]:
TempValueCount = pd.DataFrame(Data['2.3 จากข้อมูลที่ท่านทราบ ในจำนวนกิจกรรมหรือแผนกิจกรรมที่อาจเข้าข่ายเป็นการกระทำผิดฐานใช้อำนาจเหนือตลาดอย่างไม่เป็นธรรม ทั้งหมดที่ท่านระบุในข้อ 2.1 มีกิจกรรมหรือแผนกิจกรรมที่ถูกยกเลิกหรือปรับเปลี่ยนไปแล้ว เนื่องจากเหตุผลในตารางต่อไปนี้ เป็นสัดส่วนมากน้อยเพียงใด (ประมาณการ) [ผู้ประกอบการได้รับคำวินิจฉัยล่วงหน้าจากคณะกรรมการการแข่งขันทางการค้า]'].value_counts()).transpose()

TempValueCount_Excluded = TempValueCount.loc[:, TempValueCount.columns!='ไม่ทราบ/ไม่แน่ใจ']

Abuse_324_Propotion = (TempValueCount_Excluded/TempValueCount_Excluded.sum(1)[0])*100

Abuse_324_Propotion

,ร้อยละ 0-20,ร้อยละ 21-40,ร้อยละ 41-60,ร้อยละ 61-80
2.3 จากข้อมูลที่ท่านทราบ ในจำนวนกิจกรรมหรือแผนกิจกรรมที่อาจเข้าข่ายเป็นการกระทำผิดฐานใช้อำนาจเหนือตลาดอย่างไม่เป็นธรรม ทั้งหมดที่ท่านระบุในข้อ 2.1 มีกิจกรรมหรือแผนกิจกรรมที่ถูกยกเลิกหรือปรับเปลี่ยนไปแล้ว เนื่องจากเหตุผลในตารางต่อไปนี้ เป็นสัดส่วนมากน้อยเพียงใด (ประมาณการ) [ผู้ประกอบการได้รับคำวินิจฉัยล่วงหน้าจากคณะกรรมการการแข่งขันทางการค้า],57.142857,21.428571,14.285714,7.142857


In [22]:
Summary_Proportion_32 = pd.concat([Abuse_321_Propotion,Abuse_322_Propotion,Abuse_323_Propotion,Abuse_324_Propotion])

Summary_Proportion_32

,ร้อยละ 0-20,ร้อยละ 41-60,ร้อยละ 61-80,ร้อยละ 21-40
2.3 จากข้อมูลที่ท่านทราบ ในจำนวนกิจกรรมหรือแผนกิจกรรมที่อาจเข้าข่ายเป็นการกระทำผิดฐานใช้อำนาจเหนือตลาดอย่างไม่เป็นธรรม ทั้งหมดที่ท่านระบุในข้อ 2.1 มีกิจกรรมหรือแผนกิจกรรมที่ถูกยกเลิกหรือปรับเปลี่ยนไปแล้ว เนื่องจากเหตุผลในตารางต่อไปนี้ เป็นสัดส่วนมากน้อยเพียงใด (ประมาณการ) [ผู้ประกอบการต้องการป้องกันความเสี่ยงที่อาจจะถูกสอบสวนสืบสวนโดย สำนักงาน กขค. และ/หรือถูกลงโทษตาม พ.ร.บ การแข่งขันทางการค้า],53.846154,15.384615,15.384615,15.384615
2.3 จากข้อมูลที่ท่านทราบ ในจำนวนกิจกรรมหรือแผนกิจกรรมที่อาจเข้าข่ายเป็นการกระทำผิดฐานใช้อำนาจเหนือตลาดอย่างไม่เป็นธรรม ทั้งหมดที่ท่านระบุในข้อ 2.1 มีกิจกรรมหรือแผนกิจกรรมที่ถูกยกเลิกหรือปรับเปลี่ยนไปแล้ว เนื่องจากเหตุผลในตารางต่อไปนี้ เป็นสัดส่วนมากน้อยเพียงใด (ประมาณการ) [ผู้ประกอบการต้องการป้องกันความเสี่ยงที่อาจจะถูกลงโทษตามกฎหมายเฉพาะอื่น ๆ],57.142857,14.285714,14.285714,14.285714
2.3 จากข้อมูลที่ท่านทราบ ในจำนวนกิจกรรมหรือแผนกิจกรรมที่อาจเข้าข่ายเป็นการกระทำผิดฐานใช้อำนาจเหนือตลาดอย่างไม่เป็นธรรม ทั้งหมดที่ท่านระบุในข้อ 2.1 มีกิจกรรมหรือแผนกิจกรรมที่ถูกยกเลิกหรือปรับเปลี่ยนไปแล้ว เนื่องจากเหตุผลในตารางต่อไปนี้ เป็นสัดส่วนมากน้อยเพียงใด (ประมาณการ) [ผู้ประกอบการต้องการป้องกันความเสี่ยงที่อาจจะถูกฟ้องโดยผู้ประกอบการรายอื่น],57.142857,14.285714,14.285714,14.285714
2.3 จากข้อมูลที่ท่านทราบ ในจำนวนกิจกรรมหรือแผนกิจกรรมที่อาจเข้าข่ายเป็นการกระทำผิดฐานใช้อำนาจเหนือตลาดอย่างไม่เป็นธรรม ทั้งหมดที่ท่านระบุในข้อ 2.1 มีกิจกรรมหรือแผนกิจกรรมที่ถูกยกเลิกหรือปรับเปลี่ยนไปแล้ว เนื่องจากเหตุผลในตารางต่อไปนี้ เป็นสัดส่วนมากน้อยเพียงใด (ประมาณการ) [ผู้ประกอบการได้รับคำวินิจฉัยล่วงหน้าจากคณะกรรมการการแข่งขันทางการค้า],57.142857,14.285714,7.142857,21.428571


In [23]:
TempValueCount = pd.DataFrame(Data['3.3 จากข้อมูลที่ท่านทราบ ในจำนวนความร่วมมือหรือแผนความร่วมมือกับผู้ประกอบการรายอื่นที่อาจเข้าข่ายเป็นการกระทำผิดฐานตกลงร่วมกัน ผูกขาด ลด และจำกัดการแข่งขัน ทั้งหมดที่ท่านระบุในข้อ 3.1 มีความร่วมมือหรือแผนความร่วมมือที่ถูกยกเลิกหรือปรับเปลี่ยนไปแล้ว เนื่องจากเหตุผลในตารางต่อไปนี้ เป็นสัดส่วนมากน้อยเพียงใด (ประมาณการ) [ผู้ประกอบการต้องการป้องกันความเสี่ยงที่อาจจะถูกสอบสวนสืบสวนโดย สำนักงาน กขค. และ/หรือถูกลงโทษตาม พ.ร.บ การแข่งขันทางการค้า]'].value_counts()).transpose()

TempValueCount_Excluded = TempValueCount.loc[:, TempValueCount.columns!='ไม่ทราบ/ไม่แน่ใจ']

Cartels_331_Propotion = (TempValueCount_Excluded/TempValueCount_Excluded.sum(1)[0])*100

Cartels_331_Propotion

,ร้อยละ 0-20,ร้อยละ 21-40,ร้อยละ 41-60,ร้อยละ 61-80,ร้อยละ 81-100
3.3 จากข้อมูลที่ท่านทราบ ในจำนวนความร่วมมือหรือแผนความร่วมมือกับผู้ประกอบการรายอื่นที่อาจเข้าข่ายเป็นการกระทำผิดฐานตกลงร่วมกัน ผูกขาด ลด และจำกัดการแข่งขัน ทั้งหมดที่ท่านระบุในข้อ 3.1 มีความร่วมมือหรือแผนความร่วมมือที่ถูกยกเลิกหรือปรับเปลี่ยนไปแล้ว เนื่องจากเหตุผลในตารางต่อไปนี้ เป็นสัดส่วนมากน้อยเพียงใด (ประมาณการ) [ผู้ประกอบการต้องการป้องกันความเสี่ยงที่อาจจะถูกสอบสวนสืบสวนโดย สำนักงาน กขค. และ/หรือถูกลงโทษตาม พ.ร.บ การแข่งขันทางการค้า],53.846154,23.076923,7.692308,7.692308,7.692308


In [24]:
TempValueCount = pd.DataFrame(Data['3.3 จากข้อมูลที่ท่านทราบ ในจำนวนความร่วมมือหรือแผนความร่วมมือกับผู้ประกอบการรายอื่นที่อาจเข้าข่ายเป็นการกระทำผิดฐานตกลงร่วมกัน ผูกขาด ลด และจำกัดการแข่งขัน ทั้งหมดที่ท่านระบุในข้อ 3.1 มีความร่วมมือหรือแผนความร่วมมือที่ถูกยกเลิกหรือปรับเปลี่ยนไปแล้ว เนื่องจากเหตุผลในตารางต่อไปนี้ เป็นสัดส่วนมากน้อยเพียงใด (ประมาณการ) [ผู้ประกอบการต้องการป้องกันความเสี่ยงที่อาจจะถูกลงโทษตามกฎหมายเฉพาะอื่น ๆ]'].value_counts()).transpose()

TempValueCount_Excluded = TempValueCount.loc[:, TempValueCount.columns!='ไม่ทราบ/ไม่แน่ใจ']

Cartels_332_Propotion = (TempValueCount_Excluded/TempValueCount_Excluded.sum(1)[0])*100

Cartels_332_Propotion

,ร้อยละ 0-20,ร้อยละ 21-40,ร้อยละ 41-60,ร้อยละ 61-80,ร้อยละ 81-100
3.3 จากข้อมูลที่ท่านทราบ ในจำนวนความร่วมมือหรือแผนความร่วมมือกับผู้ประกอบการรายอื่นที่อาจเข้าข่ายเป็นการกระทำผิดฐานตกลงร่วมกัน ผูกขาด ลด และจำกัดการแข่งขัน ทั้งหมดที่ท่านระบุในข้อ 3.1 มีความร่วมมือหรือแผนความร่วมมือที่ถูกยกเลิกหรือปรับเปลี่ยนไปแล้ว เนื่องจากเหตุผลในตารางต่อไปนี้ เป็นสัดส่วนมากน้อยเพียงใด (ประมาณการ) [ผู้ประกอบการต้องการป้องกันความเสี่ยงที่อาจจะถูกลงโทษตามกฎหมายเฉพาะอื่น ๆ],58.333333,16.666667,8.333333,8.333333,8.333333


In [25]:
TempValueCount = pd.DataFrame(Data['3.3 จากข้อมูลที่ท่านทราบ ในจำนวนความร่วมมือหรือแผนความร่วมมือกับผู้ประกอบการรายอื่นที่อาจเข้าข่ายเป็นการกระทำผิดฐานตกลงร่วมกัน ผูกขาด ลด และจำกัดการแข่งขัน ทั้งหมดที่ท่านระบุในข้อ 3.1 มีความร่วมมือหรือแผนความร่วมมือที่ถูกยกเลิกหรือปรับเปลี่ยนไปแล้ว เนื่องจากเหตุผลในตารางต่อไปนี้ เป็นสัดส่วนมากน้อยเพียงใด (ประมาณการ) [ผู้ประกอบการต้องการป้องกันความเสี่ยงที่อาจจะถูกฟ้องโดยผู้ประกอบการรายอื่น]'].value_counts()).transpose()

TempValueCount_Excluded = TempValueCount.loc[:, TempValueCount.columns!='ไม่ทราบ/ไม่แน่ใจ']

Cartels_333_Propotion = (TempValueCount_Excluded/TempValueCount_Excluded.sum(1)[0])*100

Cartels_333_Propotion

,ร้อยละ 0-20,ร้อยละ 21-40,ร้อยละ 41-60,ร้อยละ 61-80,ร้อยละ 81-100
3.3 จากข้อมูลที่ท่านทราบ ในจำนวนความร่วมมือหรือแผนความร่วมมือกับผู้ประกอบการรายอื่นที่อาจเข้าข่ายเป็นการกระทำผิดฐานตกลงร่วมกัน ผูกขาด ลด และจำกัดการแข่งขัน ทั้งหมดที่ท่านระบุในข้อ 3.1 มีความร่วมมือหรือแผนความร่วมมือที่ถูกยกเลิกหรือปรับเปลี่ยนไปแล้ว เนื่องจากเหตุผลในตารางต่อไปนี้ เป็นสัดส่วนมากน้อยเพียงใด (ประมาณการ) [ผู้ประกอบการต้องการป้องกันความเสี่ยงที่อาจจะถูกฟ้องโดยผู้ประกอบการรายอื่น],46.153846,30.769231,7.692308,7.692308,7.692308


In [26]:
TempValueCount = pd.DataFrame(Data['3.3 จากข้อมูลที่ท่านทราบ ในจำนวนความร่วมมือหรือแผนความร่วมมือกับผู้ประกอบการรายอื่นที่อาจเข้าข่ายเป็นการกระทำผิดฐานตกลงร่วมกัน ผูกขาด ลด และจำกัดการแข่งขัน ทั้งหมดที่ท่านระบุในข้อ 3.1 มีความร่วมมือหรือแผนความร่วมมือที่ถูกยกเลิกหรือปรับเปลี่ยนไปแล้ว เนื่องจากเหตุผลในตารางต่อไปนี้ เป็นสัดส่วนมากน้อยเพียงใด (ประมาณการ) [ผู้ประกอบการได้รับคำวินิจฉัยล่วงหน้าจากคณะกรรมการการแข่งขันทางการค้า]'].value_counts()).transpose()

TempValueCount_Excluded = TempValueCount.loc[:, TempValueCount.columns!='ไม่ทราบ/ไม่แน่ใจ']

Cartels_334_Propotion = (TempValueCount_Excluded/TempValueCount_Excluded.sum(1)[0])*100

Cartels_334_Propotion

,ร้อยละ 0-20,ร้อยละ 41-60,ร้อยละ 21-40,ร้อยละ 61-80,ร้อยละ 81-100
3.3 จากข้อมูลที่ท่านทราบ ในจำนวนความร่วมมือหรือแผนความร่วมมือกับผู้ประกอบการรายอื่นที่อาจเข้าข่ายเป็นการกระทำผิดฐานตกลงร่วมกัน ผูกขาด ลด และจำกัดการแข่งขัน ทั้งหมดที่ท่านระบุในข้อ 3.1 มีความร่วมมือหรือแผนความร่วมมือที่ถูกยกเลิกหรือปรับเปลี่ยนไปแล้ว เนื่องจากเหตุผลในตารางต่อไปนี้ เป็นสัดส่วนมากน้อยเพียงใด (ประมาณการ) [ผู้ประกอบการได้รับคำวินิจฉัยล่วงหน้าจากคณะกรรมการการแข่งขันทางการค้า],53.846154,15.384615,15.384615,7.692308,7.692308


In [27]:
Summary_Proportion_33 = pd.concat([Cartels_331_Propotion,Cartels_332_Propotion,Cartels_333_Propotion,Cartels_334_Propotion])

Summary_Proportion_33

,ร้อยละ 0-20,ร้อยละ 21-40,ร้อยละ 41-60,ร้อยละ 61-80,ร้อยละ 81-100
3.3 จากข้อมูลที่ท่านทราบ ในจำนวนความร่วมมือหรือแผนความร่วมมือกับผู้ประกอบการรายอื่นที่อาจเข้าข่ายเป็นการกระทำผิดฐานตกลงร่วมกัน ผูกขาด ลด และจำกัดการแข่งขัน ทั้งหมดที่ท่านระบุในข้อ 3.1 มีความร่วมมือหรือแผนความร่วมมือที่ถูกยกเลิกหรือปรับเปลี่ยนไปแล้ว เนื่องจากเหตุผลในตารางต่อไปนี้ เป็นสัดส่วนมากน้อยเพียงใด (ประมาณการ) [ผู้ประกอบการต้องการป้องกันความเสี่ยงที่อาจจะถูกสอบสวนสืบสวนโดย สำนักงาน กขค. และ/หรือถูกลงโทษตาม พ.ร.บ การแข่งขันทางการค้า],53.846154,23.076923,7.692308,7.692308,7.692308
3.3 จากข้อมูลที่ท่านทราบ ในจำนวนความร่วมมือหรือแผนความร่วมมือกับผู้ประกอบการรายอื่นที่อาจเข้าข่ายเป็นการกระทำผิดฐานตกลงร่วมกัน ผูกขาด ลด และจำกัดการแข่งขัน ทั้งหมดที่ท่านระบุในข้อ 3.1 มีความร่วมมือหรือแผนความร่วมมือที่ถูกยกเลิกหรือปรับเปลี่ยนไปแล้ว เนื่องจากเหตุผลในตารางต่อไปนี้ เป็นสัดส่วนมากน้อยเพียงใด (ประมาณการ) [ผู้ประกอบการต้องการป้องกันความเสี่ยงที่อาจจะถูกลงโทษตามกฎหมายเฉพาะอื่น ๆ],58.333333,16.666667,8.333333,8.333333,8.333333
3.3 จากข้อมูลที่ท่านทราบ ในจำนวนความร่วมมือหรือแผนความร่วมมือกับผู้ประกอบการรายอื่นที่อาจเข้าข่ายเป็นการกระทำผิดฐานตกลงร่วมกัน ผูกขาด ลด และจำกัดการแข่งขัน ทั้งหมดที่ท่านระบุในข้อ 3.1 มีความร่วมมือหรือแผนความร่วมมือที่ถูกยกเลิกหรือปรับเปลี่ยนไปแล้ว เนื่องจากเหตุผลในตารางต่อไปนี้ เป็นสัดส่วนมากน้อยเพียงใด (ประมาณการ) [ผู้ประกอบการต้องการป้องกันความเสี่ยงที่อาจจะถูกฟ้องโดยผู้ประกอบการรายอื่น],46.153846,30.769231,7.692308,7.692308,7.692308
3.3 จากข้อมูลที่ท่านทราบ ในจำนวนความร่วมมือหรือแผนความร่วมมือกับผู้ประกอบการรายอื่นที่อาจเข้าข่ายเป็นการกระทำผิดฐานตกลงร่วมกัน ผูกขาด ลด และจำกัดการแข่งขัน ทั้งหมดที่ท่านระบุในข้อ 3.1 มีความร่วมมือหรือแผนความร่วมมือที่ถูกยกเลิกหรือปรับเปลี่ยนไปแล้ว เนื่องจากเหตุผลในตารางต่อไปนี้ เป็นสัดส่วนมากน้อยเพียงใด (ประมาณการ) [ผู้ประกอบการได้รับคำวินิจฉัยล่วงหน้าจากคณะกรรมการการแข่งขันทางการค้า],53.846154,15.384615,15.384615,7.692308,7.692308


In [28]:
TempValueCount = pd.DataFrame(Data['4.3 จากข้อมูลที่ท่านทราบ ในจำนวนกิจกรรมหรือแผนกิจกรรมที่อาจเข้าข่ายเป็นการกระทำผิดฐานปฏิบัติทางการค้าที่ไม่เป็นธรรม ทั้งหมดที่ท่านระบุในข้อ 4.1 มีกิจกรรมหรือแผนกิจกรรมที่ถูกล้มเลิกหรือปรับเปลี่ยนไปแล้ว เนื่องจากเหตุผลในตารางต่อไปนี้ เป็นสัดส่วนมากน้อยเพียงใด (ประมาณการ) [ผู้ประกอบการต้องการป้องกันความเสี่ยงที่อาจจะถูกสอบสวนสืบสวนโดย สำนักงาน กขค. และ/หรือถูกลงโทษตาม พ.ร.บ การแข่งขันทางการค้า]'].value_counts()).transpose()

TempValueCount_Excluded = TempValueCount.loc[:, TempValueCount.columns!='ไม่ทราบ/ไม่แน่ใจ']

Unfair_341_Propotion = (TempValueCount_Excluded/TempValueCount_Excluded.sum(1)[0])*100

Unfair_341_Propotion

,ร้อยละ 0-20,ร้อยละ 21-40,ร้อยละ 61-80,ร้อยละ 41-60,ร้อยละ 81-100
4.3 จากข้อมูลที่ท่านทราบ ในจำนวนกิจกรรมหรือแผนกิจกรรมที่อาจเข้าข่ายเป็นการกระทำผิดฐานปฏิบัติทางการค้าที่ไม่เป็นธรรม ทั้งหมดที่ท่านระบุในข้อ 4.1 มีกิจกรรมหรือแผนกิจกรรมที่ถูกล้มเลิกหรือปรับเปลี่ยนไปแล้ว เนื่องจากเหตุผลในตารางต่อไปนี้ เป็นสัดส่วนมากน้อยเพียงใด (ประมาณการ) [ผู้ประกอบการต้องการป้องกันความเสี่ยงที่อาจจะถูกสอบสวนสืบสวนโดย สำนักงาน กขค. และ/หรือถูกลงโทษตาม พ.ร.บ การแข่งขันทางการค้า],46.153846,23.076923,15.384615,7.692308,7.692308


In [29]:
TempValueCount = pd.DataFrame(Data['4.3 จากข้อมูลที่ท่านทราบ ในจำนวนกิจกรรมหรือแผนกิจกรรมที่อาจเข้าข่ายเป็นการกระทำผิดฐานปฏิบัติทางการค้าที่ไม่เป็นธรรม ทั้งหมดที่ท่านระบุในข้อ 4.1 มีกิจกรรมหรือแผนกิจกรรมที่ถูกล้มเลิกหรือปรับเปลี่ยนไปแล้ว เนื่องจากเหตุผลในตารางต่อไปนี้ เป็นสัดส่วนมากน้อยเพียงใด (ประมาณการ) [ผู้ประกอบการต้องการป้องกันความเสี่ยงที่อาจจะถูกลงโทษตามกฎหมายเฉพาะอื่น ๆ]'].value_counts()).transpose()

TempValueCount_Excluded = TempValueCount.loc[:, TempValueCount.columns!='ไม่ทราบ/ไม่แน่ใจ']

Unfair_342_Propotion = (TempValueCount_Excluded/TempValueCount_Excluded.sum(1)[0])*100

Unfair_342_Propotion

,ร้อยละ 0-20,ร้อยละ 61-80,ร้อยละ 41-60,ร้อยละ 81-100,ร้อยละ 21-40
4.3 จากข้อมูลที่ท่านทราบ ในจำนวนกิจกรรมหรือแผนกิจกรรมที่อาจเข้าข่ายเป็นการกระทำผิดฐานปฏิบัติทางการค้าที่ไม่เป็นธรรม ทั้งหมดที่ท่านระบุในข้อ 4.1 มีกิจกรรมหรือแผนกิจกรรมที่ถูกล้มเลิกหรือปรับเปลี่ยนไปแล้ว เนื่องจากเหตุผลในตารางต่อไปนี้ เป็นสัดส่วนมากน้อยเพียงใด (ประมาณการ) [ผู้ประกอบการต้องการป้องกันความเสี่ยงที่อาจจะถูกลงโทษตามกฎหมายเฉพาะอื่น ๆ],61.538462,15.384615,7.692308,7.692308,7.692308


In [30]:
TempValueCount = pd.DataFrame(Data['4.3 จากข้อมูลที่ท่านทราบ ในจำนวนกิจกรรมหรือแผนกิจกรรมที่อาจเข้าข่ายเป็นการกระทำผิดฐานปฏิบัติทางการค้าที่ไม่เป็นธรรม ทั้งหมดที่ท่านระบุในข้อ 4.1 มีกิจกรรมหรือแผนกิจกรรมที่ถูกล้มเลิกหรือปรับเปลี่ยนไปแล้ว เนื่องจากเหตุผลในตารางต่อไปนี้ เป็นสัดส่วนมากน้อยเพียงใด (ประมาณการ) [ผู้ประกอบการต้องการป้องกันความเสี่ยงที่อาจจะถูกฟ้องโดยผู้ประกอบการรายอื่น]'].value_counts()).transpose()

TempValueCount_Excluded = TempValueCount.loc[:, TempValueCount.columns!='ไม่ทราบ/ไม่แน่ใจ']

Unfair_343_Propotion = (TempValueCount_Excluded/TempValueCount_Excluded.sum(1)[0])*100

Unfair_343_Propotion

,ร้อยละ 0-20,ร้อยละ 21-40,ร้อยละ 41-60,ร้อยละ 61-80,ร้อยละ 81-100
4.3 จากข้อมูลที่ท่านทราบ ในจำนวนกิจกรรมหรือแผนกิจกรรมที่อาจเข้าข่ายเป็นการกระทำผิดฐานปฏิบัติทางการค้าที่ไม่เป็นธรรม ทั้งหมดที่ท่านระบุในข้อ 4.1 มีกิจกรรมหรือแผนกิจกรรมที่ถูกล้มเลิกหรือปรับเปลี่ยนไปแล้ว เนื่องจากเหตุผลในตารางต่อไปนี้ เป็นสัดส่วนมากน้อยเพียงใด (ประมาณการ) [ผู้ประกอบการต้องการป้องกันความเสี่ยงที่อาจจะถูกฟ้องโดยผู้ประกอบการรายอื่น],50.0,25.0,8.333333,8.333333,8.333333


In [31]:
TempValueCount = pd.DataFrame(Data['4.3 จากข้อมูลที่ท่านทราบ ในจำนวนกิจกรรมหรือแผนกิจกรรมที่อาจเข้าข่ายเป็นการกระทำผิดฐานปฏิบัติทางการค้าที่ไม่เป็นธรรม ทั้งหมดที่ท่านระบุในข้อ 4.1 มีกิจกรรมหรือแผนกิจกรรมที่ถูกล้มเลิกหรือปรับเปลี่ยนไปแล้ว เนื่องจากเหตุผลในตารางต่อไปนี้ เป็นสัดส่วนมากน้อยเพียงใด (ประมาณการ) [ผู้ประกอบการได้รับคำวินิจฉัยล่วงหน้าจากคณะกรรมการการแข่งขันทางการค้า]'].value_counts()).transpose()

TempValueCount_Excluded = TempValueCount.loc[:, TempValueCount.columns!='ไม่ทราบ/ไม่แน่ใจ']

Unfair_344_Propotion = (TempValueCount_Excluded/TempValueCount_Excluded.sum(1)[0])*100

Unfair_344_Propotion

,ร้อยละ 0-20,ร้อยละ 21-40,ร้อยละ 41-60,ร้อยละ 61-80,ร้อยละ 81-100
4.3 จากข้อมูลที่ท่านทราบ ในจำนวนกิจกรรมหรือแผนกิจกรรมที่อาจเข้าข่ายเป็นการกระทำผิดฐานปฏิบัติทางการค้าที่ไม่เป็นธรรม ทั้งหมดที่ท่านระบุในข้อ 4.1 มีกิจกรรมหรือแผนกิจกรรมที่ถูกล้มเลิกหรือปรับเปลี่ยนไปแล้ว เนื่องจากเหตุผลในตารางต่อไปนี้ เป็นสัดส่วนมากน้อยเพียงใด (ประมาณการ) [ผู้ประกอบการได้รับคำวินิจฉัยล่วงหน้าจากคณะกรรมการการแข่งขันทางการค้า],46.153846,23.076923,15.384615,7.692308,7.692308


In [32]:
Summary_Proportion_34 = pd.concat([Unfair_341_Propotion,Unfair_342_Propotion,Unfair_343_Propotion,Unfair_344_Propotion])

Summary_Proportion_34

,ร้อยละ 0-20,ร้อยละ 21-40,ร้อยละ 61-80,ร้อยละ 41-60,ร้อยละ 81-100
4.3 จากข้อมูลที่ท่านทราบ ในจำนวนกิจกรรมหรือแผนกิจกรรมที่อาจเข้าข่ายเป็นการกระทำผิดฐานปฏิบัติทางการค้าที่ไม่เป็นธรรม ทั้งหมดที่ท่านระบุในข้อ 4.1 มีกิจกรรมหรือแผนกิจกรรมที่ถูกล้มเลิกหรือปรับเปลี่ยนไปแล้ว เนื่องจากเหตุผลในตารางต่อไปนี้ เป็นสัดส่วนมากน้อยเพียงใด (ประมาณการ) [ผู้ประกอบการต้องการป้องกันความเสี่ยงที่อาจจะถูกสอบสวนสืบสวนโดย สำนักงาน กขค. และ/หรือถูกลงโทษตาม พ.ร.บ การแข่งขันทางการค้า],46.153846,23.076923,15.384615,7.692308,7.692308
4.3 จากข้อมูลที่ท่านทราบ ในจำนวนกิจกรรมหรือแผนกิจกรรมที่อาจเข้าข่ายเป็นการกระทำผิดฐานปฏิบัติทางการค้าที่ไม่เป็นธรรม ทั้งหมดที่ท่านระบุในข้อ 4.1 มีกิจกรรมหรือแผนกิจกรรมที่ถูกล้มเลิกหรือปรับเปลี่ยนไปแล้ว เนื่องจากเหตุผลในตารางต่อไปนี้ เป็นสัดส่วนมากน้อยเพียงใด (ประมาณการ) [ผู้ประกอบการต้องการป้องกันความเสี่ยงที่อาจจะถูกลงโทษตามกฎหมายเฉพาะอื่น ๆ],61.538462,7.692308,15.384615,7.692308,7.692308
4.3 จากข้อมูลที่ท่านทราบ ในจำนวนกิจกรรมหรือแผนกิจกรรมที่อาจเข้าข่ายเป็นการกระทำผิดฐานปฏิบัติทางการค้าที่ไม่เป็นธรรม ทั้งหมดที่ท่านระบุในข้อ 4.1 มีกิจกรรมหรือแผนกิจกรรมที่ถูกล้มเลิกหรือปรับเปลี่ยนไปแล้ว เนื่องจากเหตุผลในตารางต่อไปนี้ เป็นสัดส่วนมากน้อยเพียงใด (ประมาณการ) [ผู้ประกอบการต้องการป้องกันความเสี่ยงที่อาจจะถูกฟ้องโดยผู้ประกอบการรายอื่น],50.000000,25.000000,8.333333,8.333333,8.333333
4.3 จากข้อมูลที่ท่านทราบ ในจำนวนกิจกรรมหรือแผนกิจกรรมที่อาจเข้าข่ายเป็นการกระทำผิดฐานปฏิบัติทางการค้าที่ไม่เป็นธรรม ทั้งหมดที่ท่านระบุในข้อ 4.1 มีกิจกรรมหรือแผนกิจกรรมที่ถูกล้มเลิกหรือปรับเปลี่ยนไปแล้ว เนื่องจากเหตุผลในตารางต่อไปนี้ เป็นสัดส่วนมากน้อยเพียงใด (ประมาณการ) [ผู้ประกอบการได้รับคำวินิจฉัยล่วงหน้าจากคณะกรรมการการแข่งขันทางการค้า],46.153846,23.076923,7.692308,15.384615,7.692308


### การคำนวณ Median

In [50]:
## Complete Codes for Computing Median _ Merger 

## โค้ดสำหรับตัดคนที่ไม่ทราบออก

TempDataMedian = Data[Data['1.3 ในจำนวนแผนการรวมธุรกิจและการรวมธุรกิจทั้งหมดที่ท่านได้ระบุในข้อ 1.1. มีแผนการรวมธุรกิจและการรวมธุรกิจที่ถูกยกเลิกหรือปรับเปลี่ยนไปแล้ว เนื่องจากเหตุผลในตารางต่อไปนี้ เป็นสัดส่วนมากน้อยเพียงใด (ประมาณการ)  [ผู้ประกอบการเกรงว่าจะไม่ได้รับอนุญาตหรือจะได้รับการตั้งเงื่อนไขการรวมธุรกิจที่เข้มงวดจากคณะกรรมการการแข่งขันทางการค้า]'] != 'ไม่ทราบ/ไม่แน่ใจ']

# โค้ดคำนวณ Median

categories = ['ร้อยละ 0-20', 'ร้อยละ 21-40', 'ร้อยละ 41-60', 'ร้อยละ 61-80', 'ร้อยละ 81-100']

TempDataMedian_Sorted = pd.Categorical(TempDataMedian['1.3 ในจำนวนแผนการรวมธุรกิจและการรวมธุรกิจทั้งหมดที่ท่านได้ระบุในข้อ 1.1. มีแผนการรวมธุรกิจและการรวมธุรกิจที่ถูกยกเลิกหรือปรับเปลี่ยนไปแล้ว เนื่องจากเหตุผลในตารางต่อไปนี้ เป็นสัดส่วนมากน้อยเพียงใด (ประมาณการ)  [ผู้ประกอบการเกรงว่าจะไม่ได้รับอนุญาตหรือจะได้รับการตั้งเงื่อนไขการรวมธุรกิจที่เข้มงวดจากคณะกรรมการการแข่งขันทางการค้า]']
                                       , categories, ordered = True)

MedianValue_Merger = np.median(TempDataMedian_Sorted.codes)
MedianText_Merger = categories[int(MedianValue_Merger)]
MedianText_Merger

'ร้อยละ 21-40'

In [51]:
## Complete Codes for Computing Median _ Abuse of Dominance 

## โค้ดสำหรับตัดคนที่ไม่ทราบออก

TempDataMedian = Data[Data['2.3 จากข้อมูลที่ท่านทราบ ในจำนวนกิจกรรมหรือแผนกิจกรรมที่อาจเข้าข่ายเป็นการกระทำผิดฐานใช้อำนาจเหนือตลาดอย่างไม่เป็นธรรม ทั้งหมดที่ท่านระบุในข้อ 2.1 มีกิจกรรมหรือแผนกิจกรรมที่ถูกยกเลิกหรือปรับเปลี่ยนไปแล้ว เนื่องจากเหตุผลในตารางต่อไปนี้ เป็นสัดส่วนมากน้อยเพียงใด (ประมาณการ) [ผู้ประกอบการต้องการป้องกันความเสี่ยงที่อาจจะถูกสอบสวนสืบสวนโดย สำนักงาน กขค. และ/หรือถูกลงโทษตาม พ.ร.บ การแข่งขันทางการค้า]'] != 'ไม่ทราบ/ไม่แน่ใจ']

# โดยหลังการ Value Count ก็ตัด column สุดท้ายออก

categories = ['ร้อยละ 0-20', 'ร้อยละ 21-40', 'ร้อยละ 41-60', 'ร้อยละ 61-80', 'ร้อยละ 81-100']

TempDataMedian_Sorted = pd.Categorical(TempDataMedian['2.3 จากข้อมูลที่ท่านทราบ ในจำนวนกิจกรรมหรือแผนกิจกรรมที่อาจเข้าข่ายเป็นการกระทำผิดฐานใช้อำนาจเหนือตลาดอย่างไม่เป็นธรรม ทั้งหมดที่ท่านระบุในข้อ 2.1 มีกิจกรรมหรือแผนกิจกรรมที่ถูกยกเลิกหรือปรับเปลี่ยนไปแล้ว เนื่องจากเหตุผลในตารางต่อไปนี้ เป็นสัดส่วนมากน้อยเพียงใด (ประมาณการ) [ผู้ประกอบการต้องการป้องกันความเสี่ยงที่อาจจะถูกสอบสวนสืบสวนโดย สำนักงาน กขค. และ/หรือถูกลงโทษตาม พ.ร.บ การแข่งขันทางการค้า]']
                                       , categories, ordered = True)

MedianValue_Abuse = np.median(TempDataMedian_Sorted.codes)
MedianText_Abuse = categories[int(MedianValue_Abuse)]
MedianText_Abuse

'ร้อยละ 0-20'

In [52]:
## Complete Codes for Computing Median _ Cartels

TempDataMedian = Data[Data['3.3 จากข้อมูลที่ท่านทราบ ในจำนวนความร่วมมือหรือแผนความร่วมมือกับผู้ประกอบการรายอื่นที่อาจเข้าข่ายเป็นการกระทำผิดฐานตกลงร่วมกัน ผูกขาด ลด และจำกัดการแข่งขัน ทั้งหมดที่ท่านระบุในข้อ 3.1 มีความร่วมมือหรือแผนความร่วมมือที่ถูกยกเลิกหรือปรับเปลี่ยนไปแล้ว เนื่องจากเหตุผลในตารางต่อไปนี้ เป็นสัดส่วนมากน้อยเพียงใด (ประมาณการ) [ผู้ประกอบการต้องการป้องกันความเสี่ยงที่อาจจะถูกสอบสวนสืบสวนโดย สำนักงาน กขค. และ/หรือถูกลงโทษตาม พ.ร.บ การแข่งขันทางการค้า]'] != 'ไม่ทราบ/ไม่แน่ใจ']

categories = ['ร้อยละ 0-20', 'ร้อยละ 21-40', 'ร้อยละ 41-60', 'ร้อยละ 61-80', 'ร้อยละ 81-100']

TempDataMedian_Sorted = pd.Categorical(TempDataMedian['3.3 จากข้อมูลที่ท่านทราบ ในจำนวนความร่วมมือหรือแผนความร่วมมือกับผู้ประกอบการรายอื่นที่อาจเข้าข่ายเป็นการกระทำผิดฐานตกลงร่วมกัน ผูกขาด ลด และจำกัดการแข่งขัน ทั้งหมดที่ท่านระบุในข้อ 3.1 มีความร่วมมือหรือแผนความร่วมมือที่ถูกยกเลิกหรือปรับเปลี่ยนไปแล้ว เนื่องจากเหตุผลในตารางต่อไปนี้ เป็นสัดส่วนมากน้อยเพียงใด (ประมาณการ) [ผู้ประกอบการต้องการป้องกันความเสี่ยงที่อาจจะถูกสอบสวนสืบสวนโดย สำนักงาน กขค. และ/หรือถูกลงโทษตาม พ.ร.บ การแข่งขันทางการค้า]']
                                       , categories, ordered = True)

MedianValue_Cartels = np.median(TempDataMedian_Sorted.codes)
MedianText_Cartels = categories[int(MedianValue_Cartels)]
MedianText_Cartels



'ร้อยละ 0-20'

In [53]:
## Complete Codes for Computing Median _ Unfair

TempDataMedian = Data[Data['4.3 จากข้อมูลที่ท่านทราบ ในจำนวนกิจกรรมหรือแผนกิจกรรมที่อาจเข้าข่ายเป็นการกระทำผิดฐานปฏิบัติทางการค้าที่ไม่เป็นธรรม ทั้งหมดที่ท่านระบุในข้อ 4.1 มีกิจกรรมหรือแผนกิจกรรมที่ถูกล้มเลิกหรือปรับเปลี่ยนไปแล้ว เนื่องจากเหตุผลในตารางต่อไปนี้ เป็นสัดส่วนมากน้อยเพียงใด (ประมาณการ) [ผู้ประกอบการต้องการป้องกันความเสี่ยงที่อาจจะถูกสอบสวนสืบสวนโดย สำนักงาน กขค. และ/หรือถูกลงโทษตาม พ.ร.บ การแข่งขันทางการค้า]'] != 'ไม่ทราบ/ไม่แน่ใจ']

categories = ['ร้อยละ 0-20', 'ร้อยละ 21-40', 'ร้อยละ 41-60', 'ร้อยละ 61-80', 'ร้อยละ 81-100']

TempDataMedian_Sorted = pd.Categorical(TempDataMedian['4.3 จากข้อมูลที่ท่านทราบ ในจำนวนกิจกรรมหรือแผนกิจกรรมที่อาจเข้าข่ายเป็นการกระทำผิดฐานปฏิบัติทางการค้าที่ไม่เป็นธรรม ทั้งหมดที่ท่านระบุในข้อ 4.1 มีกิจกรรมหรือแผนกิจกรรมที่ถูกล้มเลิกหรือปรับเปลี่ยนไปแล้ว เนื่องจากเหตุผลในตารางต่อไปนี้ เป็นสัดส่วนมากน้อยเพียงใด (ประมาณการ) [ผู้ประกอบการต้องการป้องกันความเสี่ยงที่อาจจะถูกสอบสวนสืบสวนโดย สำนักงาน กขค. และ/หรือถูกลงโทษตาม พ.ร.บ การแข่งขันทางการค้า]']
                                       , categories, ordered = True)

MedianValue_Unfair = np.median(TempDataMedian_Sorted.codes)
MedianText_Unfair = categories[int(MedianValue_Unfair)]
MedianText_Unfair

'ร้อยละ 21-40'

In [54]:
## รวมเป็น Dataframe
Median_Value = [MedianText_Merger,MedianText_Abuse,MedianText_Cartels,MedianText_Unfair]
Median_Type = ["Merger" , "Abuse" , "Cartels" , "Unfair"]

Summary_Median = pd.DataFrame(Median_Value, Median_Type)

Summary_Median.columns = ['Deterrence']

Summary_Median

,Deterrence
Merger,ร้อยละ 21-40
Abuse,ร้อยละ 0-20
Cartels,ร้อยละ 0-20
Unfair,ร้อยละ 21-40


## Missing deter

In [33]:
Data.columns

Index(['ประทับเวลา',
       'หลังจากท่านได้ทราบถึงภาพรวมและวัตถุประสงค์ของแบบสอบถามฉบับนี้แล้ว ท่านยินดีตอบแบบสอบถามฉบับนี้หรือไม่',
       'ประเภทอุตสาหกรรมหลักของสถานประกอบการ ',
       'กิจการของท่านอยู่ภายใต้การกำกับของหน่วยงานใดบ้าง (สามารถตอบได้มากกว่าหนึ่งข้อ)',
       'รายได้ต่อปีของสถานประกอบการ ในปี พ.ศ. 2562 (ก่อนวิกฤตโควิด)',
       'จำนวนลูกจ้างในสถานประกอบการ',
       'ในกรณีที่จำเป็น ท่านยินดีให้ผู้วิจัย (สถาบันวิจัยเพื่อการพัฒนาประเทศไทย) ติดต่อท่านอีกครั้งเพื่อรวบรวมข้อมูลเพิ่มเติมจากข้อมูลที่ได้ท่านให้ไว้หรือไม่ ',
       'เบอร์โทรศัพท์',
       'สิ่งใดเป็นความผิดตามพระราชบัญญัติการแข่งขันทางการค้า พ.ศ. 2560',
       'ลักษณะใดเป็นเกณฑ์ของการเป็นผู้มีอำนาจเหนือตลาดตามพระราชบัญญัติการแข่งขันทางการค้า พ.ศ. 2560',
       'ค่าปรับสำหรับการกระทำผิดตามพระราชบัญญัติการแข่งขันทางการค้า พ.ศ. 2560 ขึ้นกับปัจจัยอะไรบ้าง',
       'การกระทำผิดในข้อใดหากฝ่าฝืนแล้วมีโทษจำคุก ',
       'การรวมธุรกิจลักษณะใดต้องได้รับอนุญาตจากคณะกรรมการการแข่งขันทางการค้า',
       'ข้อใดคือที่อยู่ของสำ

In [34]:
## Missing Practice

Data['1.3 ในจำนวนแผนการรวมธุรกิจและการรวมธุรกิจทั้งหมดที่ท่านได้ระบุในข้อ 1.1. มีแผนการรวมธุรกิจและการรวมธุรกิจที่ถูกยกเลิกหรือปรับเปลี่ยนไปแล้ว เนื่องจากเหตุผลในตารางต่อไปนี้ เป็นสัดส่วนมากน้อยเพียงใด (ประมาณการ)  [ผู้ประกอบการเกรงว่าจะไม่ได้รับอนุญาตหรือจะได้รับการตั้งเงื่อนไขการรวมธุรกิจที่เข้มงวดจากคณะกรรมการการแข่งขันทางการค้า]'].value_counts()['ไม่ทราบ/ไม่แน่ใจ']


20

In [35]:
Data['2.3 จากข้อมูลที่ท่านทราบ ในจำนวนกิจกรรมหรือแผนกิจกรรมที่อาจเข้าข่ายเป็นการกระทำผิดฐานใช้อำนาจเหนือตลาดอย่างไม่เป็นธรรม ทั้งหมดที่ท่านระบุในข้อ 2.1 มีกิจกรรมหรือแผนกิจกรรมที่ถูกยกเลิกหรือปรับเปลี่ยนไปแล้ว เนื่องจากเหตุผลในตารางต่อไปนี้ เป็นสัดส่วนมากน้อยเพียงใด (ประมาณการ) [ผู้ประกอบการต้องการป้องกันความเสี่ยงที่อาจจะถูกฟ้องโดยผู้ประกอบการรายอื่น]'].value_counts()['ไม่ทราบ/ไม่แน่ใจ']

21

In [36]:
Data['3.3 จากข้อมูลที่ท่านทราบ ในจำนวนความร่วมมือหรือแผนความร่วมมือกับผู้ประกอบการรายอื่นที่อาจเข้าข่ายเป็นการกระทำผิดฐานตกลงร่วมกัน ผูกขาด ลด และจำกัดการแข่งขัน ทั้งหมดที่ท่านระบุในข้อ 3.1 มีความร่วมมือหรือแผนความร่วมมือที่ถูกยกเลิกหรือปรับเปลี่ยนไปแล้ว เนื่องจากเหตุผลในตารางต่อไปนี้ เป็นสัดส่วนมากน้อยเพียงใด (ประมาณการ) [ผู้ประกอบการต้องการป้องกันความเสี่ยงที่อาจจะถูกสอบสวนสืบสวนโดย สำนักงาน กขค. และ/หรือถูกลงโทษตาม พ.ร.บ การแข่งขันทางการค้า]'].value_counts()['ไม่ทราบ/ไม่แน่ใจ']

22

In [37]:
Data['4.3 จากข้อมูลที่ท่านทราบ ในจำนวนกิจกรรมหรือแผนกิจกรรมที่อาจเข้าข่ายเป็นการกระทำผิดฐานปฏิบัติทางการค้าที่ไม่เป็นธรรม ทั้งหมดที่ท่านระบุในข้อ 4.1 มีกิจกรรมหรือแผนกิจกรรมที่ถูกล้มเลิกหรือปรับเปลี่ยนไปแล้ว เนื่องจากเหตุผลในตารางต่อไปนี้ เป็นสัดส่วนมากน้อยเพียงใด (ประมาณการ) [ผู้ประกอบการต้องการป้องกันความเสี่ยงที่อาจจะถูกสอบสวนสืบสวนโดย สำนักงาน กขค. และ/หรือถูกลงโทษตาม พ.ร.บ การแข่งขันทางการค้า]'].value_counts()['ไม่ทราบ/ไม่แน่ใจ']

22

In [38]:
Data['1.3 ในจำนวนแผนการรวมธุรกิจและการรวมธุรกิจทั้งหมดที่ท่านได้ระบุในข้อ 1.1. มีแผนการรวมธุรกิจและการรวมธุรกิจที่ถูกยกเลิกหรือปรับเปลี่ยนไปแล้ว เนื่องจากเหตุผลในตารางต่อไปนี้ เป็นสัดส่วนมากน้อยเพียงใด (ประมาณการ)  [ผู้ประกอบการเกรงว่าจะไม่ได้รับอนุญาตหรือจะได้รับการตั้งเงื่อนไขการรวมธุรกิจที่เข้มงวด จากกฎหมายเฉพาะฉบับอื่น ๆ]'].value_counts()['ไม่ทราบ/ไม่แน่ใจ']

20

In [39]:
Data['2.3 จากข้อมูลที่ท่านทราบ ในจำนวนกิจกรรมหรือแผนกิจกรรมที่อาจเข้าข่ายเป็นการกระทำผิดฐานใช้อำนาจเหนือตลาดอย่างไม่เป็นธรรม ทั้งหมดที่ท่านระบุในข้อ 2.1 มีกิจกรรมหรือแผนกิจกรรมที่ถูกยกเลิกหรือปรับเปลี่ยนไปแล้ว เนื่องจากเหตุผลในตารางต่อไปนี้ เป็นสัดส่วนมากน้อยเพียงใด (ประมาณการ) [ผู้ประกอบการต้องการป้องกันความเสี่ยงที่อาจจะถูกลงโทษตามกฎหมายเฉพาะอื่น ๆ]'].value_counts()['ไม่ทราบ/ไม่แน่ใจ']

21

In [40]:
Data['2.3 จากข้อมูลที่ท่านทราบ ในจำนวนกิจกรรมหรือแผนกิจกรรมที่อาจเข้าข่ายเป็นการกระทำผิดฐานใช้อำนาจเหนือตลาดอย่างไม่เป็นธรรม ทั้งหมดที่ท่านระบุในข้อ 2.1 มีกิจกรรมหรือแผนกิจกรรมที่ถูกยกเลิกหรือปรับเปลี่ยนไปแล้ว เนื่องจากเหตุผลในตารางต่อไปนี้ เป็นสัดส่วนมากน้อยเพียงใด (ประมาณการ) [ผู้ประกอบการต้องการป้องกันความเสี่ยงที่อาจจะถูกฟ้องโดยผู้ประกอบการรายอื่น]'].value_counts()['ไม่ทราบ/ไม่แน่ใจ']

21

In [41]:
Data['2.3 จากข้อมูลที่ท่านทราบ ในจำนวนกิจกรรมหรือแผนกิจกรรมที่อาจเข้าข่ายเป็นการกระทำผิดฐานใช้อำนาจเหนือตลาดอย่างไม่เป็นธรรม ทั้งหมดที่ท่านระบุในข้อ 2.1 มีกิจกรรมหรือแผนกิจกรรมที่ถูกยกเลิกหรือปรับเปลี่ยนไปแล้ว เนื่องจากเหตุผลในตารางต่อไปนี้ เป็นสัดส่วนมากน้อยเพียงใด (ประมาณการ) [ผู้ประกอบการได้รับคำวินิจฉัยล่วงหน้าจากคณะกรรมการการแข่งขันทางการค้า]'].value_counts()['ไม่ทราบ/ไม่แน่ใจ']

21

In [42]:
Data['3.3 จากข้อมูลที่ท่านทราบ ในจำนวนความร่วมมือหรือแผนความร่วมมือกับผู้ประกอบการรายอื่นที่อาจเข้าข่ายเป็นการกระทำผิดฐานตกลงร่วมกัน ผูกขาด ลด และจำกัดการแข่งขัน ทั้งหมดที่ท่านระบุในข้อ 3.1 มีความร่วมมือหรือแผนความร่วมมือที่ถูกยกเลิกหรือปรับเปลี่ยนไปแล้ว เนื่องจากเหตุผลในตารางต่อไปนี้ เป็นสัดส่วนมากน้อยเพียงใด (ประมาณการ) [ผู้ประกอบการต้องการป้องกันความเสี่ยงที่อาจจะถูกลงโทษตามกฎหมายเฉพาะอื่น ๆ]'].value_counts()['ไม่ทราบ/ไม่แน่ใจ']

23

In [43]:
Data['3.3 จากข้อมูลที่ท่านทราบ ในจำนวนความร่วมมือหรือแผนความร่วมมือกับผู้ประกอบการรายอื่นที่อาจเข้าข่ายเป็นการกระทำผิดฐานตกลงร่วมกัน ผูกขาด ลด และจำกัดการแข่งขัน ทั้งหมดที่ท่านระบุในข้อ 3.1 มีความร่วมมือหรือแผนความร่วมมือที่ถูกยกเลิกหรือปรับเปลี่ยนไปแล้ว เนื่องจากเหตุผลในตารางต่อไปนี้ เป็นสัดส่วนมากน้อยเพียงใด (ประมาณการ) [ผู้ประกอบการต้องการป้องกันความเสี่ยงที่อาจจะถูกฟ้องโดยผู้ประกอบการรายอื่น]'].value_counts()['ไม่ทราบ/ไม่แน่ใจ']

22

In [44]:
Data['3.3 จากข้อมูลที่ท่านทราบ ในจำนวนความร่วมมือหรือแผนความร่วมมือกับผู้ประกอบการรายอื่นที่อาจเข้าข่ายเป็นการกระทำผิดฐานตกลงร่วมกัน ผูกขาด ลด และจำกัดการแข่งขัน ทั้งหมดที่ท่านระบุในข้อ 3.1 มีความร่วมมือหรือแผนความร่วมมือที่ถูกยกเลิกหรือปรับเปลี่ยนไปแล้ว เนื่องจากเหตุผลในตารางต่อไปนี้ เป็นสัดส่วนมากน้อยเพียงใด (ประมาณการ) [ผู้ประกอบการได้รับคำวินิจฉัยล่วงหน้าจากคณะกรรมการการแข่งขันทางการค้า]'].value_counts()['ไม่ทราบ/ไม่แน่ใจ']

22

In [45]:
Data['4.3 จากข้อมูลที่ท่านทราบ ในจำนวนกิจกรรมหรือแผนกิจกรรมที่อาจเข้าข่ายเป็นการกระทำผิดฐานปฏิบัติทางการค้าที่ไม่เป็นธรรม ทั้งหมดที่ท่านระบุในข้อ 4.1 มีกิจกรรมหรือแผนกิจกรรมที่ถูกล้มเลิกหรือปรับเปลี่ยนไปแล้ว เนื่องจากเหตุผลในตารางต่อไปนี้ เป็นสัดส่วนมากน้อยเพียงใด (ประมาณการ) [ผู้ประกอบการต้องการป้องกันความเสี่ยงที่อาจจะถูกลงโทษตามกฎหมายเฉพาะอื่น ๆ]'].value_counts()['ไม่ทราบ/ไม่แน่ใจ']

22

In [46]:
Data['4.3 จากข้อมูลที่ท่านทราบ ในจำนวนกิจกรรมหรือแผนกิจกรรมที่อาจเข้าข่ายเป็นการกระทำผิดฐานปฏิบัติทางการค้าที่ไม่เป็นธรรม ทั้งหมดที่ท่านระบุในข้อ 4.1 มีกิจกรรมหรือแผนกิจกรรมที่ถูกล้มเลิกหรือปรับเปลี่ยนไปแล้ว เนื่องจากเหตุผลในตารางต่อไปนี้ เป็นสัดส่วนมากน้อยเพียงใด (ประมาณการ) [ผู้ประกอบการต้องการป้องกันความเสี่ยงที่อาจจะถูกฟ้องโดยผู้ประกอบการรายอื่น]'].value_counts()['ไม่ทราบ/ไม่แน่ใจ']

23

In [47]:
Data['4.3 จากข้อมูลที่ท่านทราบ ในจำนวนกิจกรรมหรือแผนกิจกรรมที่อาจเข้าข่ายเป็นการกระทำผิดฐานปฏิบัติทางการค้าที่ไม่เป็นธรรม ทั้งหมดที่ท่านระบุในข้อ 4.1 มีกิจกรรมหรือแผนกิจกรรมที่ถูกล้มเลิกหรือปรับเปลี่ยนไปแล้ว เนื่องจากเหตุผลในตารางต่อไปนี้ เป็นสัดส่วนมากน้อยเพียงใด (ประมาณการ) [ผู้ประกอบการได้รับคำวินิจฉัยล่วงหน้าจากคณะกรรมการการแข่งขันทางการค้า]'].value_counts()['ไม่ทราบ/ไม่แน่ใจ']

22

In [48]:
Data[].value_counts()['ไม่ทราบ/ไม่แน่ใจ']

SyntaxError: invalid syntax (<ipython-input-48-00f721929bf1>, line 1)

In [ ]:
Data[].value_counts()['ไม่ทราบ/ไม่แน่ใจ']

## Missing Attitude

In [48]:
Data['ท่านคิดเห็นอย่างไรต่อ พระราชบัญญัติการแข่งขันทางการค้า พ.ศ. 2560 และการบังคับใช้โดยสำนักงานคณะกรรมการการแข่งขันทางการค้า  [สำนักงานฯ บังคับใช้กฎหมายอย่างมีประสิทธิภาพในการยับยั้ง *การรวมธุรกิจที่ส่งผลลบต่อการแข่งขันของตลาดอย่างรุนแรง*]'].value_counts()['ไม่สามารถตอบได้']

2

In [49]:
Data['ท่านคิดเห็นอย่างไรต่อ พระราชบัญญัติการแข่งขันทางการค้า พ.ศ. 2560 และการบังคับใช้โดยสำนักงานคณะกรรมการการแข่งขันทางการค้า  [สำนักงานฯ บังคับใช้กฎหมายอย่างมีประสิทธิภาพในการยับยั้ง *การตกลงร่วมกัน ผูกขาด ลด และจำกัดการแข่งขัน*]'].value_counts()['ไม่สามารถตอบได้']

2

In [50]:
Data['ท่านคิดเห็นอย่างไรต่อ พระราชบัญญัติการแข่งขันทางการค้า พ.ศ. 2560 และการบังคับใช้โดยสำนักงานคณะกรรมการการแข่งขันทางการค้า  [สำนักงานฯ บังคับใช้กฎหมายอย่างมีประสิทธิภาพในการยับยั้ง *การใช้อำนาจเหนือตลาดอย่างไม่เป็นธรรม*]'].value_counts()['ไม่สามารถตอบได้']

2

In [51]:
Data['ท่านคิดเห็นอย่างไรต่อ พระราชบัญญัติการแข่งขันทางการค้า พ.ศ. 2560 และการบังคับใช้โดยสำนักงานคณะกรรมการการแข่งขันทางการค้า  [สำนักงานฯ บังคับใช้กฎหมายอย่างมีประสิทธิภาพในการยับยั้ง *การปฏิบัติทางการค้าที่ไม่เป็นธรรม*]'].value_counts()['ไม่สามารถตอบได้']

2

In [52]:
Data['ท่านคิดเห็นอย่างไรต่อ พระราชบัญญัติการแข่งขันทางการค้า พ.ศ. 2560 และการบังคับใช้โดยสำนักงานคณะกรรมการการแข่งขันทางการค้า  [กฎหมาย มาตรา 57 (การปฏิบัติทางการค้าที่ไม่เป็นธรรม) ควรอยู่ถูกบรรจุอยู่ใน พระราชบัญญัติการแข่งขันทางการค้า พ.ศ. 2560 ดังเช่นในปัจจุบัน มากกว่าการบรรจุอยู่ในกฎหมายลำดับรองฉบับอื่น]'].value_counts()['ไม่สามารถตอบได้']

5

In [53]:
Data['ภายใต้สถานการณ์ในปัจจุบัน ท่านคิดว่าข้อความเหล่านี้เป็นจริงมากน้อยเพียงใด  [สำนักงานฯ ได้รับอิทธิพลจากผู้อื่น/หรืออยู่ภายใต้คำสั่งของหน่วยงานภายนอก เช่น หน่วยงานของรัฐ หน่วยงานเอกชน ผู้มีอำนาจทางการเมือง ฯลฯ]'].value_counts()['ไม่สามารถตอบได้']

7

In [54]:
Data['ภายใต้สถานการณ์ในปัจจุบัน ท่านคิดว่าข้อความเหล่านี้เป็นจริงมากน้อยเพียงใด  [คณะกรรมการแข่งขันฯ มีผลประโยชน์ทับซ้อน (conflict of interest) ส่งผลต่อการตัดสินใจ]'].value_counts()['ไม่สามารถตอบได้']

14

In [55]:
Data['ภายใต้สถานการณ์ในปัจจุบัน ท่านคิดว่าข้อความเหล่านี้เป็นจริงมากน้อยเพียงใด  [ในช่วงของการสืบสวน คณะกรรมการแข่งขันฯ มีอำนาจมากและความสามารถสูงในการสืบค้นและหาข้อมูลที่เกี่ยวข้อง (Investigation power)]'].value_counts()['ไม่สามารถตอบได้']

9

In [56]:
Data['ภายใต้สถานการณ์ในปัจจุบัน ท่านคิดว่าข้อความเหล่านี้เป็นจริงมากน้อยเพียงใด  [สำนักงานฯ มีทรัพยากรบุคคลเพียงพอ ต่อการบังคับใช้กฎหมายอย่างมีประสิทธิภาพ เช่น ต้องดำเนินการตรวจสอบในหลายกรณีพร้อมกันในต่างสถานที่ ฯลฯ]'].value_counts()['ไม่สามารถตอบได้']

14

In [57]:
Data['ภายใต้สถานการณ์ในปัจจุบัน ท่านคิดว่าข้อความเหล่านี้เป็นจริงมากน้อยเพียงใด  [สำนักงานฯ มีทรัพยากรด้านการเงินเพียงพอ ต่อการบังคับใช้กฎหมายอย่างมีประสิทธิภาพ]'].value_counts()['ไม่สามารถตอบได้']

18

In [58]:
Data['ภายใต้สถานการณ์ในปัจจุบัน ท่านคิดว่าข้อความเหล่านี้เป็นจริงมากน้อยเพียงใด  [เจ้าพนักงานมีทักษะและอำนาจทางกฎหมายที่เอื้อให้สามารถทำการตรวจสอบและสอบสวนได้อย่างมีประสิทธิภาพ]'].value_counts()['ไม่สามารถตอบได้']

13

In [59]:
Data['ภายใต้สถานการณ์ในปัจจุบัน ท่านคิดว่าข้อความเหล่านี้เป็นจริงมากน้อยเพียงใด  [ผู้กระทำความผิดตามกฎหมายแข่งขันทางการค้ามีโอกาสโดนลงโทษสูง]'].value_counts()['ไม่สามารถตอบได้']

5

In [60]:
Data['ภายใต้สถานการณ์ในปัจจุบัน ท่านคิดว่าข้อความเหล่านี้เป็นจริงมากน้อยเพียงใด  [สำนักงานฯ มีอำนาจและเครื่องมือในเรื่องการกำหนดค่าปรับ/โทษที่มีประสิทธิภาพ]'].value_counts()['ไม่สามารถตอบได้']

7

In [61]:
Data['ภายใต้สถานการณ์ในปัจจุบัน ท่านคิดว่าข้อความเหล่านี้เป็นจริงมากน้อยเพียงใด  [ระดับของโทษ/ค่าปรับในภาพรวมของกฎหมาย อยู่ในระดับที่สามารถยับยั้งการกระทำที่ขัดต่อกฎหมาย]'].value_counts()['ไม่สามารถตอบได้']

9

In [62]:
Data['ภายใต้สถานการณ์ในปัจจุบัน ท่านคิดว่าข้อความเหล่านี้เป็นจริงมากน้อยเพียงใด  [ระดับของค่าปรับในกฎหมาย อยู่ในระดับที่สามารถยับยั้ง การใช้อำนาจเหนือตลาดอย่างไม่เป็นธรรม (abuse of dominant position)]'].value_counts()['ไม่สามารถตอบได้']

7

In [63]:
Data['ภายใต้สถานการณ์ในปัจจุบัน ท่านคิดว่าข้อความเหล่านี้เป็นจริงมากน้อยเพียงใด  [ระดับของค่าปรับในกฎหมาย อยู่ในระดับที่สามารถยับยั้ง การรวมธุรกิจที่ผิดกฎหมายแข่งขันทางการค้า]'].value_counts()['ไม่สามารถตอบได้']

8

In [64]:
Data['ภายใต้สถานการณ์ในปัจจุบัน ท่านคิดว่าข้อความเหล่านี้เป็นจริงมากน้อยเพียงใด  [ระดับของค่าปรับในกฎหมาย อยู่ในระดับที่สามารถยับยั้ง การปฏิบัติการค้าที่ไม่เป็นธรรม]'].value_counts()['ไม่สามารถตอบได้']

6

In [65]:
Data['ภายใต้สถานการณ์ในปัจจุบัน ท่านคิดว่าข้อความเหล่านี้เป็นจริงมากน้อยเพียงใด  [ระดับของค่าปรับในกฎหมาย อยู่ในระดับที่สามารถยับยั้ง การกระทำผิดฐานตกลงร่วมกัน ผูกขาด ลด และจำกัดการแข่งขัน (cartel & bid-riggings)]'].value_counts()['ไม่สามารถตอบได้']

7

In [66]:
Data['ภายใต้สถานการณ์ในปัจจุบัน ท่านคิดว่าข้อความเหล่านี้เป็นจริงมากน้อยเพียงใด  [สำนักงานฯ ใช้เวลาในการสืบสวนสอบสวนและตัดสินคดีนานเกินกว่าที่ควร]'].value_counts()['ไม่สามารถตอบได้']

12

In [67]:
Data['ตามความเห็นของท่าน ท่านคิดว่าการลงโทษหรือเครื่องมือแต่ละประเภทดังตารางด้านล่าง ทั้งประเภทที่มีอยู่ในกฎหมายปัจจุบันและประเภทที่อาจถูกเพิ่มขึ้นมาในอนาคต มีความสำคัญต่อการยับยั้งการกระทำที่ขัดต่อกฎหมายการแข่งขันทางการค้าในประเทศไทยมากน้อยเพียงใด  [การโดนฟ้องร้องเรียกค่าเสียหายจากผู้ประกอบการที่ได้รับผลกระทบจากการกระทำผิด (private damages actions)]'].value_counts()['ไม่ทราบ']

1

In [68]:
Data['ตามความเห็นของท่าน ท่านคิดว่าการลงโทษหรือเครื่องมือแต่ละประเภทดังตารางด้านล่าง ทั้งประเภทที่มีอยู่ในกฎหมายปัจจุบันและประเภทที่อาจถูกเพิ่มขึ้นมาในอนาคต มีความสำคัญต่อการยับยั้งการกระทำที่ขัดต่อกฎหมายการแข่งขันทางการค้าในประเทศไทยมากน้อยเพียงใด  [โทษปรับบริษัทที่กระทำผิด (fines on firm)]'].value_counts()['ไม่ทราบ']

2

In [69]:
Data['ตามความเห็นของท่าน ท่านคิดว่าการลงโทษหรือเครื่องมือแต่ละประเภทดังตารางด้านล่าง ทั้งประเภทที่มีอยู่ในกฎหมายปัจจุบันและประเภทที่อาจถูกเพิ่มขึ้นมาในอนาคต มีความสำคัญต่อการยับยั้งการกระทำที่ขัดต่อกฎหมายการแข่งขันทางการค้าในประเทศไทยมากน้อยเพียงใด  [โทษปรับตัวบุคคลที่กระทำผิด (fines on individual)]'].value_counts()['ไม่ทราบ']

2

In [70]:
Data['ตามความเห็นของท่าน ท่านคิดว่าการลงโทษหรือเครื่องมือแต่ละประเภทดังตารางด้านล่าง ทั้งประเภทที่มีอยู่ในกฎหมายปัจจุบันและประเภทที่อาจถูกเพิ่มขึ้นมาในอนาคต มีความสำคัญต่อการยับยั้งการกระทำที่ขัดต่อกฎหมายการแข่งขันทางการค้าในประเทศไทยมากน้อยเพียงใด  [โทษจำคุกบุคคลที่กระทำผิด (imprisonment)]'].value_counts()['ไม่ทราบ']

1

In [71]:
Data['ตามความเห็นของท่าน ท่านคิดว่าการลงโทษหรือเครื่องมือแต่ละประเภทดังตารางด้านล่าง ทั้งประเภทที่มีอยู่ในกฎหมายปัจจุบันและประเภทที่อาจถูกเพิ่มขึ้นมาในอนาคต มีความสำคัญต่อการยับยั้งการกระทำที่ขัดต่อกฎหมายการแข่งขันทางการค้าในประเทศไทยมากน้อยเพียงใด  [ความเสียหายต่อชื่อเสียงของบริษัท (reputational damage for the company)**]'].value_counts()['ไม่ทราบ']

1

In [72]:
Data['ตามความเห็นของท่าน ท่านคิดว่าการลงโทษหรือเครื่องมือแต่ละประเภทดังตารางด้านล่าง ทั้งประเภทที่มีอยู่ในกฎหมายปัจจุบันและประเภทที่อาจถูกเพิ่มขึ้นมาในอนาคต มีความสำคัญต่อการยับยั้งการกระทำที่ขัดต่อกฎหมายการแข่งขันทางการค้าในประเทศไทยมากน้อยเพียงใด  [ความเสียหายต่อชื่อเสียงของบุคคลที่กระทำผิด (reputational damage for the individual)**]'].value_counts()['ไม่ทราบ']

1

In [73]:
Data['ตามความเห็นของท่าน ท่านคิดว่าการลงโทษหรือเครื่องมือแต่ละประเภทดังตารางด้านล่าง ทั้งประเภทที่มีอยู่ในกฎหมายปัจจุบันและประเภทที่อาจถูกเพิ่มขึ้นมาในอนาคต มีความสำคัญต่อการยับยั้งการกระทำที่ขัดต่อกฎหมายการแข่งขันทางการค้าในประเทศไทยมากน้อยเพียงใด  [คำสั่งปลดกรรมการบริษัท (director disqualification orders)**]'].value_counts()['ไม่ทราบ']

2

In [74]:
Data['ตามความเห็นของท่าน ท่านคิดว่าการลงโทษหรือเครื่องมือแต่ละประเภทดังตารางด้านล่าง ทั้งประเภทที่มีอยู่ในกฎหมายปัจจุบันและประเภทที่อาจถูกเพิ่มขึ้นมาในอนาคต มีความสำคัญต่อการยับยั้งการกระทำที่ขัดต่อกฎหมายการแข่งขันทางการค้าในประเทศไทยมากน้อยเพียงใด  [การลดหย่อนผ่อน/ยกเว้นโทษสำหรับผู้กระทำผิดที่เป็นพยาน (leniency program)** [หากผู้ร่วมกระทำผิดในกรณีการตกลงร่วมกันผูกขาดได้มาแจ้งให้องค์กรที่กำกับดูแลการแข่งขันทราบถึงการกระทำผิด จะได้รับการยกเว้นหรือลดหย่อนโทษ]]'].value_counts()['ไม่ทราบ']

1

## Attitude

In [75]:
Data.columns

Index(['ประทับเวลา',
       'หลังจากท่านได้ทราบถึงภาพรวมและวัตถุประสงค์ของแบบสอบถามฉบับนี้แล้ว ท่านยินดีตอบแบบสอบถามฉบับนี้หรือไม่',
       'ประเภทอุตสาหกรรมหลักของสถานประกอบการ ',
       'กิจการของท่านอยู่ภายใต้การกำกับของหน่วยงานใดบ้าง (สามารถตอบได้มากกว่าหนึ่งข้อ)',
       'รายได้ต่อปีของสถานประกอบการ ในปี พ.ศ. 2562 (ก่อนวิกฤตโควิด)',
       'จำนวนลูกจ้างในสถานประกอบการ',
       'ในกรณีที่จำเป็น ท่านยินดีให้ผู้วิจัย (สถาบันวิจัยเพื่อการพัฒนาประเทศไทย) ติดต่อท่านอีกครั้งเพื่อรวบรวมข้อมูลเพิ่มเติมจากข้อมูลที่ได้ท่านให้ไว้หรือไม่ ',
       'เบอร์โทรศัพท์',
       'สิ่งใดเป็นความผิดตามพระราชบัญญัติการแข่งขันทางการค้า พ.ศ. 2560',
       'ลักษณะใดเป็นเกณฑ์ของการเป็นผู้มีอำนาจเหนือตลาดตามพระราชบัญญัติการแข่งขันทางการค้า พ.ศ. 2560',
       'ค่าปรับสำหรับการกระทำผิดตามพระราชบัญญัติการแข่งขันทางการค้า พ.ศ. 2560 ขึ้นกับปัจจัยอะไรบ้าง',
       'การกระทำผิดในข้อใดหากฝ่าฝืนแล้วมีโทษจำคุก ',
       'การรวมธุรกิจลักษณะใดต้องได้รับอนุญาตจากคณะกรรมการการแข่งขันทางการค้า',
       'ข้อใดคือที่อยู่ของสำ

In [10]:
TempValueCount = pd.DataFrame(Data['ท่านคิดเห็นอย่างไรต่อ พระราชบัญญัติการแข่งขันทางการค้า พ.ศ. 2560 และการบังคับใช้โดยสำนักงานคณะกรรมการการแข่งขันทางการค้า  [สำนักงานฯ บังคับใช้กฎหมายอย่างมีประสิทธิภาพในการยับยั้ง *การรวมธุรกิจที่ส่งผลลบต่อการแข่งขันของตลาดอย่างรุนแรง*]'].value_counts()).transpose()

TempValueCount_Excluded = TempValueCount.loc[:, TempValueCount.columns!='ไม่สามารถตอบได้']

Merger_Attitude_Enforcement_Propotion = (TempValueCount_Excluded/TempValueCount_Excluded.sum(1)[0])*100

Merger_Attitude_Enforcement_Propotion

,เห็นด้วย,เห็นด้วยอย่างยิ่ง,ไม่เห็นด้วย,ไม่เห็นด้วยอย่างยิ่ง
ท่านคิดเห็นอย่างไรต่อ พระราชบัญญัติการแข่งขันทางการค้า พ.ศ. 2560 และการบังคับใช้โดยสำนักงานคณะกรรมการการแข่งขันทางการค้า [สำนักงานฯ บังคับใช้กฎหมายอย่างมีประสิทธิภาพในการยับยั้ง *การรวมธุรกิจที่ส่งผลลบต่อการแข่งขันของตลาดอย่างรุนแรง*],63.636364,21.212121,9.090909,6.060606


In [11]:
TempValueCount = pd.DataFrame(Data['ท่านคิดเห็นอย่างไรต่อ พระราชบัญญัติการแข่งขันทางการค้า พ.ศ. 2560 และการบังคับใช้โดยสำนักงานคณะกรรมการการแข่งขันทางการค้า  [สำนักงานฯ บังคับใช้กฎหมายอย่างมีประสิทธิภาพในการยับยั้ง *การตกลงร่วมกัน ผูกขาด ลด และจำกัดการแข่งขัน*]'].value_counts()).transpose()

TempValueCount_Excluded = TempValueCount.loc[:, TempValueCount.columns!='ไม่สามารถตอบได้']

Abuse_Attitude_Enforcement_Propotion = (TempValueCount_Excluded/TempValueCount_Excluded.sum(1)[0])*100

Abuse_Attitude_Enforcement_Propotion

,เห็นด้วย,เห็นด้วยอย่างยิ่ง,ไม่เห็นด้วย,ไม่เห็นด้วยอย่างยิ่ง,เฉยๆ
ท่านคิดเห็นอย่างไรต่อ พระราชบัญญัติการแข่งขันทางการค้า พ.ศ. 2560 และการบังคับใช้โดยสำนักงานคณะกรรมการการแข่งขันทางการค้า [สำนักงานฯ บังคับใช้กฎหมายอย่างมีประสิทธิภาพในการยับยั้ง *การตกลงร่วมกัน ผูกขาด ลด และจำกัดการแข่งขัน*],57.575758,18.181818,12.121212,9.090909,3.030303


In [12]:
TempValueCount = pd.DataFrame(Data['ท่านคิดเห็นอย่างไรต่อ พระราชบัญญัติการแข่งขันทางการค้า พ.ศ. 2560 และการบังคับใช้โดยสำนักงานคณะกรรมการการแข่งขันทางการค้า  [สำนักงานฯ บังคับใช้กฎหมายอย่างมีประสิทธิภาพในการยับยั้ง *การใช้อำนาจเหนือตลาดอย่างไม่เป็นธรรม*]'].value_counts()).transpose()

TempValueCount_Excluded = TempValueCount.loc[:, TempValueCount.columns!='ไม่สามารถตอบได้']

Cartels_Attitude_Enforcement_Propotion = (TempValueCount_Excluded/TempValueCount_Excluded.sum(1)[0])*100

Cartels_Attitude_Enforcement_Propotion

,เห็นด้วย,เห็นด้วยอย่างยิ่ง,ไม่เห็นด้วย,ไม่เห็นด้วยอย่างยิ่ง
ท่านคิดเห็นอย่างไรต่อ พระราชบัญญัติการแข่งขันทางการค้า พ.ศ. 2560 และการบังคับใช้โดยสำนักงานคณะกรรมการการแข่งขันทางการค้า [สำนักงานฯ บังคับใช้กฎหมายอย่างมีประสิทธิภาพในการยับยั้ง *การใช้อำนาจเหนือตลาดอย่างไม่เป็นธรรม*],51.515152,24.242424,15.151515,9.090909


In [13]:
TempValueCount = pd.DataFrame(Data['ท่านคิดเห็นอย่างไรต่อ พระราชบัญญัติการแข่งขันทางการค้า พ.ศ. 2560 และการบังคับใช้โดยสำนักงานคณะกรรมการการแข่งขันทางการค้า  [สำนักงานฯ บังคับใช้กฎหมายอย่างมีประสิทธิภาพในการยับยั้ง *การปฏิบัติทางการค้าที่ไม่เป็นธรรม*]'].value_counts()).transpose()

TempValueCount_Excluded = TempValueCount.loc[:, TempValueCount.columns!='ไม่สามารถตอบได้']

Unfair_Attitude_Enforcement1_Propotion = (TempValueCount_Excluded/TempValueCount_Excluded.sum(1)[0])*100

Unfair_Attitude_Enforcement1_Propotion

,เห็นด้วย,เห็นด้วยอย่างยิ่ง,ไม่เห็นด้วย,ไม่เห็นด้วยอย่างยิ่ง,เฉยๆ
ท่านคิดเห็นอย่างไรต่อ พระราชบัญญัติการแข่งขันทางการค้า พ.ศ. 2560 และการบังคับใช้โดยสำนักงานคณะกรรมการการแข่งขันทางการค้า [สำนักงานฯ บังคับใช้กฎหมายอย่างมีประสิทธิภาพในการยับยั้ง *การปฏิบัติทางการค้าที่ไม่เป็นธรรม*],48.484848,27.272727,15.151515,6.060606,3.030303


In [14]:
TempValueCount = pd.DataFrame(Data['ท่านคิดเห็นอย่างไรต่อ พระราชบัญญัติการแข่งขันทางการค้า พ.ศ. 2560 และการบังคับใช้โดยสำนักงานคณะกรรมการการแข่งขันทางการค้า  [กฎหมาย มาตรา 57 (การปฏิบัติทางการค้าที่ไม่เป็นธรรม) ควรอยู่ถูกบรรจุอยู่ใน พระราชบัญญัติการแข่งขันทางการค้า พ.ศ. 2560 ดังเช่นในปัจจุบัน มากกว่าการบรรจุอยู่ในกฎหมายลำดับรองฉบับอื่น]'].value_counts()).transpose()

TempValueCount_Excluded = TempValueCount.loc[:, TempValueCount.columns!='ไม่สามารถตอบได้']

Unfair_Attitude_Enforcement2_Propotion = (TempValueCount_Excluded/TempValueCount_Excluded.sum(1)[0])*100

Unfair_Attitude_Enforcement2_Propotion

,เห็นด้วย,เห็นด้วยอย่างยิ่ง,ไม่เห็นด้วยอย่างยิ่ง,เฉยๆ,ไม่เห็นด้วย
ท่านคิดเห็นอย่างไรต่อ พระราชบัญญัติการแข่งขันทางการค้า พ.ศ. 2560 และการบังคับใช้โดยสำนักงานคณะกรรมการการแข่งขันทางการค้า [กฎหมาย มาตรา 57 (การปฏิบัติทางการค้าที่ไม่เป็นธรรม) ควรอยู่ถูกบรรจุอยู่ใน พระราชบัญญัติการแข่งขันทางการค้า พ.ศ. 2560 ดังเช่นในปัจจุบัน มากกว่าการบรรจุอยู่ในกฎหมายลำดับรองฉบับอื่น],60.0,26.666667,6.666667,3.333333,3.333333


In [15]:
Summary_Proportion = pd.concat([Merger_Attitude_Enforcement_Propotion,Abuse_Attitude_Enforcement_Propotion,Cartels_Attitude_Enforcement_Propotion,Unfair_Attitude_Enforcement1_Propotion,Unfair_Attitude_Enforcement2_Propotion])

Summary_Proportion.rename(index = {'ท่านคิดเห็นอย่างไรต่อ พระราชบัญญัติการแข่งขันทางการค้า พ.ศ. 2560 และการบังคับใช้โดยสำนักงานคณะกรรมการการแข่งขันทางการค้า  [สำนักงานฯ บังคับใช้กฎหมายอย่างมีประสิทธิภาพในการยับยั้ง *การรวมธุรกิจที่ส่งผลลบต่อการแข่งขันของตลาดอย่างรุนแรง*]':'Merger'
                                   , 'ท่านคิดเห็นอย่างไรต่อ พระราชบัญญัติการแข่งขันทางการค้า พ.ศ. 2560 และการบังคับใช้โดยสำนักงานคณะกรรมการการแข่งขันทางการค้า  [สำนักงานฯ บังคับใช้กฎหมายอย่างมีประสิทธิภาพในการยับยั้ง *การตกลงร่วมกัน ผูกขาด ลด และจำกัดการแข่งขัน*]':'Abuse'
                                   , 'ท่านคิดเห็นอย่างไรต่อ พระราชบัญญัติการแข่งขันทางการค้า พ.ศ. 2560 และการบังคับใช้โดยสำนักงานคณะกรรมการการแข่งขันทางการค้า  [สำนักงานฯ บังคับใช้กฎหมายอย่างมีประสิทธิภาพในการยับยั้ง *การใช้อำนาจเหนือตลาดอย่างไม่เป็นธรรม*]':'Cartels'
                                   , 'ท่านคิดเห็นอย่างไรต่อ พระราชบัญญัติการแข่งขันทางการค้า พ.ศ. 2560 และการบังคับใช้โดยสำนักงานคณะกรรมการการแข่งขันทางการค้า  [สำนักงานฯ บังคับใช้กฎหมายอย่างมีประสิทธิภาพในการยับยั้ง *การปฏิบัติทางการค้าที่ไม่เป็นธรรม*]':'Unfair1'
                                   , 'ท่านคิดเห็นอย่างไรต่อ พระราชบัญญัติการแข่งขันทางการค้า พ.ศ. 2560 และการบังคับใช้โดยสำนักงานคณะกรรมการการแข่งขันทางการค้า  [กฎหมาย มาตรา 57 (การปฏิบัติทางการค้าที่ไม่เป็นธรรม) ควรอยู่ถูกบรรจุอยู่ใน พระราชบัญญัติการแข่งขันทางการค้า พ.ศ. 2560 ดังเช่นในปัจจุบัน มากกว่าการบรรจุอยู่ในกฎหมายลำดับรองฉบับอื่น]':'Unfair2'}, inplace = True)

Summary_Proportion

,เห็นด้วย,เห็นด้วยอย่างยิ่ง,ไม่เห็นด้วย,ไม่เห็นด้วยอย่างยิ่ง,เฉยๆ
Merger,63.636364,21.212121,9.090909,6.060606,NaN
Abuse,57.575758,18.181818,12.121212,9.090909,3.030303
Cartels,51.515152,24.242424,15.151515,9.090909,NaN
Unfair1,48.484848,27.272727,15.151515,6.060606,3.030303
Unfair2,60.000000,26.666667,3.333333,6.666667,3.333333


In [16]:
Summary_Proportion_attitude = pd.concat([Merger_Attitude_Enforcement_Propotion,Abuse_Attitude_Enforcement_Propotion,Cartels_Attitude_Enforcement_Propotion,Unfair_Attitude_Enforcement1_Propotion,Unfair_Attitude_Enforcement2_Propotion])


Summary_Proportion_attitude

,เห็นด้วย,เห็นด้วยอย่างยิ่ง,ไม่เห็นด้วย,ไม่เห็นด้วยอย่างยิ่ง,เฉยๆ
ท่านคิดเห็นอย่างไรต่อ พระราชบัญญัติการแข่งขันทางการค้า พ.ศ. 2560 และการบังคับใช้โดยสำนักงานคณะกรรมการการแข่งขันทางการค้า [สำนักงานฯ บังคับใช้กฎหมายอย่างมีประสิทธิภาพในการยับยั้ง *การรวมธุรกิจที่ส่งผลลบต่อการแข่งขันของตลาดอย่างรุนแรง*],63.636364,21.212121,9.090909,6.060606,NaN
ท่านคิดเห็นอย่างไรต่อ พระราชบัญญัติการแข่งขันทางการค้า พ.ศ. 2560 และการบังคับใช้โดยสำนักงานคณะกรรมการการแข่งขันทางการค้า [สำนักงานฯ บังคับใช้กฎหมายอย่างมีประสิทธิภาพในการยับยั้ง *การตกลงร่วมกัน ผูกขาด ลด และจำกัดการแข่งขัน*],57.575758,18.181818,12.121212,9.090909,3.030303
ท่านคิดเห็นอย่างไรต่อ พระราชบัญญัติการแข่งขันทางการค้า พ.ศ. 2560 และการบังคับใช้โดยสำนักงานคณะกรรมการการแข่งขันทางการค้า [สำนักงานฯ บังคับใช้กฎหมายอย่างมีประสิทธิภาพในการยับยั้ง *การใช้อำนาจเหนือตลาดอย่างไม่เป็นธรรม*],51.515152,24.242424,15.151515,9.090909,NaN
ท่านคิดเห็นอย่างไรต่อ พระราชบัญญัติการแข่งขันทางการค้า พ.ศ. 2560 และการบังคับใช้โดยสำนักงานคณะกรรมการการแข่งขันทางการค้า [สำนักงานฯ บังคับใช้กฎหมายอย่างมีประสิทธิภาพในการยับยั้ง *การปฏิบัติทางการค้าที่ไม่เป็นธรรม*],48.484848,27.272727,15.151515,6.060606,3.030303
ท่านคิดเห็นอย่างไรต่อ พระราชบัญญัติการแข่งขันทางการค้า พ.ศ. 2560 และการบังคับใช้โดยสำนักงานคณะกรรมการการแข่งขันทางการค้า [กฎหมาย มาตรา 57 (การปฏิบัติทางการค้าที่ไม่เป็นธรรม) ควรอยู่ถูกบรรจุอยู่ใน พระราชบัญญัติการแข่งขันทางการค้า พ.ศ. 2560 ดังเช่นในปัจจุบัน มากกว่าการบรรจุอยู่ในกฎหมายลำดับรองฉบับอื่น],60.000000,26.666667,3.333333,6.666667,3.333333


In [30]:
att1_crosstab_firms_size = pd.crosstab(income, Summary_Proportion_attitude)

ValueError: Shape of passed values is (1, 2), indices imply (0, 2)

In [ ]:
pd.crosstab(index=Data['ประเภทอุตสาหกรรมหลักของสถานประกอบการ '],
                             columns=Data['1.3 ในจำนวนแผนการรวมธุรกิจและการรวมธุรกิจทั้งหมดที่ท่านได้ระบุในข้อ 1.1. มีแผนการรวมธุรกิจและการรวมธุรกิจที่ถูกยกเลิกหรือปรับเปลี่ยนไปแล้ว เนื่องจากเหตุผลในตารางต่อไปนี้ เป็นสัดส่วนมากน้อยเพียงใด (ประมาณการ)  [ผู้ประกอบการเกรงว่าจะไม่ได้รับอนุญาตหรือจะได้รับการตั้งเงื่อนไขการรวมธุรกิจที่เข้มงวด จากกฎหมายเฉพาะฉบับอื่น ๆ]'],
                             normalize="index")

## ภายใต้สถานการณ์ในปัจจุบัน ท่านคิดว่าข้อความเหล่านี้เป็นจริงมากน้อยเพียงใด

In [83]:
TempValueCount = pd.DataFrame(Data['ภายใต้สถานการณ์ในปัจจุบัน ท่านคิดว่าข้อความเหล่านี้เป็นจริงมากน้อยเพียงใด  [สำนักงานฯ ได้รับอิทธิพลจากผู้อื่น/หรืออยู่ภายใต้คำสั่งของหน่วยงานภายนอก เช่น หน่วยงานของรัฐ หน่วยงานเอกชน ผู้มีอำนาจทางการเมือง ฯลฯ]'].value_counts()).transpose()

TempValueCount_Excluded = TempValueCount.loc[:, TempValueCount.columns!='ไม่สามารถตอบได้']

Independent_issue_Propotion = (TempValueCount_Excluded/TempValueCount_Excluded.sum(1)[0])*100

Independent_issue_Propotion

,จริง,ไม่จริง,จริงอย่างยิ่ง,ไม่จริงอย่างยิ่ง,เฉยๆ
ภายใต้สถานการณ์ในปัจจุบัน ท่านคิดว่าข้อความเหล่านี้เป็นจริงมากน้อยเพียงใด [สำนักงานฯ ได้รับอิทธิพลจากผู้อื่น/หรืออยู่ภายใต้คำสั่งของหน่วยงานภายนอก เช่น หน่วยงานของรัฐ หน่วยงานเอกชน ผู้มีอำนาจทางการเมือง ฯลฯ],46.428571,25.0,17.857143,7.142857,3.571429


In [84]:
TempValueCount = pd.DataFrame(Data['ภายใต้สถานการณ์ในปัจจุบัน ท่านคิดว่าข้อความเหล่านี้เป็นจริงมากน้อยเพียงใด  [คณะกรรมการแข่งขันฯ มีผลประโยชน์ทับซ้อน (conflict of interest) ส่งผลต่อการตัดสินใจ]'].value_counts()).transpose()

TempValueCount_Excluded = TempValueCount.loc[:, TempValueCount.columns!='ไม่สามารถตอบได้']

Conflict_issue_Propotion = (TempValueCount_Excluded/TempValueCount_Excluded.sum(1)[0])*100

Conflict_issue_Propotion

,ไม่จริง,จริง,จริงอย่างยิ่ง,ไม่จริงอย่างยิ่ง,เฉยๆ
ภายใต้สถานการณ์ในปัจจุบัน ท่านคิดว่าข้อความเหล่านี้เป็นจริงมากน้อยเพียงใด [คณะกรรมการแข่งขันฯ มีผลประโยชน์ทับซ้อน (conflict of interest) ส่งผลต่อการตัดสินใจ],33.333333,33.333333,23.809524,4.761905,4.761905


In [85]:
TempValueCount = pd.DataFrame(Data['ภายใต้สถานการณ์ในปัจจุบัน ท่านคิดว่าข้อความเหล่านี้เป็นจริงมากน้อยเพียงใด  [ในช่วงของการสืบสวน คณะกรรมการแข่งขันฯ มีอำนาจมากและความสามารถสูงในการสืบค้นและหาข้อมูลที่เกี่ยวข้อง (Investigation power)]'].value_counts()).transpose()

TempValueCount_Excluded = TempValueCount.loc[:, TempValueCount.columns!='ไม่สามารถตอบได้']

Investigation_issue_Propotion = (TempValueCount_Excluded/TempValueCount_Excluded.sum(1)[0])*100

Investigation_issue_Propotion

,จริง,ไม่จริง,ไม่จริงอย่างยิ่ง,เฉยๆ
ภายใต้สถานการณ์ในปัจจุบัน ท่านคิดว่าข้อความเหล่านี้เป็นจริงมากน้อยเพียงใด [ในช่วงของการสืบสวน คณะกรรมการแข่งขันฯ มีอำนาจมากและความสามารถสูงในการสืบค้นและหาข้อมูลที่เกี่ยวข้อง (Investigation power)],42.307692,34.615385,11.538462,11.538462


In [86]:
TempValueCount = pd.DataFrame(Data['ภายใต้สถานการณ์ในปัจจุบัน ท่านคิดว่าข้อความเหล่านี้เป็นจริงมากน้อยเพียงใด  [สำนักงานฯ มีทรัพยากรบุคคลเพียงพอ ต่อการบังคับใช้กฎหมายอย่างมีประสิทธิภาพ เช่น ต้องดำเนินการตรวจสอบในหลายกรณีพร้อมกันในต่างสถานที่ ฯลฯ]'].value_counts()).transpose()

TempValueCount_Excluded = TempValueCount.loc[:, TempValueCount.columns!='ไม่สามารถตอบได้']

Human_Resource_issue_Propotion = (TempValueCount_Excluded/TempValueCount_Excluded.sum(1)[0])*100

Human_Resource_issue_Propotion

,ไม่จริง,จริง,ไม่จริงอย่างยิ่ง,เฉยๆ
ภายใต้สถานการณ์ในปัจจุบัน ท่านคิดว่าข้อความเหล่านี้เป็นจริงมากน้อยเพียงใด [สำนักงานฯ มีทรัพยากรบุคคลเพียงพอ ต่อการบังคับใช้กฎหมายอย่างมีประสิทธิภาพ เช่น ต้องดำเนินการตรวจสอบในหลายกรณีพร้อมกันในต่างสถานที่ ฯลฯ],42.857143,33.333333,14.285714,9.52381


In [87]:
TempValueCount = pd.DataFrame(Data['ภายใต้สถานการณ์ในปัจจุบัน ท่านคิดว่าข้อความเหล่านี้เป็นจริงมากน้อยเพียงใด  [สำนักงานฯ มีทรัพยากรด้านการเงินเพียงพอ ต่อการบังคับใช้กฎหมายอย่างมีประสิทธิภาพ]'].value_counts()).transpose()

TempValueCount_Excluded = TempValueCount.loc[:, TempValueCount.columns!='ไม่สามารถตอบได้']

Financial_Resource_issue_Propotion = (TempValueCount_Excluded/TempValueCount_Excluded.sum(1)[0])*100

Financial_Resource_issue_Propotion

,จริง,ไม่จริง,ไม่จริงอย่างยิ่ง,เฉยๆ,จริงอย่างยิ่ง
ภายใต้สถานการณ์ในปัจจุบัน ท่านคิดว่าข้อความเหล่านี้เป็นจริงมากน้อยเพียงใด [สำนักงานฯ มีทรัพยากรด้านการเงินเพียงพอ ต่อการบังคับใช้กฎหมายอย่างมีประสิทธิภาพ],41.176471,35.294118,11.764706,5.882353,5.882353


In [88]:
TempValueCount = pd.DataFrame(Data['ภายใต้สถานการณ์ในปัจจุบัน ท่านคิดว่าข้อความเหล่านี้เป็นจริงมากน้อยเพียงใด  [เจ้าพนักงานมีทักษะและอำนาจทางกฎหมายที่เอื้อให้สามารถทำการตรวจสอบและสอบสวนได้อย่างมีประสิทธิภาพ]'].value_counts()).transpose()

TempValueCount_Excluded = TempValueCount.loc[:, TempValueCount.columns!='ไม่สามารถตอบได้']

Skill_Staff_issue_Propotion = (TempValueCount_Excluded/TempValueCount_Excluded.sum(1)[0])*100

Skill_Staff_issue_Propotion

,ไม่จริง,จริง,ไม่จริงอย่างยิ่ง,เฉยๆ
ภายใต้สถานการณ์ในปัจจุบัน ท่านคิดว่าข้อความเหล่านี้เป็นจริงมากน้อยเพียงใด [เจ้าพนักงานมีทักษะและอำนาจทางกฎหมายที่เอื้อให้สามารถทำการตรวจสอบและสอบสวนได้อย่างมีประสิทธิภาพ],50.0,36.363636,9.090909,4.545455


In [89]:
TempValueCount = pd.DataFrame(Data['ภายใต้สถานการณ์ในปัจจุบัน ท่านคิดว่าข้อความเหล่านี้เป็นจริงมากน้อยเพียงใด  [ผู้กระทำความผิดตามกฎหมายแข่งขันทางการค้ามีโอกาสโดนลงโทษสูง]'].value_counts()).transpose()

TempValueCount_Excluded = TempValueCount.loc[:, TempValueCount.columns!='ไม่สามารถตอบได้']

Prob_Punishment_Propotion = (TempValueCount_Excluded/TempValueCount_Excluded.sum(1)[0])*100

Prob_Punishment_Propotion

,จริง,ไม่จริง,ไม่จริงอย่างยิ่ง,เฉยๆ,จริงอย่างยิ่ง
ภายใต้สถานการณ์ในปัจจุบัน ท่านคิดว่าข้อความเหล่านี้เป็นจริงมากน้อยเพียงใด [ผู้กระทำความผิดตามกฎหมายแข่งขันทางการค้ามีโอกาสโดนลงโทษสูง],46.666667,23.333333,16.666667,6.666667,6.666667


In [90]:
TempValueCount = pd.DataFrame(Data['ภายใต้สถานการณ์ในปัจจุบัน ท่านคิดว่าข้อความเหล่านี้เป็นจริงมากน้อยเพียงใด  [สำนักงานฯ มีอำนาจและเครื่องมือในเรื่องการกำหนดค่าปรับ/โทษที่มีประสิทธิภาพ]'].value_counts()).transpose()

TempValueCount_Excluded = TempValueCount.loc[:, TempValueCount.columns!='ไม่สามารถตอบได้']

Instrument_issue_Propotion = (TempValueCount_Excluded/TempValueCount_Excluded.sum(1)[0])*100

Instrument_issue_Propotion

,จริง,ไม่จริง,เฉยๆ,ไม่จริงอย่างยิ่ง,จริงอย่างยิ่ง
ภายใต้สถานการณ์ในปัจจุบัน ท่านคิดว่าข้อความเหล่านี้เป็นจริงมากน้อยเพียงใด [สำนักงานฯ มีอำนาจและเครื่องมือในเรื่องการกำหนดค่าปรับ/โทษที่มีประสิทธิภาพ],46.428571,25.0,17.857143,7.142857,3.571429


In [91]:
TempValueCount = pd.DataFrame(Data['ภายใต้สถานการณ์ในปัจจุบัน ท่านคิดว่าข้อความเหล่านี้เป็นจริงมากน้อยเพียงใด  [ระดับของโทษ/ค่าปรับในภาพรวมของกฎหมาย อยู่ในระดับที่สามารถยับยั้งการกระทำที่ขัดต่อกฎหมาย]'].value_counts()).transpose()

TempValueCount_Excluded = TempValueCount.loc[:, TempValueCount.columns!='ไม่สามารถตอบได้']

Punishment_Level_issue_Propotion = (TempValueCount_Excluded/TempValueCount_Excluded.sum(1)[0])*100

Punishment_Level_issue_Propotion

,จริง,ไม่จริง,ไม่จริงอย่างยิ่ง,เฉยๆ,จริงอย่างยิ่ง
ภายใต้สถานการณ์ในปัจจุบัน ท่านคิดว่าข้อความเหล่านี้เป็นจริงมากน้อยเพียงใด [ระดับของโทษ/ค่าปรับในภาพรวมของกฎหมาย อยู่ในระดับที่สามารถยับยั้งการกระทำที่ขัดต่อกฎหมาย],42.307692,26.923077,15.384615,11.538462,3.846154


In [92]:
TempValueCount = pd.DataFrame(Data['ภายใต้สถานการณ์ในปัจจุบัน ท่านคิดว่าข้อความเหล่านี้เป็นจริงมากน้อยเพียงใด  [ระดับของค่าปรับในกฎหมาย อยู่ในระดับที่สามารถยับยั้ง การใช้อำนาจเหนือตลาดอย่างไม่เป็นธรรม (abuse of dominant position)]'].value_counts()).transpose()

TempValueCount_Excluded = TempValueCount.loc[:, TempValueCount.columns!='ไม่สามารถตอบได้']

Fine_Level_Abuse_Propotion = (TempValueCount_Excluded/TempValueCount_Excluded.sum(1)[0])*100

Fine_Level_Abuse_Propotion

,จริง,ไม่จริง,ไม่จริงอย่างยิ่ง,เฉยๆ,จริงอย่างยิ่ง
ภายใต้สถานการณ์ในปัจจุบัน ท่านคิดว่าข้อความเหล่านี้เป็นจริงมากน้อยเพียงใด [ระดับของค่าปรับในกฎหมาย อยู่ในระดับที่สามารถยับยั้ง การใช้อำนาจเหนือตลาดอย่างไม่เป็นธรรม (abuse of dominant position)],35.714286,28.571429,14.285714,14.285714,7.142857


In [93]:
TempValueCount = pd.DataFrame(Data['ภายใต้สถานการณ์ในปัจจุบัน ท่านคิดว่าข้อความเหล่านี้เป็นจริงมากน้อยเพียงใด  [ระดับของค่าปรับในกฎหมาย อยู่ในระดับที่สามารถยับยั้ง การรวมธุรกิจที่ผิดกฎหมายแข่งขันทางการค้า]'].value_counts()).transpose()

TempValueCount_Excluded = TempValueCount.loc[:, TempValueCount.columns!='ไม่สามารถตอบได้']

Fine_Level_Merger_Propotion = (TempValueCount_Excluded/TempValueCount_Excluded.sum(1)[0])*100

Fine_Level_Merger_Propotion

,จริง,ไม่จริง,ไม่จริงอย่างยิ่ง,จริงอย่างยิ่ง,เฉยๆ
ภายใต้สถานการณ์ในปัจจุบัน ท่านคิดว่าข้อความเหล่านี้เป็นจริงมากน้อยเพียงใด [ระดับของค่าปรับในกฎหมาย อยู่ในระดับที่สามารถยับยั้ง การรวมธุรกิจที่ผิดกฎหมายแข่งขันทางการค้า],37.037037,33.333333,18.518519,7.407407,3.703704


In [94]:
TempValueCount = pd.DataFrame(Data['ภายใต้สถานการณ์ในปัจจุบัน ท่านคิดว่าข้อความเหล่านี้เป็นจริงมากน้อยเพียงใด  [ระดับของค่าปรับในกฎหมาย อยู่ในระดับที่สามารถยับยั้ง การปฏิบัติการค้าที่ไม่เป็นธรรม]'].value_counts()).transpose()

TempValueCount_Excluded = TempValueCount.loc[:, TempValueCount.columns!='ไม่สามารถตอบได้']

Fine_Level_Unfair_Propotion = (TempValueCount_Excluded/TempValueCount_Excluded.sum(1)[0])*100

Fine_Level_Unfair_Propotion

,ไม่จริง,จริง,ไม่จริงอย่างยิ่ง,เฉยๆ,จริงอย่างยิ่ง
ภายใต้สถานการณ์ในปัจจุบัน ท่านคิดว่าข้อความเหล่านี้เป็นจริงมากน้อยเพียงใด [ระดับของค่าปรับในกฎหมาย อยู่ในระดับที่สามารถยับยั้ง การปฏิบัติการค้าที่ไม่เป็นธรรม],34.482759,34.482759,13.793103,10.344828,6.896552


In [95]:
TempValueCount = pd.DataFrame(Data['ภายใต้สถานการณ์ในปัจจุบัน ท่านคิดว่าข้อความเหล่านี้เป็นจริงมากน้อยเพียงใด  [ระดับของค่าปรับในกฎหมาย อยู่ในระดับที่สามารถยับยั้ง การกระทำผิดฐานตกลงร่วมกัน ผูกขาด ลด และจำกัดการแข่งขัน (cartel & bid-riggings)]'].value_counts()).transpose()

TempValueCount_Excluded = TempValueCount.loc[:, TempValueCount.columns!='ไม่สามารถตอบได้']

Fine_Level_Cartel_Propotion = (TempValueCount_Excluded/TempValueCount_Excluded.sum(1)[0])*100

Fine_Level_Cartel_Propotion

,จริง,ไม่จริง,ไม่จริงอย่างยิ่ง,จริงอย่างยิ่ง,เฉยๆ
ภายใต้สถานการณ์ในปัจจุบัน ท่านคิดว่าข้อความเหล่านี้เป็นจริงมากน้อยเพียงใด [ระดับของค่าปรับในกฎหมาย อยู่ในระดับที่สามารถยับยั้ง การกระทำผิดฐานตกลงร่วมกัน ผูกขาด ลด และจำกัดการแข่งขัน (cartel & bid-riggings)],39.285714,32.142857,14.285714,7.142857,7.142857


In [96]:
TempValueCount = pd.DataFrame(Data['ภายใต้สถานการณ์ในปัจจุบัน ท่านคิดว่าข้อความเหล่านี้เป็นจริงมากน้อยเพียงใด  [สำนักงานฯ ใช้เวลาในการสืบสวนสอบสวนและตัดสินคดีนานเกินกว่าที่ควร]'].value_counts()).transpose()

TempValueCount_Excluded = TempValueCount.loc[:, TempValueCount.columns!='ไม่สามารถตอบได้']

Investigation_Time_Propotion = (TempValueCount_Excluded/TempValueCount_Excluded.sum(1)[0])*100

Investigation_Time_Propotion

,จริง,จริงอย่างยิ่ง,เฉยๆ,ไม่จริงอย่างยิ่ง,ไม่จริง
ภายใต้สถานการณ์ในปัจจุบัน ท่านคิดว่าข้อความเหล่านี้เป็นจริงมากน้อยเพียงใด [สำนักงานฯ ใช้เวลาในการสืบสวนสอบสวนและตัดสินคดีนานเกินกว่าที่ควร],43.478261,26.086957,13.043478,8.695652,8.695652


In [97]:
Summary_Proportion_att2 = pd.concat([Independent_issue_Propotion,Conflict_issue_Propotion,Investigation_issue_Propotion,Human_Resource_issue_Propotion,Financial_Resource_issue_Propotion,Skill_Staff_issue_Propotion,Prob_Punishment_Propotion,Instrument_issue_Propotion,Punishment_Level_issue_Propotion,Fine_Level_Abuse_Propotion,Fine_Level_Merger_Propotion,Fine_Level_Unfair_Propotion,Fine_Level_Cartel_Propotion,Investigation_Time_Propotion])

Summary_Proportion_att2

,จริง,ไม่จริง,จริงอย่างยิ่ง,ไม่จริงอย่างยิ่ง,เฉยๆ
ภายใต้สถานการณ์ในปัจจุบัน ท่านคิดว่าข้อความเหล่านี้เป็นจริงมากน้อยเพียงใด [สำนักงานฯ ได้รับอิทธิพลจากผู้อื่น/หรืออยู่ภายใต้คำสั่งของหน่วยงานภายนอก เช่น หน่วยงานของรัฐ หน่วยงานเอกชน ผู้มีอำนาจทางการเมือง ฯลฯ],46.428571,25.000000,17.857143,7.142857,3.571429
ภายใต้สถานการณ์ในปัจจุบัน ท่านคิดว่าข้อความเหล่านี้เป็นจริงมากน้อยเพียงใด [คณะกรรมการแข่งขันฯ มีผลประโยชน์ทับซ้อน (conflict of interest) ส่งผลต่อการตัดสินใจ],33.333333,33.333333,23.809524,4.761905,4.761905
ภายใต้สถานการณ์ในปัจจุบัน ท่านคิดว่าข้อความเหล่านี้เป็นจริงมากน้อยเพียงใด [ในช่วงของการสืบสวน คณะกรรมการแข่งขันฯ มีอำนาจมากและความสามารถสูงในการสืบค้นและหาข้อมูลที่เกี่ยวข้อง (Investigation power)],42.307692,34.615385,NaN,11.538462,11.538462
ภายใต้สถานการณ์ในปัจจุบัน ท่านคิดว่าข้อความเหล่านี้เป็นจริงมากน้อยเพียงใด [สำนักงานฯ มีทรัพยากรบุคคลเพียงพอ ต่อการบังคับใช้กฎหมายอย่างมีประสิทธิภาพ เช่น ต้องดำเนินการตรวจสอบในหลายกรณีพร้อมกันในต่างสถานที่ ฯลฯ],33.333333,42.857143,NaN,14.285714,9.523810
ภายใต้สถานการณ์ในปัจจุบัน ท่านคิดว่าข้อความเหล่านี้เป็นจริงมากน้อยเพียงใด [สำนักงานฯ มีทรัพยากรด้านการเงินเพียงพอ ต่อการบังคับใช้กฎหมายอย่างมีประสิทธิภาพ],41.176471,35.294118,5.882353,11.764706,5.882353
ภายใต้สถานการณ์ในปัจจุบัน ท่านคิดว่าข้อความเหล่านี้เป็นจริงมากน้อยเพียงใด [เจ้าพนักงานมีทักษะและอำนาจทางกฎหมายที่เอื้อให้สามารถทำการตรวจสอบและสอบสวนได้อย่างมีประสิทธิภาพ],36.363636,50.000000,NaN,9.090909,4.545455
ภายใต้สถานการณ์ในปัจจุบัน ท่านคิดว่าข้อความเหล่านี้เป็นจริงมากน้อยเพียงใด [ผู้กระทำความผิดตามกฎหมายแข่งขันทางการค้ามีโอกาสโดนลงโทษสูง],46.666667,23.333333,6.666667,16.666667,6.666667
ภายใต้สถานการณ์ในปัจจุบัน ท่านคิดว่าข้อความเหล่านี้เป็นจริงมากน้อยเพียงใด [สำนักงานฯ มีอำนาจและเครื่องมือในเรื่องการกำหนดค่าปรับ/โทษที่มีประสิทธิภาพ],46.428571,25.000000,3.571429,7.142857,17.857143
ภายใต้สถานการณ์ในปัจจุบัน ท่านคิดว่าข้อความเหล่านี้เป็นจริงมากน้อยเพียงใด [ระดับของโทษ/ค่าปรับในภาพรวมของกฎหมาย อยู่ในระดับที่สามารถยับยั้งการกระทำที่ขัดต่อกฎหมาย],42.307692,26.923077,3.846154,15.384615,11.538462
ภายใต้สถานการณ์ในปัจจุบัน ท่านคิดว่าข้อความเหล่านี้เป็นจริงมากน้อยเพียงใด [ระดับของค่าปรับในกฎหมาย อยู่ในระดับที่สามารถยับยั้ง การใช้อำนาจเหนือตลาดอย่างไม่เป็นธรรม (abuse of dominant position)],35.714286,28.571429,7.142857,14.285714,14.285714


## โทษที่มีผลต่อ deterrence 

In [98]:
Data.columns

Index(['ประทับเวลา',
       'หลังจากท่านได้ทราบถึงภาพรวมและวัตถุประสงค์ของแบบสอบถามฉบับนี้แล้ว ท่านยินดีตอบแบบสอบถามฉบับนี้หรือไม่',
       'ประเภทอุตสาหกรรมหลักของสถานประกอบการ ',
       'กิจการของท่านอยู่ภายใต้การกำกับของหน่วยงานใดบ้าง (สามารถตอบได้มากกว่าหนึ่งข้อ)',
       'รายได้ต่อปีของสถานประกอบการ ในปี พ.ศ. 2562 (ก่อนวิกฤตโควิด)',
       'จำนวนลูกจ้างในสถานประกอบการ',
       'ในกรณีที่จำเป็น ท่านยินดีให้ผู้วิจัย (สถาบันวิจัยเพื่อการพัฒนาประเทศไทย) ติดต่อท่านอีกครั้งเพื่อรวบรวมข้อมูลเพิ่มเติมจากข้อมูลที่ได้ท่านให้ไว้หรือไม่ ',
       'เบอร์โทรศัพท์',
       'สิ่งใดเป็นความผิดตามพระราชบัญญัติการแข่งขันทางการค้า พ.ศ. 2560',
       'ลักษณะใดเป็นเกณฑ์ของการเป็นผู้มีอำนาจเหนือตลาดตามพระราชบัญญัติการแข่งขันทางการค้า พ.ศ. 2560',
       'ค่าปรับสำหรับการกระทำผิดตามพระราชบัญญัติการแข่งขันทางการค้า พ.ศ. 2560 ขึ้นกับปัจจัยอะไรบ้าง',
       'การกระทำผิดในข้อใดหากฝ่าฝืนแล้วมีโทษจำคุก ',
       'การรวมธุรกิจลักษณะใดต้องได้รับอนุญาตจากคณะกรรมการการแข่งขันทางการค้า',
       'ข้อใดคือที่อยู่ของสำ

In [99]:
TempValueCount = pd.DataFrame(Data['ตามความเห็นของท่าน ท่านคิดว่าการลงโทษหรือเครื่องมือแต่ละประเภทดังตารางด้านล่าง ทั้งประเภทที่มีอยู่ในกฎหมายปัจจุบันและประเภทที่อาจถูกเพิ่มขึ้นมาในอนาคต มีความสำคัญต่อการยับยั้งการกระทำที่ขัดต่อกฎหมายการแข่งขันทางการค้าในประเทศไทยมากน้อยเพียงใด  [การโดนฟ้องร้องเรียกค่าเสียหายจากผู้ประกอบการที่ได้รับผลกระทบจากการกระทำผิด (private damages actions)]'].value_counts()).transpose()

TempValueCount_Excluded = TempValueCount.loc[:, TempValueCount.columns!='ไม่ทราบ']

Private_Damage_Action_Propotion = (TempValueCount_Excluded/TempValueCount_Excluded.sum(1)[0])*100

Private_Damage_Action_Propotion

,มีผลมาก,มีผลเล็กน้อย,มีผลปานกลาง,ไม่มีผลเลย
ตามความเห็นของท่าน ท่านคิดว่าการลงโทษหรือเครื่องมือแต่ละประเภทดังตารางด้านล่าง ทั้งประเภทที่มีอยู่ในกฎหมายปัจจุบันและประเภทที่อาจถูกเพิ่มขึ้นมาในอนาคต มีความสำคัญต่อการยับยั้งการกระทำที่ขัดต่อกฎหมายการแข่งขันทางการค้าในประเทศไทยมากน้อยเพียงใด [การโดนฟ้องร้องเรียกค่าเสียหายจากผู้ประกอบการที่ได้รับผลกระทบจากการกระทำผิด (private damages actions)],58.823529,23.529412,8.823529,8.823529


In [100]:
TempValueCount = pd.DataFrame(Data['ตามความเห็นของท่าน ท่านคิดว่าการลงโทษหรือเครื่องมือแต่ละประเภทดังตารางด้านล่าง ทั้งประเภทที่มีอยู่ในกฎหมายปัจจุบันและประเภทที่อาจถูกเพิ่มขึ้นมาในอนาคต มีความสำคัญต่อการยับยั้งการกระทำที่ขัดต่อกฎหมายการแข่งขันทางการค้าในประเทศไทยมากน้อยเพียงใด  [โทษปรับบริษัทที่กระทำผิด (fines on firm)]'].value_counts()).transpose()

TempValueCount_Excluded = TempValueCount.loc[:, TempValueCount.columns!='ไม่ทราบ']

Fines_Firm_Propotion = (TempValueCount_Excluded/TempValueCount_Excluded.sum(1)[0])*100

Fines_Firm_Propotion

,มีผลมาก,มีผลปานกลาง,มีผลเล็กน้อย,ไม่มีผลเลย
ตามความเห็นของท่าน ท่านคิดว่าการลงโทษหรือเครื่องมือแต่ละประเภทดังตารางด้านล่าง ทั้งประเภทที่มีอยู่ในกฎหมายปัจจุบันและประเภทที่อาจถูกเพิ่มขึ้นมาในอนาคต มีความสำคัญต่อการยับยั้งการกระทำที่ขัดต่อกฎหมายการแข่งขันทางการค้าในประเทศไทยมากน้อยเพียงใด [โทษปรับบริษัทที่กระทำผิด (fines on firm)],45.454545,27.272727,15.151515,12.121212


In [101]:
TempValueCount = pd.DataFrame(Data['ตามความเห็นของท่าน ท่านคิดว่าการลงโทษหรือเครื่องมือแต่ละประเภทดังตารางด้านล่าง ทั้งประเภทที่มีอยู่ในกฎหมายปัจจุบันและประเภทที่อาจถูกเพิ่มขึ้นมาในอนาคต มีความสำคัญต่อการยับยั้งการกระทำที่ขัดต่อกฎหมายการแข่งขันทางการค้าในประเทศไทยมากน้อยเพียงใด  [โทษปรับตัวบุคคลที่กระทำผิด (fines on individual)]'].value_counts()).transpose()

TempValueCount_Excluded = TempValueCount.loc[:, TempValueCount.columns!='ไม่ทราบ']

Fines_Individual_Propotion = (TempValueCount_Excluded/TempValueCount_Excluded.sum(1)[0])*100

Fines_Individual_Propotion

,มีผลมาก,มีผลปานกลาง,มีผลเล็กน้อย,ไม่มีผลเลย
ตามความเห็นของท่าน ท่านคิดว่าการลงโทษหรือเครื่องมือแต่ละประเภทดังตารางด้านล่าง ทั้งประเภทที่มีอยู่ในกฎหมายปัจจุบันและประเภทที่อาจถูกเพิ่มขึ้นมาในอนาคต มีความสำคัญต่อการยับยั้งการกระทำที่ขัดต่อกฎหมายการแข่งขันทางการค้าในประเทศไทยมากน้อยเพียงใด [โทษปรับตัวบุคคลที่กระทำผิด (fines on individual)],48.484848,21.212121,18.181818,12.121212


In [102]:
TempValueCount = pd.DataFrame(Data['ตามความเห็นของท่าน ท่านคิดว่าการลงโทษหรือเครื่องมือแต่ละประเภทดังตารางด้านล่าง ทั้งประเภทที่มีอยู่ในกฎหมายปัจจุบันและประเภทที่อาจถูกเพิ่มขึ้นมาในอนาคต มีความสำคัญต่อการยับยั้งการกระทำที่ขัดต่อกฎหมายการแข่งขันทางการค้าในประเทศไทยมากน้อยเพียงใด  [โทษจำคุกบุคคลที่กระทำผิด (imprisonment)]'].value_counts()).transpose()

TempValueCount_Excluded = TempValueCount.loc[:, TempValueCount.columns!='ไม่ทราบ']

Imprisonment_Propotion = (TempValueCount_Excluded/TempValueCount_Excluded.sum(1)[0])*100

Imprisonment_Propotion

,มีผลมาก,มีผลเล็กน้อย,มีผลปานกลาง,ไม่มีผลเลย
ตามความเห็นของท่าน ท่านคิดว่าการลงโทษหรือเครื่องมือแต่ละประเภทดังตารางด้านล่าง ทั้งประเภทที่มีอยู่ในกฎหมายปัจจุบันและประเภทที่อาจถูกเพิ่มขึ้นมาในอนาคต มีความสำคัญต่อการยับยั้งการกระทำที่ขัดต่อกฎหมายการแข่งขันทางการค้าในประเทศไทยมากน้อยเพียงใด [โทษจำคุกบุคคลที่กระทำผิด (imprisonment)],50.0,20.588235,14.705882,14.705882


In [103]:
TempValueCount = pd.DataFrame(Data['ตามความเห็นของท่าน ท่านคิดว่าการลงโทษหรือเครื่องมือแต่ละประเภทดังตารางด้านล่าง ทั้งประเภทที่มีอยู่ในกฎหมายปัจจุบันและประเภทที่อาจถูกเพิ่มขึ้นมาในอนาคต มีความสำคัญต่อการยับยั้งการกระทำที่ขัดต่อกฎหมายการแข่งขันทางการค้าในประเทศไทยมากน้อยเพียงใด  [ความเสียหายต่อชื่อเสียงของบริษัท (reputational damage for the company)**]'].value_counts()).transpose()

TempValueCount_Excluded = TempValueCount.loc[:, TempValueCount.columns!='ไม่ทราบ']

Reputation_Company_Propotion = (TempValueCount_Excluded/TempValueCount_Excluded.sum(1)[0])*100

Reputation_Company_Propotion

,มีผลมาก,มีผลปานกลาง,มีผลเล็กน้อย,ไม่มีผลเลย
ตามความเห็นของท่าน ท่านคิดว่าการลงโทษหรือเครื่องมือแต่ละประเภทดังตารางด้านล่าง ทั้งประเภทที่มีอยู่ในกฎหมายปัจจุบันและประเภทที่อาจถูกเพิ่มขึ้นมาในอนาคต มีความสำคัญต่อการยับยั้งการกระทำที่ขัดต่อกฎหมายการแข่งขันทางการค้าในประเทศไทยมากน้อยเพียงใด [ความเสียหายต่อชื่อเสียงของบริษัท (reputational damage for the company)**],50.0,29.411765,17.647059,2.941176


In [104]:
TempValueCount = pd.DataFrame(Data['ตามความเห็นของท่าน ท่านคิดว่าการลงโทษหรือเครื่องมือแต่ละประเภทดังตารางด้านล่าง ทั้งประเภทที่มีอยู่ในกฎหมายปัจจุบันและประเภทที่อาจถูกเพิ่มขึ้นมาในอนาคต มีความสำคัญต่อการยับยั้งการกระทำที่ขัดต่อกฎหมายการแข่งขันทางการค้าในประเทศไทยมากน้อยเพียงใด  [ความเสียหายต่อชื่อเสียงของบุคคลที่กระทำผิด (reputational damage for the individual)**]'].value_counts()).transpose()

TempValueCount_Excluded = TempValueCount.loc[:, TempValueCount.columns!='ไม่ทราบ']

Reputation_Individual_Propotion = (TempValueCount_Excluded/TempValueCount_Excluded.sum(1)[0])*100

Reputation_Individual_Propotion

,มีผลมาก,มีผลเล็กน้อย,มีผลปานกลาง,ไม่มีผลเลย
ตามความเห็นของท่าน ท่านคิดว่าการลงโทษหรือเครื่องมือแต่ละประเภทดังตารางด้านล่าง ทั้งประเภทที่มีอยู่ในกฎหมายปัจจุบันและประเภทที่อาจถูกเพิ่มขึ้นมาในอนาคต มีความสำคัญต่อการยับยั้งการกระทำที่ขัดต่อกฎหมายการแข่งขันทางการค้าในประเทศไทยมากน้อยเพียงใด [ความเสียหายต่อชื่อเสียงของบุคคลที่กระทำผิด (reputational damage for the individual)**],52.941176,23.529412,20.588235,2.941176


In [105]:
TempValueCount = pd.DataFrame(Data['ตามความเห็นของท่าน ท่านคิดว่าการลงโทษหรือเครื่องมือแต่ละประเภทดังตารางด้านล่าง ทั้งประเภทที่มีอยู่ในกฎหมายปัจจุบันและประเภทที่อาจถูกเพิ่มขึ้นมาในอนาคต มีความสำคัญต่อการยับยั้งการกระทำที่ขัดต่อกฎหมายการแข่งขันทางการค้าในประเทศไทยมากน้อยเพียงใด  [คำสั่งปลดกรรมการบริษัท (director disqualification orders)**]'].value_counts()).transpose()

TempValueCount_Excluded = TempValueCount.loc[:, TempValueCount.columns!='ไม่ทราบ']

Disqualification_Propotion = (TempValueCount_Excluded/TempValueCount_Excluded.sum(1)[0])*100

Disqualification_Propotion

,มีผลมาก,มีผลปานกลาง,มีผลเล็กน้อย,ไม่มีผลเลย
ตามความเห็นของท่าน ท่านคิดว่าการลงโทษหรือเครื่องมือแต่ละประเภทดังตารางด้านล่าง ทั้งประเภทที่มีอยู่ในกฎหมายปัจจุบันและประเภทที่อาจถูกเพิ่มขึ้นมาในอนาคต มีความสำคัญต่อการยับยั้งการกระทำที่ขัดต่อกฎหมายการแข่งขันทางการค้าในประเทศไทยมากน้อยเพียงใด [คำสั่งปลดกรรมการบริษัท (director disqualification orders)**],51.515152,21.212121,18.181818,9.090909


In [106]:
TempValueCount = pd.DataFrame(Data['ตามความเห็นของท่าน ท่านคิดว่าการลงโทษหรือเครื่องมือแต่ละประเภทดังตารางด้านล่าง ทั้งประเภทที่มีอยู่ในกฎหมายปัจจุบันและประเภทที่อาจถูกเพิ่มขึ้นมาในอนาคต มีความสำคัญต่อการยับยั้งการกระทำที่ขัดต่อกฎหมายการแข่งขันทางการค้าในประเทศไทยมากน้อยเพียงใด  [การลดหย่อนผ่อน/ยกเว้นโทษสำหรับผู้กระทำผิดที่เป็นพยาน (leniency program)** [หากผู้ร่วมกระทำผิดในกรณีการตกลงร่วมกันผูกขาดได้มาแจ้งให้องค์กรที่กำกับดูแลการแข่งขันทราบถึงการกระทำผิด จะได้รับการยกเว้นหรือลดหย่อนโทษ]]'].value_counts()).transpose()

TempValueCount_Excluded = TempValueCount.loc[:, TempValueCount.columns!='ไม่ทราบ']

Leniency_Propotion = (TempValueCount_Excluded/TempValueCount_Excluded.sum(1)[0])*100

Leniency_Propotion

,มีผลมาก,มีผลปานกลาง,มีผลเล็กน้อย,ไม่มีผลเลย
ตามความเห็นของท่าน ท่านคิดว่าการลงโทษหรือเครื่องมือแต่ละประเภทดังตารางด้านล่าง ทั้งประเภทที่มีอยู่ในกฎหมายปัจจุบันและประเภทที่อาจถูกเพิ่มขึ้นมาในอนาคต มีความสำคัญต่อการยับยั้งการกระทำที่ขัดต่อกฎหมายการแข่งขันทางการค้าในประเทศไทยมากน้อยเพียงใด [การลดหย่อนผ่อน/ยกเว้นโทษสำหรับผู้กระทำผิดที่เป็นพยาน (leniency program)** [หากผู้ร่วมกระทำผิดในกรณีการตกลงร่วมกันผูกขาดได้มาแจ้งให้องค์กรที่กำกับดูแลการแข่งขันทราบถึงการกระทำผิด จะได้รับการยกเว้นหรือลดหย่อนโทษ]],44.117647,38.235294,11.764706,5.882353


In [107]:
Summary_Proportion_att3 = pd.concat([Private_Damage_Action_Propotion,Fines_Firm_Propotion,Fines_Individual_Propotion,Imprisonment_Propotion,Reputation_Company_Propotion,Reputation_Individual_Propotion,Disqualification_Propotion,Leniency_Propotion])


Summary_Proportion_att3

,มีผลมาก,มีผลเล็กน้อย,มีผลปานกลาง,ไม่มีผลเลย
ตามความเห็นของท่าน ท่านคิดว่าการลงโทษหรือเครื่องมือแต่ละประเภทดังตารางด้านล่าง ทั้งประเภทที่มีอยู่ในกฎหมายปัจจุบันและประเภทที่อาจถูกเพิ่มขึ้นมาในอนาคต มีความสำคัญต่อการยับยั้งการกระทำที่ขัดต่อกฎหมายการแข่งขันทางการค้าในประเทศไทยมากน้อยเพียงใด [การโดนฟ้องร้องเรียกค่าเสียหายจากผู้ประกอบการที่ได้รับผลกระทบจากการกระทำผิด (private damages actions)],58.823529,23.529412,8.823529,8.823529
ตามความเห็นของท่าน ท่านคิดว่าการลงโทษหรือเครื่องมือแต่ละประเภทดังตารางด้านล่าง ทั้งประเภทที่มีอยู่ในกฎหมายปัจจุบันและประเภทที่อาจถูกเพิ่มขึ้นมาในอนาคต มีความสำคัญต่อการยับยั้งการกระทำที่ขัดต่อกฎหมายการแข่งขันทางการค้าในประเทศไทยมากน้อยเพียงใด [โทษปรับบริษัทที่กระทำผิด (fines on firm)],45.454545,15.151515,27.272727,12.121212
ตามความเห็นของท่าน ท่านคิดว่าการลงโทษหรือเครื่องมือแต่ละประเภทดังตารางด้านล่าง ทั้งประเภทที่มีอยู่ในกฎหมายปัจจุบันและประเภทที่อาจถูกเพิ่มขึ้นมาในอนาคต มีความสำคัญต่อการยับยั้งการกระทำที่ขัดต่อกฎหมายการแข่งขันทางการค้าในประเทศไทยมากน้อยเพียงใด [โทษปรับตัวบุคคลที่กระทำผิด (fines on individual)],48.484848,18.181818,21.212121,12.121212
ตามความเห็นของท่าน ท่านคิดว่าการลงโทษหรือเครื่องมือแต่ละประเภทดังตารางด้านล่าง ทั้งประเภทที่มีอยู่ในกฎหมายปัจจุบันและประเภทที่อาจถูกเพิ่มขึ้นมาในอนาคต มีความสำคัญต่อการยับยั้งการกระทำที่ขัดต่อกฎหมายการแข่งขันทางการค้าในประเทศไทยมากน้อยเพียงใด [โทษจำคุกบุคคลที่กระทำผิด (imprisonment)],50.000000,20.588235,14.705882,14.705882
ตามความเห็นของท่าน ท่านคิดว่าการลงโทษหรือเครื่องมือแต่ละประเภทดังตารางด้านล่าง ทั้งประเภทที่มีอยู่ในกฎหมายปัจจุบันและประเภทที่อาจถูกเพิ่มขึ้นมาในอนาคต มีความสำคัญต่อการยับยั้งการกระทำที่ขัดต่อกฎหมายการแข่งขันทางการค้าในประเทศไทยมากน้อยเพียงใด [ความเสียหายต่อชื่อเสียงของบริษัท (reputational damage for the company)**],50.000000,17.647059,29.411765,2.941176
ตามความเห็นของท่าน ท่านคิดว่าการลงโทษหรือเครื่องมือแต่ละประเภทดังตารางด้านล่าง ทั้งประเภทที่มีอยู่ในกฎหมายปัจจุบันและประเภทที่อาจถูกเพิ่มขึ้นมาในอนาคต มีความสำคัญต่อการยับยั้งการกระทำที่ขัดต่อกฎหมายการแข่งขันทางการค้าในประเทศไทยมากน้อยเพียงใด [ความเสียหายต่อชื่อเสียงของบุคคลที่กระทำผิด (reputational damage for the individual)**],52.941176,23.529412,20.588235,2.941176
ตามความเห็นของท่าน ท่านคิดว่าการลงโทษหรือเครื่องมือแต่ละประเภทดังตารางด้านล่าง ทั้งประเภทที่มีอยู่ในกฎหมายปัจจุบันและประเภทที่อาจถูกเพิ่มขึ้นมาในอนาคต มีความสำคัญต่อการยับยั้งการกระทำที่ขัดต่อกฎหมายการแข่งขันทางการค้าในประเทศไทยมากน้อยเพียงใด [คำสั่งปลดกรรมการบริษัท (director disqualification orders)**],51.515152,18.181818,21.212121,9.090909
ตามความเห็นของท่าน ท่านคิดว่าการลงโทษหรือเครื่องมือแต่ละประเภทดังตารางด้านล่าง ทั้งประเภทที่มีอยู่ในกฎหมายปัจจุบันและประเภทที่อาจถูกเพิ่มขึ้นมาในอนาคต มีความสำคัญต่อการยับยั้งการกระทำที่ขัดต่อกฎหมายการแข่งขันทางการค้าในประเทศไทยมากน้อยเพียงใด [การลดหย่อนผ่อน/ยกเว้นโทษสำหรับผู้กระทำผิดที่เป็นพยาน (leniency program)** [หากผู้ร่วมกระทำผิดในกรณีการตกลงร่วมกันผูกขาดได้มาแจ้งให้องค์กรที่กำกับดูแลการแข่งขันทราบถึงการกระทำผิด จะได้รับการยกเว้นหรือลดหย่อนโทษ]],44.117647,11.764706,38.235294,5.882353


## Cross-Tab

In [108]:
dict_S4P1 = {'มีจำนวนมาก':'มาก',
 'มีจำนวนค่อนข้างมาก':'ค่อนข้างมาก',
 'มีจำนวนปานกลาง':'ปานกลาง',
 'มีจำนวนค่อนข้างน้อย':'ค่อนข้างน้อย',
 'มีจำนวนน้อย':'น้อย',
 'ทราบว่าไม่มีเลย':'ไม่มีเลย',
 'ท่านไม่เคยได้รับข้อมูลดังกล่าว':'ไม่เคยได้รับข้อมูลดังกล่าว'}

Questions = ['1.1 จากข้อมูลที่ท่านทราบ ผู้ประกอบการในอุตสาหกรรมเดียวกับท่าน (อุตสาหกรรมที่ท่านเลือกในตอนต้น) ได้มีการวางแผนรวมธุรกิจหรือได้ทำการรวมธุรกิจ ที่ท่านมองว่าไม่น่าได้รับการอนุญาตหรือมีความเป็นไปได้สูงที่จะถูกตั้งเงื่อนไขการรวมธุรกิจที่เข้มงวด ในช่วงหลังจากได้มีการบังคับใช้ พ.ร.บ. การแข่งขันทางการค้า พ.ศ. 2560 (ตั้งแต่กรกฎาคม 2560 จนถึงปัจจุบัน) จำนวนมากน้อยเพียงใด',
'2.1 จากข้อมูลที่ท่านทราบ ผู้ประกอบการในอุตสาหกรรมเดียวกับท่าน (อุตสาหกรรมที่ท่านเลือกในตอนต้น) ได้มีกิจกรรมทางธุรกิจหรือได้วางแผนที่จะทำกิจกรรมทางธุรกิจที่ท่านมองว่าอาจเข้าข่ายเป็นการกระทำผิดฐานใช้อำนาจเหนือตลาดอย่างไม่เป็นธรรม (abuse of dominant position) ในช่วงหลังจากได้มีการบังคับใช้ พ.ร.บ. การแข่งขันทางการค้า พ.ศ. 2560 (ตั้งแต่กรกฎาคม 2560 จนถึงปัจจุบัน) จำนวนมากน้อยเพียงใด ',
'3.1 จากข้อมูลที่ท่านทราบ ผู้ประกอบการในอุตสาหกรรมเดียวกับท่าน (อุตสาหกรรมที่ท่านเลือกในตอนต้น) ได้มีความร่วมมือหรือได้วางแผนที่จะมีความร่วมมือกับผู้ประกอบการรายอื่นที่ท่านมองว่าอาจเข้าข่ายเป็นการกระทำผิดฐานตกลงร่วมกัน ผูกขาด ลด และจำกัดการแข่งขัน (cartel & bid-riggings) ในช่วงหลังจากได้มีการบังคับใช้ พ.ร.บ. การแข่งขันทางการค้า พ.ศ. 2560 (ตั้งแต่กรกฎาคม 2560 จนถึงปัจจุบัน) จำนวนมากน้อยเพียงใด',
'4.1. จากข้อมูลที่ท่านทราบ ผู้ประกอบการในอุตสาหกรรมเดียวกับท่าน (อุตสาหกรรมที่ท่านเลือกในตอนต้น) ได้มีกิจกรรมทางธุรกิจหรือได้วางแผนที่จะทำกิจกรรมทางธุรกิจที่ท่านมองว่าอาจเข้าข่ายเป็นการกระทำผิดฐานปฏิบัติทางการค้าที่ไม่เป็นธรรม (unfair trade practices) ในช่วงหลังจากได้มีการบังคับใช้ พ.ร.บ. การแข่งขันทางการค้า พ.ศ. 2560 (ตั้งแต่กรกฎาคม 2560 จนถึงปัจจุบัน) จำนวนมากน้อยเพียงใด']
Simplify = ['Mergers','Abuse of Dominance','Cartels','Unfair Trade Practices']

Var='Q_category' #CT กับตัวแปรนี้

Graph_CT = [None]*len(Questions)
i = 0
for x,k in zip(Questions, Simplify):
    Graph_CT[i] = Data.groupby([Var,x])['ประทับเวลา'].count().to_frame().reset_index()
    Graph_CT[i].rename(columns = {x:k} , inplace = True)
    Graph_CT[i][k].replace(dict_S4P1, inplace=True)
    Graph_CT[i] = Graph_CT[i].set_index([Var,k]).unstack()
    Graph_CT[i].columns =Graph_CT[i].columns.droplevel(0)
    Graph_CT[i]['Type']=Graph_CT[i].columns.name
    Graph_CT[i] = Graph_CT[i].reset_index().set_index(['Type','Q_category'])
    Graph_CT[i].columns.name=None
    Graph_CT[i]
    i+=1

Graph_CT_data = pd.concat([Graph_CT[0],Graph_CT[1],Graph_CT[2],Graph_CT[3]])
Graph_CT_data = Graph_CT_data[['มาก','ค่อนข้างมาก','ปานกลาง','ค่อนข้างน้อย','น้อย','ไม่มีเลย','ไม่เคยได้รับข้อมูลดังกล่าว']]
Graph_CT_data = Graph_CT_data.replace(np.nan,0)
Graph_CT_data

มาก  ค่อนข้างมาก  ปานกลาง  ค่อนข้างน้อย  \
Type                   Q_category                                            
Mergers                high        0.0          2.0      0.0           2.0   
                       low         1.0          3.0      4.0           3.0   
Abuse of Dominance     high        0.0          0.0      2.0           0.0   
                       low         2.0          1.0      4.0           0.0   
Cartels                high        0.0          0.0      0.0           0.0   
                       low         1.0          3.0      1.0           1.0   
Unfair Trade Practices high        0.0          1.0      1.0           1.0   
                       low         2.0          1.0      2.0           1.0   

                                   น้อย  ไม่มีเลย  ไม่เคยได้รับข้อมูลดังกล่าว  
Type                   Q_category                                              
Mergers                high         2.0       2.0                         2.0  
                       low          4.0       0.0                        10.0  
Abuse of Dominance     high         0.0       4.0                         4.0  
                       low          5.0       0.0                        12.0  
Cartels                high         2.0       4.0                         4.0  
                       low          4.0       1.0                        14.0  
Unfair Trade Practices high         0.0       4.0                         3.0  
                       low          3.0       3.0                        13.0

In [109]:
#deterrance

dict_S4P3 = {'ร้อยละ 0-20':'20-0 %',
 'ร้อยละ 21-40':'40-21 %',
 'ร้อยละ 41-60':'60-41 %',
 'ร้อยละ 61-80':'80-61 %',
 'ร้อยละ 81-100':'100-81 %',
 'ไม่ทราบ/ไม่แน่ใจ':'ไม่ทราบ/ไม่แน่ใจ'}

Questions = ['1.2 ในจำนวนแผนการรวมธุรกิจและการรวมธุรกิจทั้งหมดที่ท่านได้ระบุในข้อ 1.1 มีจำนวนแผนการรวมธุรกิจและการรวมธุรกิจที่ถูกยกเลิกหรือปรับเปลี่ยนไปแล้ว เป็นสัดส่วนมากน้อยเพียงใด (ประมาณการ)',
             '2.2 จากข้อมูลที่ท่านทราบ ในจำนวนกิจกรรมหรือแผนกิจกรรมที่อาจเข้าข่ายเป็นการกระทำผิดฐานใช้อำนาจเหนือตลาดอย่างไม่เป็นธรรม ทั้งหมดที่ท่านระบุในข้อ 2.1 มีกิจกรรมหรือแผนกิจกรรมที่ถูกยกเลิกหรือปรับเปลี่ยนไปแล้ว เป็นสัดส่วนมากน้อยเพียงใด (ประมาณการ)',
             '3.2 จากข้อมูลที่ท่านทราบ ในจำนวนความร่วมมือหรือแผนความร่วมมือกับผู้ประกอบการรายอื่นที่อาจเข้าข่ายเป็นการกระทำผิดฐานตกลงร่วมกัน ผูกขาด ลด และจำกัดการแข่งขัน ทั้งหมดที่ท่านระบุในข้อ 3.1 มีความร่วมมือหรือแผนความร่วมมือที่ถูกยกเลิกหรือปรับเปลี่ยนไปแล้ว เป็นสัดส่วนมากน้อยเพียงใด  (ประมาณการ)',
             '4.2 จากข้อมูลที่ท่านทราบ ในจำนวนกิจกรรมหรือแผนกิจกรรมที่อาจเข้าข่ายเป็นการกระทำผิดฐานปฏิบัติทางการค้าที่ไม่เป็นธรรม ทั้งหมดที่ท่านระบุในข้อ 4.4.1 มีกิจกรรมหรือแผนกิจกรรมที่ถูกยกเลิกหรือปรับเปลี่ยนไปแล้ว เป็นสัดส่วนมากน้อยเพียงใด (ประมาณการ)']
Simplify = ['Merger','Abuse of Dominance','Cartels','Unfair Trade Practices']
Answer_S4P3 = ['100-81 %', '80-61 %', '60-41 %', '40-21 %', '20-0 %','ไม่ทราบ/ไม่แน่ใจ']

Graph_CT = [None]*len(Questions)
i = 0
for x,k in zip(Questions, Simplify):
    Graph_CT[i] = Data.groupby(['Q_category',x])['ประทับเวลา'].count().to_frame().reset_index()
    Graph_CT[i].rename(columns = {x:k} , inplace = True)
    Graph_CT[i][k].replace(dict_S4P3, inplace=True)
    Graph_CT[i] = Graph_CT[i].set_index(['Q_category',k]).unstack()
    Graph_CT[i].columns =Graph_CT[i].columns.droplevel(0)
    Graph_CT[i]['Deterrance']=Graph_CT[i].columns.name
    Graph_CT[i] = Graph_CT[i].reset_index().set_index(['Deterrance','Q_category'])
    Graph_CT[i].columns.name=None
    for col in Answer_S4P3:
        if col not in Graph_CT[i].columns:
            Graph_CT[i][col]=0
    i+=1

Graph_CT_data_2 = pd.concat([Graph_CT[0],Graph_CT[1],Graph_CT[2],Graph_CT[3]])
Graph_CT_data_2 = Graph_CT_data_2[Answer_S4P3]
Graph_CT_data_2

100-81 %  80-61 %  60-41 %  40-21 %  \
Deterrance             Q_category                                        
Merger                 high             0.0      NaN      NaN      NaN   
                       low              0.0      2.0      1.0      3.0   
Abuse of Dominance     high             0.0      NaN      NaN      NaN   
                       low              0.0      1.0      1.0      1.0   
Cartels                high             NaN      NaN      0.0      NaN   
                       low              1.0      1.0      0.0      1.0   
Unfair Trade Practices high             NaN      0.0      0.0      1.0   
                       low              1.0      0.0      0.0      2.0   

                                   20-0 %  ไม่ทราบ/ไม่แน่ใจ  
Deterrance             Q_category                            
Merger                 high           3.0               7.0  
                       low            4.0              15.0  
Abuse of Dominance     high           3.0               7.0  
                       low            6.0              16.0  
Cartels                high           3.0               7.0  
                       low            5.0              17.0  
Unfair Trade Practices high           3.0               6.0  
                       low            6.0              16.0

In [113]:
#deterrance

dict_S4P3 = {'ร้อยละ 0-20':'20-0 %',
 'ร้อยละ 21-40':'40-21 %',
 'ร้อยละ 41-60':'60-41 %',
 'ร้อยละ 61-80':'80-61 %',
 'ร้อยละ 81-100':'100-81 %',
 'ไม่ทราบ/ไม่แน่ใจ':'ไม่ทราบ/ไม่แน่ใจ'}

Questions = ['1.3 ในจำนวนแผนการรวมธุรกิจและการรวมธุรกิจทั้งหมดที่ท่านได้ระบุในข้อ 1.1. มีแผนการรวมธุรกิจและการรวมธุรกิจที่ถูกยกเลิกหรือปรับเปลี่ยนไปแล้ว เนื่องจากเหตุผลในตารางต่อไปนี้ เป็นสัดส่วนมากน้อยเพียงใด (ประมาณการ)  [ผู้ประกอบการเกรงว่าจะไม่ได้รับอนุญาตหรือจะได้รับการตั้งเงื่อนไขการรวมธุรกิจที่เข้มงวดจากคณะกรรมการการแข่งขันทางการค้า]',
             '2.3 จากข้อมูลที่ท่านทราบ ในจำนวนกิจกรรมหรือแผนกิจกรรมที่อาจเข้าข่ายเป็นการกระทำผิดฐานใช้อำนาจเหนือตลาดอย่างไม่เป็นธรรม ทั้งหมดที่ท่านระบุในข้อ 2.1 มีกิจกรรมหรือแผนกิจกรรมที่ถูกยกเลิกหรือปรับเปลี่ยนไปแล้ว เนื่องจากเหตุผลในตารางต่อไปนี้ เป็นสัดส่วนมากน้อยเพียงใด (ประมาณการ) [ผู้ประกอบการต้องการป้องกันความเสี่ยงที่อาจจะถูกสอบสวนสืบสวนโดย สำนักงาน กขค. และ/หรือถูกลงโทษตาม พ.ร.บ การแข่งขันทางการค้า]',
             '3.3 จากข้อมูลที่ท่านทราบ ในจำนวนความร่วมมือหรือแผนความร่วมมือกับผู้ประกอบการรายอื่นที่อาจเข้าข่ายเป็นการกระทำผิดฐานตกลงร่วมกัน ผูกขาด ลด และจำกัดการแข่งขัน ทั้งหมดที่ท่านระบุในข้อ 3.1 มีความร่วมมือหรือแผนความร่วมมือที่ถูกยกเลิกหรือปรับเปลี่ยนไปแล้ว เนื่องจากเหตุผลในตารางต่อไปนี้ เป็นสัดส่วนมากน้อยเพียงใด (ประมาณการ) [ผู้ประกอบการต้องการป้องกันความเสี่ยงที่อาจจะถูกสอบสวนสืบสวนโดย สำนักงาน กขค. และ/หรือถูกลงโทษตาม พ.ร.บ การแข่งขันทางการค้า]',
             '4.3 จากข้อมูลที่ท่านทราบ ในจำนวนกิจกรรมหรือแผนกิจกรรมที่อาจเข้าข่ายเป็นการกระทำผิดฐานปฏิบัติทางการค้าที่ไม่เป็นธรรม ทั้งหมดที่ท่านระบุในข้อ 4.1 มีกิจกรรมหรือแผนกิจกรรมที่ถูกล้มเลิกหรือปรับเปลี่ยนไปแล้ว เนื่องจากเหตุผลในตารางต่อไปนี้ เป็นสัดส่วนมากน้อยเพียงใด (ประมาณการ) [ผู้ประกอบการต้องการป้องกันความเสี่ยงที่อาจจะถูกสอบสวนสืบสวนโดย สำนักงาน กขค. และ/หรือถูกลงโทษตาม พ.ร.บ การแข่งขันทางการค้า]']
Simplify = ['Merger','Abuse of Dominance','Cartels','Unfair Trade Practices']
Answer_S4P3 = ['100-81 %', '80-61 %', '60-41 %', '40-21 %', '20-0 %','ไม่ทราบ/ไม่แน่ใจ']

Graph_CT = [None]*len(Questions)
i = 0
for x,k in zip(Questions, Simplify):
    Graph_CT[i] = Data.groupby(['Q_category',x])['ประทับเวลา'].count().to_frame().reset_index()
    Graph_CT[i].rename(columns = {x:k} , inplace = True)
    Graph_CT[i][k].replace(dict_S4P3, inplace=True)
    Graph_CT[i] = Graph_CT[i].set_index(['Q_category',k]).unstack()
    Graph_CT[i].columns =Graph_CT[i].columns.droplevel(0)
    Graph_CT[i]['Deterrance']=Graph_CT[i].columns.name
    Graph_CT[i] = Graph_CT[i].reset_index().set_index(['Deterrance','Q_category'])
    Graph_CT[i].columns.name=None
    for col in Answer_S4P3:
        if col not in Graph_CT[i].columns:
            Graph_CT[i][col]=0
    i+=1

Graph_CT_data_4 = pd.concat([Graph_CT[0],Graph_CT[1],Graph_CT[2],Graph_CT[3]])
Graph_CT_data_4 = Graph_CT_data_2[Answer_S4P3]
Graph_CT_data_4

100-81 %  80-61 %  60-41 %  40-21 %  \
Deterrance             Q_category                                        
Merger                 high             0.0      NaN      NaN      NaN   
                       low              0.0      2.0      1.0      3.0   
Abuse of Dominance     high             0.0      NaN      NaN      NaN   
                       low              0.0      1.0      1.0      1.0   
Cartels                high             NaN      NaN      0.0      NaN   
                       low              1.0      1.0      0.0      1.0   
Unfair Trade Practices high             NaN      0.0      0.0      1.0   
                       low              1.0      0.0      0.0      2.0   

                                   20-0 %  ไม่ทราบ/ไม่แน่ใจ  
Deterrance             Q_category                            
Merger                 high           3.0               7.0  
                       low            4.0              15.0  
Abuse of Dominance     high           3.0               7.0  
                       low            6.0              16.0  
Cartels                high           3.0               7.0  
                       low            5.0              17.0  
Unfair Trade Practices high           3.0               6.0  
                       low            6.0              16.0

In [111]:
#prob detect and convict

dict_S4P1 = {แ}

Questions = ['ภายใต้สถานการณ์ในปัจจุบัน ท่านคิดว่าข้อความเหล่านี้เป็นจริงมากน้อยเพียงใด  [ผู้กระทำความผิดตามกฎหมายแข่งขันทางการค้ามีโอกาสโดนลงโทษสูง]']
Simplify = ['ผู้กระทำความผิดตามกฎหมายแข่งขันทางการค้ามีโอกาสโดนลงโทษสูง','สำนักงานฯ ใช้เวลาในการสืบสวนสอบสวนและตัดสินคดีนานเกินกว่าที่ควร']
Answer_S3P2 = ['ไม่จริงอย่างยิ่ง','ไม่จริง','จริง','จริงอย่างยิ่ง','เฉยๆ','ไม่สามารถตอบได้']

Graph_CT = [None]*len(Questions)
i = 0
for x,k in zip(Questions, Simplify):
    Graph_CT[i] = Data.groupby(['Q_category',x])['ประทับเวลา'].count().to_frame().reset_index()
    Graph_CT[i].rename(columns = {x:k} , inplace = True)
    #Graph_CT[i][k].replace(dict_S3P2, inplace=True)
    Graph_CT[i] = Graph_CT[i].set_index(['Q_category',k]).unstack()
    Graph_CT[i].columns =Graph_CT[i].columns.droplevel(0)
    Graph_CT[i]['Type']=Graph_CT[i].columns.name
    Graph_CT[i] = Graph_CT[i].reset_index().set_index(['Type','Q_category'])
    Graph_CT[i].columns.name=None
    for col in Answer_S3P2:
        if col not in Graph_CT[i].columns:
            Graph_CT[i][col]=0
    i+=1

Graph_CT_data_3 = pd.concat([Graph_CT[0]])
Graph_CT_data_3 = Graph_CT_data_3[Answer_S3P2]
Graph_CT_data_3

ไม่จริงอย่างยิ่ง  \
Type                                               Q_category                     
ผู้กระทำความผิดตามกฎหมายแข่งขันทางการค้ามีโอกาส... high                     NaN   
                                                   low                      5.0   

                                                               ไม่จริง  จริง  \
Type                                               Q_category                  
ผู้กระทำความผิดตามกฎหมายแข่งขันทางการค้ามีโอกาส... high            1.0   8.0   
                                                   low             6.0   6.0   

                                                               จริงอย่างยิ่ง  \
Type                                               Q_category                  
ผู้กระทำความผิดตามกฎหมายแข่งขันทางการค้ามีโอกาส... high                  1.0   
                                                   low                   1.0   

                                                               เฉยๆ  \
Type                                               Q_category         
ผู้กระทำความผิดตามกฎหมายแข่งขันทางการค้ามีโอกาส... high         NaN   
                                                   low          2.0   

                                                               ไม่สามารถตอบได้  
Type                                               Q_category                   
ผู้กระทำความผิดตามกฎหมายแข่งขันทางการค้ามีโอกาส... high                    NaN  
                                                   low                     5.0

In [18]:
#ตัวอย่าง Cross-tab Attitude 2

import re

dict_S4P1 = {'มีจำนวนมาก':'มาก',
 'มีจำนวนค่อนข้างมาก':'ค่อนข้างมาก',
 'มีจำนวนปานกลาง':'ปานกลาง',
 'มีจำนวนค่อนข้างน้อย':'ค่อนข้างน้อย',
 'มีจำนวนน้อย':'น้อย',
 'ทราบว่าไม่มีเลย':'ไม่มีเลย',
 'ท่านไม่เคยได้รับข้อมูลดังกล่าว':'ไม่เคยได้รับข้อมูลดังกล่าว'}

Questions = [
       'ภายใต้สถานการณ์ในปัจจุบัน ท่านคิดว่าข้อความเหล่านี้เป็นจริงมากน้อยเพียงใด  [สำนักงานฯ ได้รับอิทธิพลจากผู้อื่น/หรืออยู่ภายใต้คำสั่งของหน่วยงานภายนอก เช่น หน่วยงานของรัฐ หน่วยงานเอกชน ผู้มีอำนาจทางการเมือง ฯลฯ]',
       'ภายใต้สถานการณ์ในปัจจุบัน ท่านคิดว่าข้อความเหล่านี้เป็นจริงมากน้อยเพียงใด  [คณะกรรมการแข่งขันฯ มีผลประโยชน์ทับซ้อน (conflict of interest) ส่งผลต่อการตัดสินใจ]',
       'ภายใต้สถานการณ์ในปัจจุบัน ท่านคิดว่าข้อความเหล่านี้เป็นจริงมากน้อยเพียงใด  [ในช่วงของการสืบสวน คณะกรรมการแข่งขันฯ มีอำนาจมากและความสามารถสูงในการสืบค้นและหาข้อมูลที่เกี่ยวข้อง (Investigation power)]',
       'ภายใต้สถานการณ์ในปัจจุบัน ท่านคิดว่าข้อความเหล่านี้เป็นจริงมากน้อยเพียงใด  [สำนักงานฯ มีทรัพยากรบุคคลเพียงพอ ต่อการบังคับใช้กฎหมายอย่างมีประสิทธิภาพ เช่น ต้องดำเนินการตรวจสอบในหลายกรณีพร้อมกันในต่างสถานที่ ฯลฯ]',
       'ภายใต้สถานการณ์ในปัจจุบัน ท่านคิดว่าข้อความเหล่านี้เป็นจริงมากน้อยเพียงใด  [สำนักงานฯ มีทรัพยากรด้านการเงินเพียงพอ ต่อการบังคับใช้กฎหมายอย่างมีประสิทธิภาพ]',
       'ภายใต้สถานการณ์ในปัจจุบัน ท่านคิดว่าข้อความเหล่านี้เป็นจริงมากน้อยเพียงใด  [เจ้าพนักงานมีทักษะและอำนาจทางกฎหมายที่เอื้อให้สามารถทำการตรวจสอบและสอบสวนได้อย่างมีประสิทธิภาพ]',
       'ภายใต้สถานการณ์ในปัจจุบัน ท่านคิดว่าข้อความเหล่านี้เป็นจริงมากน้อยเพียงใด  [ผู้กระทำความผิดตามกฎหมายแข่งขันทางการค้ามีโอกาสโดนลงโทษสูง]',
       'ภายใต้สถานการณ์ในปัจจุบัน ท่านคิดว่าข้อความเหล่านี้เป็นจริงมากน้อยเพียงใด  [สำนักงานฯ มีอำนาจและเครื่องมือในเรื่องการกำหนดค่าปรับ/โทษที่มีประสิทธิภาพ]',
       'ภายใต้สถานการณ์ในปัจจุบัน ท่านคิดว่าข้อความเหล่านี้เป็นจริงมากน้อยเพียงใด  [ระดับของโทษ/ค่าปรับในภาพรวมของกฎหมาย อยู่ในระดับที่สามารถยับยั้งการกระทำที่ขัดต่อกฎหมาย]',
       'ภายใต้สถานการณ์ในปัจจุบัน ท่านคิดว่าข้อความเหล่านี้เป็นจริงมากน้อยเพียงใด  [ระดับของค่าปรับในกฎหมาย อยู่ในระดับที่สามารถยับยั้ง การใช้อำนาจเหนือตลาดอย่างไม่เป็นธรรม (abuse of dominant position)]',
       'ภายใต้สถานการณ์ในปัจจุบัน ท่านคิดว่าข้อความเหล่านี้เป็นจริงมากน้อยเพียงใด  [ระดับของค่าปรับในกฎหมาย อยู่ในระดับที่สามารถยับยั้ง การรวมธุรกิจที่ผิดกฎหมายแข่งขันทางการค้า]',
       'ภายใต้สถานการณ์ในปัจจุบัน ท่านคิดว่าข้อความเหล่านี้เป็นจริงมากน้อยเพียงใด  [ระดับของค่าปรับในกฎหมาย อยู่ในระดับที่สามารถยับยั้ง การปฏิบัติการค้าที่ไม่เป็นธรรม]',
       'ภายใต้สถานการณ์ในปัจจุบัน ท่านคิดว่าข้อความเหล่านี้เป็นจริงมากน้อยเพียงใด  [ระดับของค่าปรับในกฎหมาย อยู่ในระดับที่สามารถยับยั้ง การกระทำผิดฐานตกลงร่วมกัน ผูกขาด ลด และจำกัดการแข่งขัน (cartel & bid-riggings)]',
       'ภายใต้สถานการณ์ในปัจจุบัน ท่านคิดว่าข้อความเหล่านี้เป็นจริงมากน้อยเพียงใด  [สำนักงานฯ ใช้เวลาในการสืบสวนสอบสวนและตัดสินคดีนานเกินกว่าที่ควร]']

Simplify = []
for i in Questions:
    x = re.search("(?<=\[).+?(?=\])", i)
    Simplify.append(x.group())
    #print(x.group())
#print(Simplify)

Answer_S3P2 = ['ไม่จริงอย่างยิ่ง','ไม่จริง','จริง','จริงอย่างยิ่ง','เฉยๆ','ไม่สามารถตอบได้']

col_corp_category = 'ประเภทสถานประกอบการ ในปี พ.ศ. 2562 (ก่อนวิกฤตโควิด)'
Var = col_corp_category

Graph_CT = [None]*len(Questions)
i = 0
for x,k in zip(Questions, Simplify):
    Graph_CT[i] = Data.groupby([Var,x])['ประทับเวลา'].count().to_frame().reset_index()
    Graph_CT[i].rename(columns = {x:k} , inplace = True)
    #Graph_CT[i][k].replace(dict_S3P2, inplace=True)
    Graph_CT[i] = Graph_CT[i].set_index([Var,k]).unstack()
    Graph_CT[i].columns =Graph_CT[i].columns.droplevel(0)
    Graph_CT[i]['Type']=Graph_CT[i].columns.name
    Graph_CT[i] = Graph_CT[i].reset_index().set_index(['Type',Var])
    Graph_CT[i].columns.name=None
    for col in Answer_S3P2:
        if col not in Graph_CT[i].columns:
            Graph_CT[i][col]=0
    i+=1

Graph_CT_data_4 = pd.concat([Graph_CT[0],Graph_CT[1],Graph_CT[2],Graph_CT[3],Graph_CT[4],Graph_CT[5],Graph_CT[6],
                            Graph_CT[7],Graph_CT[8],Graph_CT[9],Graph_CT[10],Graph_CT[11],Graph_CT[12],Graph_CT[13]])
Graph_CT_data_4 = Graph_CT_data_4[Answer_S3P2].replace(np.nan,0)
Graph_CT_data_4

ไม่จริงอย่างยิ่ง  \
Type                                               ประเภทสถานประกอบการ ในปี พ.ศ. 2562 (ก่อนวิกฤตโค...                     
สำนักงานฯ ได้รับอิทธิพลจากผู้อื่น/หรืออยู่ภายใต... Big Corp                                                         2.0   
                                                   SME                                                              0.0   
คณะกรรมการแข่งขันฯ มีผลประโยชน์ทับซ้อน (conflic... Big Corp                                                         1.0   
                                                   SME                                                              0.0   
ในช่วงของการสืบสวน คณะกรรมการแข่งขันฯ มีอำนาจมา... Big Corp                                                         1.0   
                                                   SME                                                              2.0   
สำนักงานฯ มีทรัพยากรบุคคลเพียงพอ ต่อการบังคับใช... Big Corp                                                         1.0   
                                                   SME                                                              2.0   
สำนักงานฯ มีทรัพยากรด้านการเงินเพียงพอ ต่อการบั... Big Corp                                                         1.0   
                                                   SME                                                              1.0   
เจ้าพนักงานมีทักษะและอำนาจทางกฎหมายที่เอื้อให้ส... Big Corp                                                         1.0   
                                                   SME                                                              1.0   
ผู้กระทำความผิดตามกฎหมายแข่งขันทางการค้ามีโอกาส... Big Corp                                                         1.0   
                                                   SME                                                              4.0   
สำนักงานฯ มีอำนาจและเครื่องมือในเรื่องการกำหนดค... Big Corp                                                         1.0   
                                                   SME                                                              1.0   
ระดับของโทษ/ค่าปรับในภาพรวมของกฎหมาย อยู่ในระดั... Big Corp                                                         1.0   
                                                   SME                                                              3.0   
ระดับของค่าปรับในกฎหมาย อยู่ในระดับที่สามารถยับ... Big Corp                                                         1.0   
                                                   SME                                                              3.0   
ระดับของค่าปรับในกฎหมาย อยู่ในระดับที่สามารถยับ... Big Corp                                                         1.0   
                                                   SME                                                              4.0   
ระดับของค่าปรับในกฎหมาย อยู่ในระดับที่สามารถยับ... Big Corp                                                         1.0   
                                                   SME                                                              3.0   
ระดับของค่าปรับในกฎหมาย อยู่ในระดับที่สามารถยับ... Big Corp                                                         2.0   
                                                   SME                                                              2.0   
สำนักงานฯ ใช้เวลาในการสืบสวนสอบสวนและตัดสินคดีน... Big Corp                                                         2.0   
                                                   SME                                                              0.0   

                                                                                                       ไม่จริง  \
Type                                               ประเภทสถานประกอบการ ในปี พ.ศ. 2562 (ก่อนวิกฤตโค...            
สำนักงานฯ ได้รับอิทธิพลจากผู้อื่น/หรืออยู่ภายใต... Big Corp                                                6.0   
                                                   SME                

In [19]:
#Cross-tab Attitude 1

import re

dict_S4P1 = {'มีจำนวนมาก':'มาก',
 'มีจำนวนค่อนข้างมาก':'ค่อนข้างมาก',
 'มีจำนวนปานกลาง':'ปานกลาง',
 'มีจำนวนค่อนข้างน้อย':'ค่อนข้างน้อย',
 'มีจำนวนน้อย':'น้อย',
 'ทราบว่าไม่มีเลย':'ไม่มีเลย',
 'ท่านไม่เคยได้รับข้อมูลดังกล่าว':'ไม่เคยได้รับข้อมูลดังกล่าว'}

Questions = [
       'ท่านคิดเห็นอย่างไรต่อ พระราชบัญญัติการแข่งขันทางการค้า พ.ศ. 2560 และการบังคับใช้โดยสำนักงานคณะกรรมการการแข่งขันทางการค้า  [สำนักงานฯ บังคับใช้กฎหมายอย่างมีประสิทธิภาพในการยับยั้ง *การรวมธุรกิจที่ส่งผลลบต่อการแข่งขันของตลาดอย่างรุนแรง*]',
       'ท่านคิดเห็นอย่างไรต่อ พระราชบัญญัติการแข่งขันทางการค้า พ.ศ. 2560 และการบังคับใช้โดยสำนักงานคณะกรรมการการแข่งขันทางการค้า  [สำนักงานฯ บังคับใช้กฎหมายอย่างมีประสิทธิภาพในการยับยั้ง *การตกลงร่วมกัน ผูกขาด ลด และจำกัดการแข่งขัน*]',
       'ท่านคิดเห็นอย่างไรต่อ พระราชบัญญัติการแข่งขันทางการค้า พ.ศ. 2560 และการบังคับใช้โดยสำนักงานคณะกรรมการการแข่งขันทางการค้า  [สำนักงานฯ บังคับใช้กฎหมายอย่างมีประสิทธิภาพในการยับยั้ง *การใช้อำนาจเหนือตลาดอย่างไม่เป็นธรรม*]',
       'ท่านคิดเห็นอย่างไรต่อ พระราชบัญญัติการแข่งขันทางการค้า พ.ศ. 2560 และการบังคับใช้โดยสำนักงานคณะกรรมการการแข่งขันทางการค้า  [สำนักงานฯ บังคับใช้กฎหมายอย่างมีประสิทธิภาพในการยับยั้ง *การปฏิบัติทางการค้าที่ไม่เป็นธรรม*]',
       'ท่านคิดเห็นอย่างไรต่อ พระราชบัญญัติการแข่งขันทางการค้า พ.ศ. 2560 และการบังคับใช้โดยสำนักงานคณะกรรมการการแข่งขันทางการค้า  [กฎหมาย มาตรา 57 (การปฏิบัติทางการค้าที่ไม่เป็นธรรม) ควรอยู่ถูกบรรจุอยู่ใน พระราชบัญญัติการแข่งขันทางการค้า พ.ศ. 2560 ดังเช่นในปัจจุบัน มากกว่าการบรรจุอยู่ในกฎหมายลำดับรองฉบับอื่น]']

Simplify = []
for i in Questions:
    x = re.search("(?<=\[).+?(?=\])", i)
    Simplify.append(x.group())
    print(x.group())
print(Simplify)

Answer_S3P2 = ['เห็นด้วยอย่างยิ่ง','เห็นด้วย','เฉยๆ','ไม่เห็นด้วย','ไม่เห็นด้วยอย่างยิ่ง','ไม่สามารถตอบได้']

col_corp_category = 'ประเภทสถานประกอบการ ในปี พ.ศ. 2562 (ก่อนวิกฤตโควิด)'
Var = col_corp_category

Graph_CT = [None]*len(Questions)
i = 0
for x,k in zip(Questions, Simplify):
    Graph_CT[i] = Data.groupby([Var,x])['ประทับเวลา'].count().to_frame().reset_index()
    Graph_CT[i].rename(columns = {x:k} , inplace = True)
    #Graph_CT[i][k].replace(dict_S3P2, inplace=True)
    Graph_CT[i] = Graph_CT[i].set_index([Var,k]).unstack()
    Graph_CT[i].columns =Graph_CT[i].columns.droplevel(0)
    Graph_CT[i]['Type']=Graph_CT[i].columns.name
    Graph_CT[i] = Graph_CT[i].reset_index().set_index(['Type',Var])
    Graph_CT[i].columns.name=None
    for col in Answer_S3P2:
        if col not in Graph_CT[i].columns:
            Graph_CT[i][col]=0
    i+=1

Graph_CT_data_5 = pd.concat([Graph_CT[0],Graph_CT[1],Graph_CT[2],Graph_CT[3],Graph_CT[4]])
Graph_CT_data_5 = Graph_CT_data_5[Answer_S3P2].replace(np.nan,0)
Graph_CT_data_5

สำนักงานฯ บังคับใช้กฎหมายอย่างมีประสิทธิภาพในการยับยั้ง *การรวมธุรกิจที่ส่งผลลบต่อการแข่งขันของตลาดอย่างรุนแรง*
สำนักงานฯ บังคับใช้กฎหมายอย่างมีประสิทธิภาพในการยับยั้ง *การตกลงร่วมกัน ผูกขาด ลด และจำกัดการแข่งขัน*
สำนักงานฯ บังคับใช้กฎหมายอย่างมีประสิทธิภาพในการยับยั้ง *การใช้อำนาจเหนือตลาดอย่างไม่เป็นธรรม*
สำนักงานฯ บังคับใช้กฎหมายอย่างมีประสิทธิภาพในการยับยั้ง *การปฏิบัติทางการค้าที่ไม่เป็นธรรม*
กฎหมาย มาตรา 57 (การปฏิบัติทางการค้าที่ไม่เป็นธรรม) ควรอยู่ถูกบรรจุอยู่ใน พระราชบัญญัติการแข่งขันทางการค้า พ.ศ. 2560 ดังเช่นในปัจจุบัน มากกว่าการบรรจุอยู่ในกฎหมายลำดับรองฉบับอื่น
['สำนักงานฯ บังคับใช้กฎหมายอย่างมีประสิทธิภาพในการยับยั้ง *การรวมธุรกิจที่ส่งผลลบต่อการแข่งขันของตลาดอย่างรุนแรง*', 'สำนักงานฯ บังคับใช้กฎหมายอย่างมีประสิทธิภาพในการยับยั้ง *การตกลงร่วมกัน ผูกขาด ลด และจำกัดการแข่งขัน*', 'สำนักงานฯ บังคับใช้กฎหมายอย่างมีประสิทธิภาพในการยับยั้ง *การใช้อำนาจเหนือตลาดอย่างไม่เป็นธรรม*', 'สำนักงานฯ บังคับใช้กฎหมายอย่างมีประสิทธิภาพในการยับยั้ง *การปฏิบัติทางการค้าที่ไม่เป็นธรรม*', 'กฎหมา

เห็นด้วยอย่างยิ่ง  \
Type                                               ประเภทสถานประกอบการ ในปี พ.ศ. 2562 (ก่อนวิกฤตโค...                      
สำนักงานฯ บังคับใช้กฎหมายอย่างมีประสิทธิภาพในกา... Big Corp                                                          4.0   
                                                   SME                                                               3.0   
สำนักงานฯ บังคับใช้กฎหมายอย่างมีประสิทธิภาพในกา... Big Corp                                                          4.0   
                                                   SME                                                               2.0   
สำนักงานฯ บังคับใช้กฎหมายอย่างมีประสิทธิภาพในกา... Big Corp                                                          6.0   
                                                   SME                                                               2.0   
สำนักงานฯ บังคับใช้กฎหมายอย่างมีประสิทธิภาพในกา... Big Corp                                                          6.0   
                                                   SME                                                               3.0   
กฎหมาย มาตรา 57 (การปฏิบัติทางการค้าที่ไม่เป็นธ... Big Corp                                                          6.0   
                                                   SME                                                               2.0   

                                                                                                       เห็นด้วย  \
Type                                               ประเภทสถานประกอบการ ในปี พ.ศ. 2562 (ก่อนวิกฤตโค...             
สำนักงานฯ บังคับใช้กฎหมายอย่างมีประสิทธิภาพในกา... Big Corp                                                14.0   
                                                   SME                                                      7.0   
สำนักงานฯ บังคับใช้กฎหมายอย่างมีประสิทธิภาพในกา... Big Corp                                                11.0   
                                                   SME                                                      8.0   
สำนักงานฯ บังคับใช้กฎหมายอย่างมีประสิทธิภาพในกา... Big Corp                                                10.0   
                                                   SME                                                      7.0   
สำนักงานฯ บังคับใช้กฎหมายอย่างมีประสิทธิภาพในกา... Big Corp                                                10.0   
                                                   SME                                                      6.0   
กฎหมาย มาตรา 57 (การปฏิบัติทางการค้าที่ไม่เป็นธ... Big Corp                                                13.0   
                                                   SME                                                      5.0   

                                                                                                       เฉยๆ  \
Type                                               ประเภทสถานประกอบการ ในปี พ.ศ. 2562 (ก่อนวิกฤตโค...         
สำนักงานฯ บังคับใช้กฎหมายอย่างมีประสิทธิภาพในกา... Big Corp                                             0.0   
                                                   SME                                                  0.0   
สำนักงานฯ บังคับใช้กฎหมายอย่างมีประสิทธิภาพในกา... Big Corp                                             1.0   
                                                   SME                                                  0.0   
สำนักงานฯ บังคับใช้กฎหมายอย่างมีประสิทธิภาพในกา... Big Corp                                             0.0   
                                                   SME                                                  0.0   
สำนักงานฯ บังคับใช้กฎหมายอย่างมีประสิทธิภาพในกา... Big Corp                                             1.0   
                                                   SME                                                  0.0   
กฎหมาย มาตรา 57 (การปฏิบัติทางการค้าที่ไม่เป็นธ... Big Corp                                             1.0   
            

In [20]:
#ตัวอย่าง Cross-tab Attitude 3

import re

dict_S4P1 = {'มีจำนวนมาก':'มาก',
 'มีจำนวนค่อนข้างมาก':'ค่อนข้างมาก',
 'มีจำนวนปานกลาง':'ปานกลาง',
 'มีจำนวนค่อนข้างน้อย':'ค่อนข้างน้อย',
 'มีจำนวนน้อย':'น้อย',
 'ทราบว่าไม่มีเลย':'ไม่มีเลย',
 'ท่านไม่เคยได้รับข้อมูลดังกล่าว':'ไม่เคยได้รับข้อมูลดังกล่าว'}

Questions = [
       'ตามความเห็นของท่าน ท่านคิดว่าการลงโทษหรือเครื่องมือแต่ละประเภทดังตารางด้านล่าง ทั้งประเภทที่มีอยู่ในกฎหมายปัจจุบันและประเภทที่อาจถูกเพิ่มขึ้นมาในอนาคต มีความสำคัญต่อการยับยั้งการกระทำที่ขัดต่อกฎหมายการแข่งขันทางการค้าในประเทศไทยมากน้อยเพียงใด  [การโดนฟ้องร้องเรียกค่าเสียหายจากผู้ประกอบการที่ได้รับผลกระทบจากการกระทำผิด (private damages actions)]',
       'ตามความเห็นของท่าน ท่านคิดว่าการลงโทษหรือเครื่องมือแต่ละประเภทดังตารางด้านล่าง ทั้งประเภทที่มีอยู่ในกฎหมายปัจจุบันและประเภทที่อาจถูกเพิ่มขึ้นมาในอนาคต มีความสำคัญต่อการยับยั้งการกระทำที่ขัดต่อกฎหมายการแข่งขันทางการค้าในประเทศไทยมากน้อยเพียงใด  [โทษปรับบริษัทที่กระทำผิด (fines on firm)]',
       'ตามความเห็นของท่าน ท่านคิดว่าการลงโทษหรือเครื่องมือแต่ละประเภทดังตารางด้านล่าง ทั้งประเภทที่มีอยู่ในกฎหมายปัจจุบันและประเภทที่อาจถูกเพิ่มขึ้นมาในอนาคต มีความสำคัญต่อการยับยั้งการกระทำที่ขัดต่อกฎหมายการแข่งขันทางการค้าในประเทศไทยมากน้อยเพียงใด  [โทษปรับตัวบุคคลที่กระทำผิด (fines on individual)]',
       'ตามความเห็นของท่าน ท่านคิดว่าการลงโทษหรือเครื่องมือแต่ละประเภทดังตารางด้านล่าง ทั้งประเภทที่มีอยู่ในกฎหมายปัจจุบันและประเภทที่อาจถูกเพิ่มขึ้นมาในอนาคต มีความสำคัญต่อการยับยั้งการกระทำที่ขัดต่อกฎหมายการแข่งขันทางการค้าในประเทศไทยมากน้อยเพียงใด  [โทษจำคุกบุคคลที่กระทำผิด (imprisonment)]',
       'ตามความเห็นของท่าน ท่านคิดว่าการลงโทษหรือเครื่องมือแต่ละประเภทดังตารางด้านล่าง ทั้งประเภทที่มีอยู่ในกฎหมายปัจจุบันและประเภทที่อาจถูกเพิ่มขึ้นมาในอนาคต มีความสำคัญต่อการยับยั้งการกระทำที่ขัดต่อกฎหมายการแข่งขันทางการค้าในประเทศไทยมากน้อยเพียงใด  [ความเสียหายต่อชื่อเสียงของบริษัท (reputational damage for the company)**]',
       'ตามความเห็นของท่าน ท่านคิดว่าการลงโทษหรือเครื่องมือแต่ละประเภทดังตารางด้านล่าง ทั้งประเภทที่มีอยู่ในกฎหมายปัจจุบันและประเภทที่อาจถูกเพิ่มขึ้นมาในอนาคต มีความสำคัญต่อการยับยั้งการกระทำที่ขัดต่อกฎหมายการแข่งขันทางการค้าในประเทศไทยมากน้อยเพียงใด  [ความเสียหายต่อชื่อเสียงของบุคคลที่กระทำผิด (reputational damage for the individual)**]',
       'ตามความเห็นของท่าน ท่านคิดว่าการลงโทษหรือเครื่องมือแต่ละประเภทดังตารางด้านล่าง ทั้งประเภทที่มีอยู่ในกฎหมายปัจจุบันและประเภทที่อาจถูกเพิ่มขึ้นมาในอนาคต มีความสำคัญต่อการยับยั้งการกระทำที่ขัดต่อกฎหมายการแข่งขันทางการค้าในประเทศไทยมากน้อยเพียงใด  [คำสั่งปลดกรรมการบริษัท (director disqualification orders)**]',
       'ตามความเห็นของท่าน ท่านคิดว่าการลงโทษหรือเครื่องมือแต่ละประเภทดังตารางด้านล่าง ทั้งประเภทที่มีอยู่ในกฎหมายปัจจุบันและประเภทที่อาจถูกเพิ่มขึ้นมาในอนาคต มีความสำคัญต่อการยับยั้งการกระทำที่ขัดต่อกฎหมายการแข่งขันทางการค้าในประเทศไทยมากน้อยเพียงใด  [การลดหย่อนผ่อน/ยกเว้นโทษสำหรับผู้กระทำผิดที่เป็นพยาน (leniency program)** [หากผู้ร่วมกระทำผิดในกรณีการตกลงร่วมกันผูกขาดได้มาแจ้งให้องค์กรที่กำกับดูแลการแข่งขันทราบถึงการกระทำผิด จะได้รับการยกเว้นหรือลดหย่อนโทษ]]']

Simplify = []
for i in Questions:
    x = re.search("(?<=\[).+?(?=\])", i)
    Simplify.append(x.group())
    print(x.group())
print(Simplify)

Answer_S3P2 = ['มีผลมาก','มีผลปานกลาง','มีผลเล็กน้อย','ไม่มีผลเลย','ไม่ทราบ']

col_corp_category = 'ประเภทสถานประกอบการ ในปี พ.ศ. 2562 (ก่อนวิกฤตโควิด)'
Var = col_corp_category

Graph_CT = [None]*len(Questions)
i = 0
for x,k in zip(Questions, Simplify):
    Graph_CT[i] = Data.groupby([Var,x])['ประทับเวลา'].count().to_frame().reset_index()
    Graph_CT[i].rename(columns = {x:k} , inplace = True)
    #Graph_CT[i][k].replace(dict_S3P2, inplace=True)
    Graph_CT[i] = Graph_CT[i].set_index([Var,k]).unstack()
    Graph_CT[i].columns =Graph_CT[i].columns.droplevel(0)
    Graph_CT[i]['Type']=Graph_CT[i].columns.name
    Graph_CT[i] = Graph_CT[i].reset_index().set_index(['Type',Var])
    Graph_CT[i].columns.name=None
    for col in Answer_S3P2:
        if col not in Graph_CT[i].columns:
            Graph_CT[i][col]=0
    i+=1

Graph_CT_data_6 = pd.concat([Graph_CT[0],Graph_CT[1],Graph_CT[2],Graph_CT[3],Graph_CT[4],Graph_CT[5],Graph_CT[6],
                            Graph_CT[7]])
Graph_CT_data_6 = Graph_CT_data_6[Answer_S3P2].replace(np.nan,0)
Graph_CT_data_6

การโดนฟ้องร้องเรียกค่าเสียหายจากผู้ประกอบการที่ได้รับผลกระทบจากการกระทำผิด (private damages actions)
โทษปรับบริษัทที่กระทำผิด (fines on firm)
โทษปรับตัวบุคคลที่กระทำผิด (fines on individual)
โทษจำคุกบุคคลที่กระทำผิด (imprisonment)
ความเสียหายต่อชื่อเสียงของบริษัท (reputational damage for the company)**
ความเสียหายต่อชื่อเสียงของบุคคลที่กระทำผิด (reputational damage for the individual)**
คำสั่งปลดกรรมการบริษัท (director disqualification orders)**
การลดหย่อนผ่อน/ยกเว้นโทษสำหรับผู้กระทำผิดที่เป็นพยาน (leniency program)** [หากผู้ร่วมกระทำผิดในกรณีการตกลงร่วมกันผูกขาดได้มาแจ้งให้องค์กรที่กำกับดูแลการแข่งขันทราบถึงการกระทำผิด จะได้รับการยกเว้นหรือลดหย่อนโทษ
['การโดนฟ้องร้องเรียกค่าเสียหายจากผู้ประกอบการที่ได้รับผลกระทบจากการกระทำผิด (private damages actions)', 'โทษปรับบริษัทที่กระทำผิด (fines on firm)', 'โทษปรับตัวบุคคลที่กระทำผิด (fines on individual)', 'โทษจำคุกบุคคลที่กระทำผิด (imprisonment)', 'ความเสียหายต่อชื่อเสียงของบริษัท (reputational damage for the company)**', 'ความเสียหายต่อชื่อเ

มีผลมาก  \
Type                                               ประเภทสถานประกอบการ ในปี พ.ศ. 2562 (ก่อนวิกฤตโค...            
การโดนฟ้องร้องเรียกค่าเสียหายจากผู้ประกอบการที่... Big Corp                                               15.0   
                                                   SME                                                     5.0   
โทษปรับบริษัทที่กระทำผิด (fines on firm)           Big Corp                                               12.0   
                                                   SME                                                     3.0   
โทษปรับตัวบุคคลที่กระทำผิด (fines on individual)   Big Corp                                               13.0   
                                                   SME                                                     3.0   
โทษจำคุกบุคคลที่กระทำผิด (imprisonment)            Big Corp                                               14.0   
                                                   SME                                                     3.0   
ความเสียหายต่อชื่อเสียงของบริษัท (reputational ... Big Corp                                               14.0   
                                                   SME                                                     3.0   
ความเสียหายต่อชื่อเสียงของบุคคลที่กระทำผิด (rep... Big Corp                                               14.0   
                                                   SME                                                     4.0   
คำสั่งปลดกรรมการบริษัท (director disqualificati... Big Corp                                               14.0   
                                                   SME                                                     3.0   
การลดหย่อนผ่อน/ยกเว้นโทษสำหรับผู้กระทำผิดที่เป็... Big Corp                                               11.0   
                                                   SME                                                     4.0   

                                                                                                       มีผลปานกลาง  \
Type                                               ประเภทสถานประกอบการ ในปี พ.ศ. 2562 (ก่อนวิกฤตโค...                
การโดนฟ้องร้องเรียกค่าเสียหายจากผู้ประกอบการที่... Big Corp                                                    2.0   
                                                   SME                                                         1.0   
โทษปรับบริษัทที่กระทำผิด (fines on firm)           Big Corp                                                    6.0   
                                                   SME                                                         3.0   
โทษปรับตัวบุคคลที่กระทำผิด (fines on individual)   Big Corp                                                    5.0   
                                                   SME                                                         2.0   
โทษจำคุกบุคคลที่กระทำผิด (imprisonment)            Big Corp                                                    3.0   
                                                   SME                                                         2.0   
ความเสียหายต่อชื่อเสียงของบริษัท (reputational ... Big Corp                                                    6.0   
                                                   SME                                                         4.0   
ความเสียหายต่อชื่อเสียงของบุคคลที่กระทำผิด (rep... Big Corp                                                    5.0   
                                                   SME                                                         2.0   
คำสั่งปลดกรรมการบริษัท (director disqualificati... Big Corp                                                    3.0   
                                                   SME                                                         4.0   
การลดหย่อนผ่อน/ยกเว้นโทษสำหรับผู้กระทำผิดที่เป็... Big Corp                                                    8.0   
                                            

In [21]:
# Cross-tab deter X_2

import re

dict_S4P1 = {'ร้อยละ 0-20':'20-0 %',
 'ร้อยละ 21-40':'40-21 %',
 'ร้อยละ 41-60':'60-41 %',
 'ร้อยละ 61-80':'80-61 %',
 'ร้อยละ 81-100':'100-81 %',
 'ไม่ทราบ/ไม่แน่ใจ':'ไม่ทราบ/ไม่แน่ใจ'}

Questions = [
       '1.2 ในจำนวนแผนการรวมธุรกิจและการรวมธุรกิจทั้งหมดที่ท่านได้ระบุในข้อ 1.1 มีจำนวนแผนการรวมธุรกิจและการรวมธุรกิจที่ถูกยกเลิกหรือปรับเปลี่ยนไปแล้ว เป็นสัดส่วนมากน้อยเพียงใด (ประมาณการ)',
       '2.2 จากข้อมูลที่ท่านทราบ ในจำนวนกิจกรรมหรือแผนกิจกรรมที่อาจเข้าข่ายเป็นการกระทำผิดฐานใช้อำนาจเหนือตลาดอย่างไม่เป็นธรรม ทั้งหมดที่ท่านระบุในข้อ 2.1 มีกิจกรรมหรือแผนกิจกรรมที่ถูกยกเลิกหรือปรับเปลี่ยนไปแล้ว เป็นสัดส่วนมากน้อยเพียงใด (ประมาณการ)',
       '3.2 จากข้อมูลที่ท่านทราบ ในจำนวนความร่วมมือหรือแผนความร่วมมือกับผู้ประกอบการรายอื่นที่อาจเข้าข่ายเป็นการกระทำผิดฐานตกลงร่วมกัน ผูกขาด ลด และจำกัดการแข่งขัน ทั้งหมดที่ท่านระบุในข้อ 3.1 มีความร่วมมือหรือแผนความร่วมมือที่ถูกยกเลิกหรือปรับเปลี่ยนไปแล้ว เป็นสัดส่วนมากน้อยเพียงใด  (ประมาณการ)',
       '4.2 จากข้อมูลที่ท่านทราบ ในจำนวนกิจกรรมหรือแผนกิจกรรมที่อาจเข้าข่ายเป็นการกระทำผิดฐานปฏิบัติทางการค้าที่ไม่เป็นธรรม ทั้งหมดที่ท่านระบุในข้อ 4.4.1 มีกิจกรรมหรือแผนกิจกรรมที่ถูกยกเลิกหรือปรับเปลี่ยนไปแล้ว เป็นสัดส่วนมากน้อยเพียงใด (ประมาณการ)']

Simplify = ['Merger','Abuse of Dominance','Cartels','Unfair Trade Practices']

Answer_S3P2 = ['100-81 %', '80-61 %', '60-41 %', '40-21 %', '20-0 %','ไม่ทราบ/ไม่แน่ใจ']

col_corp_category = 'ประเภทสถานประกอบการ ในปี พ.ศ. 2562 (ก่อนวิกฤตโควิด)'
Var = col_corp_category

Graph_CT = [None]*len(Questions)
i = 0
for x,k in zip(Questions, Simplify):
    Graph_CT[i] = Data.groupby([Var,x])['ประทับเวลา'].count().to_frame().reset_index()
    Graph_CT[i].rename(columns = {x:k} , inplace = True)
    Graph_CT[i][k].replace(dict_S4P1, inplace=True)
    Graph_CT[i] = Graph_CT[i].set_index([Var,k]).unstack()
    Graph_CT[i].columns =Graph_CT[i].columns.droplevel(0)
    Graph_CT[i]['Type']=Graph_CT[i].columns.name
    Graph_CT[i] = Graph_CT[i].reset_index().set_index(['Type',Var])
    Graph_CT[i].columns.name=None
    for col in Answer_S3P2:
        if col not in Graph_CT[i].columns:
            Graph_CT[i][col]=0
    i+=1

Graph_CT_data_7 = pd.concat([Graph_CT[0],Graph_CT[1],Graph_CT[2],Graph_CT[3]])
Graph_CT_data_7 = Graph_CT_data_7[Answer_S3P2].replace(np.nan,0)
Graph_CT_data_7

100-81 %  \
Type                   ประเภทสถานประกอบการ ในปี พ.ศ. 2562 (ก่อนวิกฤตโค...             
Merger                 Big Corp                                                 0.0   
                       SME                                                      0.0   
Abuse of Dominance     Big Corp                                                 0.0   
                       SME                                                      0.0   
Cartels                Big Corp                                                 0.0   
                       SME                                                      1.0   
Unfair Trade Practices Big Corp                                                 0.0   
                       SME                                                      1.0   

                                                                           80-61 %  \
Type                   ประเภทสถานประกอบการ ในปี พ.ศ. 2562 (ก่อนวิกฤตโค...            
Merger                 Big Corp                                                1.0   
                       SME                                                     1.0   
Abuse of Dominance     Big Corp                                                0.0   
                       SME                                                     1.0   
Cartels                Big Corp                                                0.0   
                       SME                                                     1.0   
Unfair Trade Practices Big Corp                                                0.0   
                       SME                                                     0.0   

                                                                           60-41 %  \
Type                   ประเภทสถานประกอบการ ในปี พ.ศ. 2562 (ก่อนวิกฤตโค...            
Merger                 Big Corp                                                1.0   
                       SME                                                     0.0   
Abuse of Dominance     Big Corp                                                1.0   
                       SME                                                     0.0   
Cartels                Big Corp                                                0.0   
                       SME                                                     0.0   
Unfair Trade Practices Big Corp                                                0.0   
                       SME                                                     0.0   

                                                                           40-21 %  \
Type                   ประเภทสถานประกอบการ ในปี พ.ศ. 2562 (ก่อนวิกฤตโค...            
Merger                 Big Corp                                                2.0   
                       SME                                                     1.0   
Abuse of Dominance     Big Corp                                                0.0   
                       SME                                                     1.0   
Cartels                Big Corp                                                1.0   
                       SME                                                     0.0   
Unfair Trade Practices Big Corp                                                2.0   
                       SME                                                     1.0   

                                                                           20-0 %  \
Type                   ประเภทสถานประกอบการ ในปี พ.ศ. 2562 (ก่อนวิกฤตโค...           
Merger                 Big Corp                                               5.0   
                       SME                                                    2.0   
Abuse of Dominance     Big Corp                                               7.0   
                       SME                                                    2.0   
Cartels                Big Corp                                               5.0   
                       SME

In [22]:
# Cross-tab deter 

import re

dict_S4P1 = {'ร้อยละ 0-20':'20-0 %',
 'ร้อยละ 21-40':'40-21 %',
 'ร้อยละ 41-60':'60-41 %',
 'ร้อยละ 61-80':'80-61 %',
 'ร้อยละ 81-100':'100-81 %',
 'ไม่ทราบ/ไม่แน่ใจ':'ไม่ทราบ/ไม่แน่ใจ'}

Questions = [
       '1.3 ในจำนวนแผนการรวมธุรกิจและการรวมธุรกิจทั้งหมดที่ท่านได้ระบุในข้อ 1.1. มีแผนการรวมธุรกิจและการรวมธุรกิจที่ถูกยกเลิกหรือปรับเปลี่ยนไปแล้ว เนื่องจากเหตุผลในตารางต่อไปนี้ เป็นสัดส่วนมากน้อยเพียงใด (ประมาณการ)  [ผู้ประกอบการเกรงว่าจะไม่ได้รับอนุญาตหรือจะได้รับการตั้งเงื่อนไขการรวมธุรกิจที่เข้มงวดจากคณะกรรมการการแข่งขันทางการค้า]',
       '2.3 จากข้อมูลที่ท่านทราบ ในจำนวนกิจกรรมหรือแผนกิจกรรมที่อาจเข้าข่ายเป็นการกระทำผิดฐานใช้อำนาจเหนือตลาดอย่างไม่เป็นธรรม ทั้งหมดที่ท่านระบุในข้อ 2.1 มีกิจกรรมหรือแผนกิจกรรมที่ถูกยกเลิกหรือปรับเปลี่ยนไปแล้ว เนื่องจากเหตุผลในตารางต่อไปนี้ เป็นสัดส่วนมากน้อยเพียงใด (ประมาณการ) [ผู้ประกอบการต้องการป้องกันความเสี่ยงที่อาจจะถูกสอบสวนสืบสวนโดย สำนักงาน กขค. และ/หรือถูกลงโทษตาม พ.ร.บ การแข่งขันทางการค้า]',
       '3.3 จากข้อมูลที่ท่านทราบ ในจำนวนความร่วมมือหรือแผนความร่วมมือกับผู้ประกอบการรายอื่นที่อาจเข้าข่ายเป็นการกระทำผิดฐานตกลงร่วมกัน ผูกขาด ลด และจำกัดการแข่งขัน ทั้งหมดที่ท่านระบุในข้อ 3.1 มีความร่วมมือหรือแผนความร่วมมือที่ถูกยกเลิกหรือปรับเปลี่ยนไปแล้ว เนื่องจากเหตุผลในตารางต่อไปนี้ เป็นสัดส่วนมากน้อยเพียงใด (ประมาณการ) [ผู้ประกอบการต้องการป้องกันความเสี่ยงที่อาจจะถูกสอบสวนสืบสวนโดย สำนักงาน กขค. และ/หรือถูกลงโทษตาม พ.ร.บ การแข่งขันทางการค้า]',
       '4.3 จากข้อมูลที่ท่านทราบ ในจำนวนกิจกรรมหรือแผนกิจกรรมที่อาจเข้าข่ายเป็นการกระทำผิดฐานปฏิบัติทางการค้าที่ไม่เป็นธรรม ทั้งหมดที่ท่านระบุในข้อ 4.1 มีกิจกรรมหรือแผนกิจกรรมที่ถูกล้มเลิกหรือปรับเปลี่ยนไปแล้ว เนื่องจากเหตุผลในตารางต่อไปนี้ เป็นสัดส่วนมากน้อยเพียงใด (ประมาณการ) [ผู้ประกอบการต้องการป้องกันความเสี่ยงที่อาจจะถูกสอบสวนสืบสวนโดย สำนักงาน กขค. และ/หรือถูกลงโทษตาม พ.ร.บ การแข่งขันทางการค้า]']

Simplify = ['Merger','Abuse of Dominance','Cartels','Unfair Trade Practices']

Answer_S3P2 = ['100-81 %', '80-61 %', '60-41 %', '40-21 %', '20-0 %','ไม่ทราบ/ไม่แน่ใจ']

col_corp_category = 'ประเภทสถานประกอบการ ในปี พ.ศ. 2562 (ก่อนวิกฤตโควิด)'
Var = col_corp_category

Graph_CT = [None]*len(Questions)
i = 0
for x,k in zip(Questions, Simplify):
    Graph_CT[i] = Data.groupby([Var,x])['ประทับเวลา'].count().to_frame().reset_index()
    Graph_CT[i].rename(columns = {x:k} , inplace = True)
    Graph_CT[i][k].replace(dict_S4P1, inplace=True)
    Graph_CT[i] = Graph_CT[i].set_index([Var,k]).unstack()
    Graph_CT[i].columns =Graph_CT[i].columns.droplevel(0)
    Graph_CT[i]['Type']=Graph_CT[i].columns.name
    Graph_CT[i] = Graph_CT[i].reset_index().set_index(['Type',Var])
    Graph_CT[i].columns.name=None
    for col in Answer_S3P2:
        if col not in Graph_CT[i].columns:
            Graph_CT[i][col]=0
    i+=1

Graph_CT_data_8 = pd.concat([Graph_CT[0],Graph_CT[1],Graph_CT[2],Graph_CT[3]])
Graph_CT_data_8 = Graph_CT_data_8[Answer_S3P2].replace(np.nan,0)
Graph_CT_data_8

100-81 %  \
Type                   ประเภทสถานประกอบการ ในปี พ.ศ. 2562 (ก่อนวิกฤตโค...             
Merger                 Big Corp                                                 0.0   
                       SME                                                      0.0   
Abuse of Dominance     Big Corp                                                 0.0   
                       SME                                                      0.0   
Cartels                Big Corp                                                 0.0   
                       SME                                                      1.0   
Unfair Trade Practices Big Corp                                                 0.0   
                       SME                                                      1.0   

                                                                           80-61 %  \
Type                   ประเภทสถานประกอบการ ในปี พ.ศ. 2562 (ก่อนวิกฤตโค...            
Merger                 Big Corp                                                1.0   
                       SME                                                     1.0   
Abuse of Dominance     Big Corp                                                2.0   
                       SME                                                     0.0   
Cartels                Big Corp                                                1.0   
                       SME                                                     0.0   
Unfair Trade Practices Big Corp                                                2.0   
                       SME                                                     0.0   

                                                                           60-41 %  \
Type                   ประเภทสถานประกอบการ ในปี พ.ศ. 2562 (ก่อนวิกฤตโค...            
Merger                 Big Corp                                                2.0   
                       SME                                                     0.0   
Abuse of Dominance     Big Corp                                                2.0   
                       SME                                                     0.0   
Cartels                Big Corp                                                1.0   
                       SME                                                     0.0   
Unfair Trade Practices Big Corp                                                1.0   
                       SME                                                     0.0   

                                                                           40-21 %  \
Type                   ประเภทสถานประกอบการ ในปี พ.ศ. 2562 (ก่อนวิกฤตโค...            
Merger                 Big Corp                                                2.0   
                       SME                                                     2.0   
Abuse of Dominance     Big Corp                                                0.0   
                       SME                                                     2.0   
Cartels                Big Corp                                                1.0   
                       SME                                                     2.0   
Unfair Trade Practices Big Corp                                                1.0   
                       SME                                                     2.0   

                                                                           20-0 %  \
Type                   ประเภทสถานประกอบการ ในปี พ.ศ. 2562 (ก่อนวิกฤตโค...           
Merger                 Big Corp                                               5.0   
                       SME                                                    2.0   
Abuse of Dominance     Big Corp                                               5.0   
                       SME                                                    2.0   
Cartels                Big Corp                                               5.0   
                       SME

## Excel

In [114]:
import xlsxwriter
writer = pd.ExcelWriter('Firms_Final3', engine='xlsxwriter')

Summary_Proportion_deter.to_excel(writer, sheet_name='deter')
Summary_Median.to_excel(writer, sheet_name='median')
Summary_Proportion_Fail_Violation.to_excel(writer, sheet_name='fail vio')
Summary_Proportion_31.to_excel(writer, sheet_name='31')
Summary_Proportion_32.to_excel(writer, sheet_name='32')
Summary_Proportion_33.to_excel(writer, sheet_name='33')
Summary_Proportion_34.to_excel(writer, sheet_name='34')
Summary_Proportion_attitude.to_excel(writer, sheet_name='att1')
Summary_Proportion_att2.to_excel(writer, sheet_name='att2')
Summary_Proportion_att3.to_excel(writer, sheet_name='att3')
Graph_CT_data.to_excel(writer, sheet_name='cross-tab1')
Graph_CT_data_2.to_excel(writer, sheet_name='cross-tab2')
Graph_CT_data_4.to_excel(writer, sheet_name='cross-tab4')
Graph_CT_data_3.to_excel(writer, sheet_name='cross-tab3')

writer.save()

In [26]:
import xlsxwriter

writer = pd.ExcelWriter("C:\\Users\\triso\\Downloads\\Firms_Sep2.xlsx", engine='xlsxwriter')

income.to_excel(writer, sheet_name='Firms_size')
Knowledge_q_proportion.to_excel(writer, sheet_name='knowledge_sep_firm_size')
#tabel.to_excel(writer, sheet_name='tabel')
Graph_CT_data_5.to_excel(writer, sheet_name='Att1_firms_size')
Graph_CT_data_4.to_excel(writer, sheet_name='Att2_firms_size')
Graph_CT_data_6.to_excel(writer, sheet_name='Att3')
Graph_CT_data_7.to_excel(writer, sheet_name='deter_X_2')
Graph_CT_data_8.to_excel(writer, sheet_name='deter_X_3')


writer.save()